Wikipedia analysis using Latent Dirichlet Allocation (LDA)
==========================================================

Download Stopwords
------------------

In [ ]:
wget https://raw.githubusercontent.com/peterdalle/svensktext/master/stoppord/stoppord.csv

  

>     --2020-12-04 09:42:29--  https://raw.githubusercontent.com/peterdalle/svensktext/master/stoppord/stoppord.csv
>     Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
>     Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
>     HTTP request sent, awaiting response... 200 OK
>     Length: 1936 (1.9K) [text/plain]
>     Saving to: ‘stoppord.csv’
>
>          0K .                                                     100% 17.9M=0s
>
>     2020-12-04 09:42:29 (17.9 MB/s) - ‘stoppord.csv’ saved [1936/1936]

  

Move to DBFS

In [ ]:
val localpath="file:/databricks/driver/stoppord.csv"
val dir = "dbfs:/datasets/05_lda/"

dbutils.fs.mkdirs(dir)

dbutils.fs.cp(localpath, dir)


  

>     localpath: String = file:/databricks/driver/stoppord.csv
>     dir: String = dbfs:/datasets/05_lda/
>     res26: Boolean = true

In [ ]:
val stopword_path = "dbfs:/datasets/05_lda/stoppord.csv"
display(dbutils.fs.ls(stopword_path))

  

[TABLE]

  

Download the Swedish Wikipedia dump
-----------------------------------

In [ ]:
wget https://dumps.wikimedia.org/svwiki/20201120/svwiki-20201120-pages-articles.xml.bz2

  

>     --2020-11-30 09:16:33--  https://dumps.wikimedia.org/svwiki/20201120/svwiki-20201120-pages-articles.xml.bz2
>     Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.155.106, 2620:0:861:4:208:80:155:106
>     Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.155.106|:443... connected.
>     HTTP request sent, awaiting response... 200 OK
>     Length: 1525903265 (1.4G) [application/octet-stream]
>     Saving to: ‘svwiki-20201120-pages-articles.xml.bz2’
>
>          0K .......... .......... .......... .......... ..........  0%  636K 39m2s
>         50K .......... .......... .......... .......... ..........  0%  636K 39m1s
>        100K .......... .......... .......... .......... ..........  0% 1.45M 31m36s
>        150K .......... .......... .......... .......... ..........  0%  116M 23m45s
>        200K .......... .......... .......... .......... ..........  0% 1.09M 23m27s
>        250K .......... .......... .......... .......... ..........  0% 92.2M 19m35s
>        300K .......... .......... .......... .......... ..........  0% 1.45M 19m11s
>        350K .......... .......... .......... .......... ..........  0%  131M 16m48s
>        400K .......... .......... .......... .......... ..........  0%  160M 14m57s
>        450K .......... .......... .......... .......... ..........  0% 1.12M 15m37s
>        500K .......... .......... .......... .......... ..........  0%  139M 14m13s
>        550K .......... .......... .......... .......... ..........  0%  129M 13m3s
>        600K .......... .......... .......... .......... ..........  0%  231M 12m3s
>        650K .......... .......... .......... .......... ..........  0% 1.49M 12m21s
>        700K .......... .......... .......... .......... ..........  0%  153M 11m32s
>        750K .......... .......... .......... .......... ..........  0%  128M 10m50s
>        800K .......... .......... .......... .......... ..........  0%  148M 10m12s
>        850K .......... .......... .......... .......... ..........  0%  155M 9m39s
>        900K .......... .......... .......... .......... ..........  0%  155M 9m9s
>        950K .......... .......... .......... .......... ..........  0% 1.13M 9m46s
>       1000K .......... .......... .......... .......... ..........  0%  125M 9m18s
>       1050K .......... .......... .......... .......... ..........  0%  168M 8m53s
>       1100K .......... .......... .......... .......... ..........  0%  202M 8m31s
>       1150K .......... .......... .......... .......... ..........  0%  178M 8m10s
>       1200K .......... .......... .......... .......... ..........  0%  186M 7m50s
>       1250K .......... .......... .......... .......... ..........  0%  172M 7m33s
>       1300K .......... .......... .......... .......... ..........  0% 18.3M 7m19s
>       1350K .......... .......... .......... .......... ..........  0% 1.66M 7m34s
>       1400K .......... .......... .......... .......... ..........  0%  131M 7m19s
>       1450K .......... .......... .......... .......... ..........  0%  166M 7m5s
>       1500K .......... .......... .......... .......... ..........  0%  147M 6m51s
>       1550K .......... .......... .......... .......... ..........  0%  125M 6m39s
>       1600K .......... .......... .......... .......... ..........  0%  154M 6m27s
>       1650K .......... .......... .......... .......... ..........  0% 3.33M 6m28s
>       1700K .......... .......... .......... .......... ..........  0% 3.60M 6m29s
>       1750K .......... .......... .......... .......... ..........  0% 2.83M 6m32s
>       1800K .......... .......... .......... .......... ..........  0%  150M 6m22s
>       1850K .......... .......... .......... .......... ..........  0% 3.19M 6m24s
>       1900K .......... .......... .......... .......... ..........  0% 3.47M 6m25s
>       1950K .......... .......... .......... .......... ..........  0% 5.15M 6m22s
>       2000K .......... .......... .......... .......... ..........  0% 3.91M 6m22s
>       2050K .......... .......... .......... .......... ..........  0%  111M 6m13s
>       2100K .......... .......... .......... .......... ..........  0% 3.09M 6m15s
>       2150K .......... .......... .......... .......... ..........  0% 4.13M 6m15s
>       2200K .......... .......... .......... .......... ..........  0% 4.42M 6m14s
>       2250K .......... .......... .......... .......... ..........  0%  158M 6m6s
>       2300K .......... .......... .......... .......... ..........  0% 3.96M 6m6s
>       2350K .......... .......... .......... .......... ..........  0% 2.94M 6m9s
>       2400K .......... .......... .......... .......... ..........  0% 3.54M 6m9s
>       2450K .......... .......... .......... .......... ..........  0% 5.16M 6m8s
>       2500K .......... .......... .......... .......... ..........  0%  129M 6m1s
>       2550K .......... .......... .......... .......... ..........  0% 4.13M 6m0s
>       2600K .......... .......... .......... .......... ..........  0% 3.04M 6m3s
>       2650K .......... .......... .......... .......... ..........  0% 2.89M 6m5s
>       2700K .......... .......... .......... .......... ..........  0%  148M 5m59s
>       2750K .......... .......... .......... .......... ..........  0% 2.94M 6m1s
>       2800K .......... .......... .......... .......... ..........  0% 2.81M 6m4s
>       2850K .......... .......... .......... .......... ..........  0% 3.18M 6m5s
>       2900K .......... .......... .......... .......... ..........  0% 2.94M 6m8s
>       2950K .......... .......... .......... .......... ..........  0%  169M 6m2s
>       3000K .......... .......... .......... .......... ..........  0% 3.03M 6m4s
>       3050K .......... .......... .......... .......... ..........  0% 3.30M 6m5s
>       3100K .......... .......... .......... .......... ..........  0% 4.11M 6m5s
>       3150K .......... .......... .......... .......... ..........  0% 3.04M 6m6s
>       3200K .......... .......... .......... .......... ..........  0%  149M 6m1s
>       3250K .......... .......... .......... .......... ..........  0% 3.15M 6m2s
>       3300K .......... .......... .......... .......... ..........  0% 3.06M 6m4s
>       3350K .......... .......... .......... .......... ..........  0% 3.13M 6m5s
>       3400K .......... .......... .......... .......... ..........  0%  125M 6m0s
>       3450K .......... .......... .......... .......... ..........  0% 3.11M 6m2s
>       3500K .......... .......... .......... .......... ..........  0% 3.92M 6m2s
>       3550K .......... .......... .......... .......... ..........  0% 3.18M 6m3s
>       3600K .......... .......... .......... .......... ..........  0% 3.19M 6m5s
>       3650K .......... .......... .......... .......... ..........  0%  181M 6m0s
>       3700K .......... .......... .......... .......... ..........  0% 3.13M 6m1s
>       3750K .......... .......... .......... .......... ..........  0% 3.38M 6m2s
>       3800K .......... .......... .......... .......... ..........  0% 3.58M 6m2s
>       3850K .......... .......... .......... .......... ..........  0%  155M 5m58s
>       3900K .......... .......... .......... .......... ..........  0% 3.21M 5m59s
>       3950K .......... .......... .......... .......... ..........  0% 3.62M 6m0s
>       4000K .......... .......... .......... .......... ..........  0% 4.40M 5m59s
>       4050K .......... .......... .......... .......... ..........  0% 4.02M 5m59s
>       4100K .......... .......... .......... .......... ..........  0%  146M 5m55s
>       4150K .......... .......... .......... .......... ..........  0% 4.20M 5m55s
>       4200K .......... .......... .......... .......... ..........  0% 3.84M 5m55s
>       4250K .......... .......... .......... .......... ..........  0% 4.71M 5m55s
>       4300K .......... .......... .......... .......... ..........  0%  144M 5m51s
>       4350K .......... .......... .......... .......... ..........  0% 3.87M 5m51s
>       4400K .......... .......... .......... .......... ..........  0% 3.22M 5m52s
>       4450K .......... .......... .......... .......... ..........  0% 3.02M 5m53s
>       4500K .......... .......... .......... .......... ..........  0% 3.98M 5m54s
>       4550K .......... .......... .......... .......... ..........  0%  122M 5m50s
>       4600K .......... .......... .......... .......... ..........  0% 2.95M 5m51s
>       4650K .......... .......... .......... .......... ..........  0% 3.25M 5m52s
>       4700K .......... .......... .......... .......... ..........  0% 3.10M 5m54s
>       4750K .......... .......... .......... .......... ..........  0% 1.94M 5m58s
>       4800K .......... .......... .......... .......... ..........  0%  139M 5m54s
>       4850K .......... .......... .......... .......... ..........  0% 4.61M 5m54s
>       4900K .......... .......... .......... .......... ..........  0% 4.13M 5m54s
>       4950K .......... .......... .......... .......... ..........  0% 3.11M 5m55s
>       5000K .......... .......... .......... .......... ..........  0%  180M 5m51s
>       5050K .......... .......... .......... .......... ..........  0% 4.27M 5m51s
>       5100K .......... .......... .......... .......... ..........  0% 4.26M 5m51s
>       5150K .......... .......... .......... .......... ..........  0% 3.04M 5m52s
>       5200K .......... .......... .......... .......... ..........  0% 6.14M 5m51s
>       5250K .......... .......... .......... .......... ..........  0%  126M 5m48s
>       5300K .......... .......... .......... .......... ..........  0% 3.15M 5m49s
>       5350K .......... .......... .......... .......... ..........  0% 4.92M 5m48s
>       5400K .......... .......... .......... .......... ..........  0% 4.31M 5m48s
>       5450K .......... .......... .......... .......... ..........  0%  124M 5m45s
>       5500K .......... .......... .......... .......... ..........  0% 4.16M 5m45s
>       5550K .......... .......... .......... .......... ..........  0% 3.92M 5m45s
>       5600K .......... .......... .......... .......... ..........  0% 3.63M 5m46s
>       5650K .......... .......... .......... .......... ..........  0% 3.73M 5m46s
>       5700K .......... .......... .......... .......... ..........  0%  159M 5m43s
>       5750K .......... .......... .......... .......... ..........  0% 3.83M 5m44s
>       5800K .......... .......... .......... .......... ..........  0% 3.16M 5m45s
>       5850K .......... .......... .......... .......... ..........  0% 3.65M 5m45s
>       5900K .......... .......... .......... .......... ..........  0%  140M 5m42s
>       5950K .......... .......... .......... .......... ..........  0% 3.06M 5m43s
>       6000K .......... .......... .......... .......... ..........  0% 4.60M 5m43s
>       6050K .......... .......... .......... .......... ..........  0% 4.08M 5m43s
>       6100K .......... .......... .......... .......... ..........  0% 4.14M 5m43s
>       6150K .......... .......... .......... .......... ..........  0%  123M 5m41s
>       6200K .......... .......... .......... .......... ..........  0% 4.11M 5m41s
>       6250K .......... .......... .......... .......... ..........  0% 3.34M 5m41s
>       6300K .......... .......... .......... .......... ..........  0% 5.49M 5m41s
>       6350K .......... .......... .......... .......... ..........  0% 3.28M 5m41s
>       6400K .......... .......... .......... .......... ..........  0%  154M 5m39s
>       6450K .......... .......... .......... .......... ..........  0% 3.41M 5m40s
>       6500K .......... .......... .......... .......... ..........  0% 4.11M 5m40s
>       6550K .......... .......... .......... .......... ..........  0% 3.34M 5m40s
>       6600K .......... .......... .......... .......... ..........  0%  161M 5m38s
>       6650K .......... .......... .......... .......... ..........  0% 1.65M 5m42s
>       6700K .......... .......... .......... .......... ..........  0% 4.39M 5m42s
>       6750K .......... .......... .......... .......... ..........  0% 3.26M 5m43s
>       6800K .......... .......... .......... .......... ..........  0% 3.33M 5m43s
>       6850K .......... .......... .......... .......... ..........  0%  144M 5m41s
>       6900K .......... .......... .......... .......... ..........  0% 3.34M 5m41s
>       6950K .......... .......... .......... .......... ..........  0% 3.35M 5m42s
>       7000K .......... .......... .......... .......... ..........  0% 3.30M 5m43s
>       7050K .......... .......... .......... .......... ..........  0%  182M 5m40s
>       7100K .......... .......... .......... .......... ..........  0% 3.35M 5m41s
>       7150K .......... .......... .......... .......... ..........  0% 4.28M 5m41s
>       7200K .......... .......... .......... .......... ..........  0% 5.78M 5m40s
>       7250K .......... .......... .......... .......... ..........  0% 3.36M 5m41s
>       7300K .......... .......... .......... .......... ..........  0%  154M 5m39s
>       7350K .......... .......... .......... .......... ..........  0% 3.39M 5m39s
>       7400K .......... .......... .......... .......... ..........  0% 3.37M 5m40s
>       7450K .......... .......... .......... .......... ..........  0% 4.40M 5m40s
>       7500K .......... .......... .......... .......... ..........  0%  156M 5m38s
>       7550K .......... .......... .......... .......... ..........  0% 3.44M 5m38s
>       7600K .......... .......... .......... .......... ..........  0% 3.45M 5m39s
>       7650K .......... .......... .......... .......... ..........  0% 3.49M 5m39s
>       7700K .......... .......... .......... .......... ..........  0% 3.49M 5m40s
>       7750K .......... .......... .......... .......... ..........  0%  114M 5m37s
>       7800K .......... .......... .......... .......... ..........  0% 3.42M 5m38s
>       7850K .......... .......... .......... .......... ..........  0%  712K 5m49s
>       7900K .......... .......... .......... .......... ..........  0% 2.18M 5m51s
>       7950K .......... .......... .......... .......... ..........  0% 16.9M 5m49s
>       8000K .......... .......... .......... .......... ..........  0%  167M 5m47s
>       8050K .......... .......... .......... .......... ..........  0% 3.22M 5m48s
>       8100K .......... .......... .......... .......... ..........  0% 3.35M 5m48s
>       8150K .......... .......... .......... .......... ..........  0% 3.64M 5m49s
>       8200K .......... .......... .......... .......... ..........  0%  123M 5m47s
>       8250K .......... .......... .......... .......... ..........  0% 4.12M 5m47s
>       8300K .......... .......... .......... .......... ..........  0% 4.85M 5m46s
>       8350K .......... .......... .......... .......... ..........  0% 3.37M 5m47s
>       8400K .......... .......... .......... .......... ..........  0% 4.76M 5m47s
>       8450K .......... .......... .......... .......... ..........  0%  133M 5m45s
>       8500K .......... .......... .......... .......... ..........  0% 3.65M 5m45s
>       8550K .......... .......... .......... .......... ..........  0% 4.26M 5m45s
>       8600K .......... .......... .......... .......... ..........  0% 3.45M 5m45s
>       8650K .......... .......... .......... .......... ..........  0%  167M 5m43s
>       8700K .......... .......... .......... .......... ..........  0% 3.47M 5m44s
>       8750K .......... .......... .......... .......... ..........  0% 3.48M 5m44s
>       8800K .......... .......... .......... .......... ..........  0% 3.45M 5m44s
>       8850K .......... .......... .......... .......... ..........  0% 3.44M 5m45s
>       8900K .......... .......... .......... .......... ..........  0%  122M 5m43s
>       8950K .......... .......... .......... .......... ..........  0% 4.21M 5m43s
>       9000K .......... .......... .......... .......... ..........  0% 4.50M 5m43s
>       9050K .......... .......... .......... .......... ..........  0% 3.32M 5m43s
>       9100K .......... .......... .......... .......... ..........  0%  141M 5m42s
>       9150K .......... .......... .......... .......... ..........  0% 5.64M 5m41s
>       9200K .......... .......... .......... .......... ..........  0% 3.26M 5m42s
>       9250K .......... .......... .......... .......... ..........  0% 4.49M 5m41s
>       9300K .......... .......... .......... .......... ..........  0% 3.30M 5m42s
>       9350K .......... .......... .......... .......... ..........  0% 87.0M 5m40s
>       9400K .......... .......... .......... .......... ..........  0% 3.37M 5m41s
>       9450K .......... .......... .......... .......... ..........  0% 4.02M 5m41s
>       9500K .......... .......... .......... .......... ..........  0% 3.26M 5m41s
>       9550K .......... .......... .......... .......... ..........  0% 4.00M 5m41s
>       9600K .......... .......... .......... .......... ..........  0%  154M 5m40s
>       9650K .......... .......... .......... .......... ..........  0% 3.80M 5m40s
>       9700K .......... .......... .......... .......... ..........  0% 3.99M 5m40s
>       9750K .......... .......... .......... .......... ..........  0% 3.18M 5m41s
>       9800K .......... .......... .......... .......... ..........  0%  234M 5m39s
>       9850K .......... .......... .......... .......... ..........  0% 3.06M 5m40s
>       9900K .......... .......... .......... .......... ..........  0% 3.33M 5m40s
>       9950K .......... .......... .......... .......... ..........  0% 3.95M 5m40s
>      10000K .......... .......... .......... .......... ..........  0% 3.14M 5m41s
>      10050K .......... .......... .......... .......... ..........  0%  175M 5m39s
>      10100K .......... .......... .......... .......... ..........  0% 3.24M 5m40s
>      10150K .......... .......... .......... .......... ..........  0% 3.06M 5m40s
>      10200K .......... .......... .......... .......... ..........  0% 3.21M 5m41s
>      10250K .......... .......... .......... .......... ..........  0% 99.8M 5m39s
>      10300K .......... .......... .......... .......... ..........  0% 3.23M 5m40s
>      10350K .......... .......... .......... .......... ..........  0% 3.26M 5m40s
>      10400K .......... .......... .......... .......... ..........  0% 3.14M 5m41s
>      10450K .......... .......... .......... .......... ..........  0% 3.08M 5m41s
>      10500K .......... .......... .......... .......... ..........  0%  146M 5m40s
>      10550K .......... .......... .......... .......... ..........  0% 3.11M 5m40s
>      10600K .......... .......... .......... .......... ..........  0% 3.21M 5m41s
>      10650K .......... .......... .......... .......... ..........  0% 4.03M 5m41s
>      10700K .......... .......... .......... .......... ..........  0%  125M 5m39s
>      10750K .......... .......... .......... .......... ..........  0% 4.31M 5m39s
>      10800K .......... .......... .......... .......... ..........  0% 3.16M 5m40s
>      10850K .......... .......... .......... .......... ..........  0% 3.21M 5m40s
>      10900K .......... .......... .......... .......... ..........  0% 3.19M 5m41s
>      10950K .......... .......... .......... .......... ..........  0%  136M 5m39s
>      11000K .......... .......... .......... .......... ..........  0% 3.18M 5m40s
>      11050K .......... .......... .......... .......... ..........  0% 3.19M 5m40s
>      11100K .......... .......... .......... .......... ..........  0% 5.20M 5m40s
>      11150K .......... .......... .......... .......... ..........  0% 4.31M 5m40s
>      11200K .......... .......... .......... .......... ..........  0%  131M 5m38s
>      11250K .......... .......... .......... .......... ..........  0% 3.20M 5m39s
>      11300K .......... .......... .......... .......... ..........  0% 3.07M 5m40s
>      11350K .......... .......... .......... .......... ..........  0% 3.16M 5m40s
>      11400K .......... .......... .......... .......... ..........  0%  136M 5m39s
>      11450K .......... .......... .......... .......... ..........  0% 4.18M 5m39s
>      11500K .......... .......... .......... .......... ..........  0% 3.91M 5m39s
>      11550K .......... .......... .......... .......... ..........  0% 4.12M 5m39s
>      11600K .......... .......... .......... .......... ..........  0% 4.41M 5m39s
>      11650K .......... .......... .......... .......... ..........  0%  135M 5m37s
>      11700K .......... .......... .......... .......... ..........  0% 3.17M 5m38s
>      11750K .......... .......... .......... .......... ..........  0% 3.20M 5m38s
>      11800K .......... .......... .......... .......... ..........  0% 3.24M 5m39s
>      11850K .......... .......... .......... .......... ..........  0%  197M 5m37s
>      11900K .......... .......... .......... .......... ..........  0% 3.26M 5m38s
>      11950K .......... .......... .......... .......... ..........  0% 4.31M 5m38s
>      12000K .......... .......... .......... .......... ..........  0% 4.12M 5m38s
>      12050K .......... .......... .......... .......... ..........  0% 4.00M 5m38s
>      12100K .......... .......... .......... .......... ..........  0%  149M 5m36s
>      12150K .......... .......... .......... .......... ..........  0% 3.35M 5m37s
>      12200K .......... .......... .......... .......... ..........  0% 4.87M 5m37s
>      12250K .......... .......... .......... .......... ..........  0% 3.28M 5m37s
>      12300K .......... .......... .......... .......... ..........  0%  196M 5m36s
>      12350K .......... .......... .......... .......... ..........  0% 3.22M 5m36s
>      12400K .......... .......... .......... .......... ..........  0% 3.59M 5m36s
>      12450K .......... .......... .......... .......... ..........  0% 3.13M 5m37s
>      12500K .......... .......... .......... .......... ..........  0% 4.09M 5m37s
>      12550K .......... .......... .......... .......... ..........  0%  114M 5m36s
>      12600K .......... .......... .......... .......... ..........  0% 4.58M 5m36s
>      12650K .......... .......... .......... .......... ..........  0% 4.27M 5m36s
>      12700K .......... .......... .......... .......... ..........  0% 3.15M 5m36s
>      12750K .......... .......... .......... .......... ..........  0% 3.18M 5m36s
>      12800K .......... .......... .......... .......... ..........  0%  180M 5m35s
>      12850K .......... .......... .......... .......... ..........  0% 3.14M 5m36s
>      12900K .......... .......... .......... .......... ..........  0% 3.18M 5m36s
>      12950K .......... .......... .......... .......... ..........  0% 3.34M 5m36s
>      13000K .......... .......... .......... .......... ..........  0%  161M 5m35s
>      13050K .......... .......... .......... .......... ..........  0% 3.95M 5m35s
>      13100K .......... .......... .......... .......... ..........  0% 4.23M 5m35s
>      13150K .......... .......... .......... .......... ..........  0% 3.19M 5m36s
>      13200K .......... .......... .......... .......... ..........  0% 3.15M 5m36s
>      13250K .......... .......... .......... .......... ..........  0%  138M 5m35s
>      13300K .......... .......... .......... .......... ..........  0% 3.22M 5m35s
>      13350K .......... .......... .......... .......... ..........  0% 3.27M 5m36s
>      13400K .......... .......... .......... .......... ..........  0% 3.25M 5m36s
>      13450K .......... .......... .......... .......... ..........  0%  141M 5m35s
>      13500K .......... .......... .......... .......... ..........  0% 4.49M 5m35s
>      13550K .......... .......... .......... .......... ..........  0% 3.24M 5m35s
>      13600K .......... .......... .......... .......... ..........  0% 3.25M 5m36s
>      13650K .......... .......... .......... .......... ..........  0% 3.28M 5m36s
>      13700K .......... .......... .......... .......... ..........  0%  144M 5m35s
>      13750K .......... .......... .......... .......... ..........  0% 3.31M 5m35s
>      13800K .......... .......... .......... .......... ..........  0% 3.68M 5m35s
>      13850K .......... .......... .......... .......... ..........  0% 5.33M 5m35s
>      13900K .......... .......... .......... .......... ..........  0%  165M 5m34s
>      13950K .......... .......... .......... .......... ..........  0% 3.15M 5m34s
>      14000K .......... .......... .......... .......... ..........  0% 2.97M 5m35s
>      14050K .......... .......... .......... .......... ..........  0% 3.11M 5m35s
>      14100K .......... .......... .......... .......... ..........  0% 3.16M 5m36s
>      14150K .......... .......... .......... .......... ..........  0%  145M 5m35s
>      14200K .......... .......... .......... .......... ..........  0% 3.09M 5m35s
>      14250K .......... .......... .......... .......... ..........  0% 3.16M 5m35s
>      14300K .......... .......... .......... .......... ..........  0% 3.18M 5m36s
>      14350K .......... .......... .......... .......... ..........  0% 3.15M 5m36s
>      14400K .......... .......... .......... .......... ..........  0%  188M 5m35s
>      14450K .......... .......... .......... .......... ..........  0% 3.18M 5m36s
>      14500K .......... .......... .......... .......... ..........  0% 5.12M 5m35s
>      14550K .......... .......... .......... .......... ..........  0% 3.17M 5m36s
>      14600K .......... .......... .......... .......... ..........  0%  138M 5m35s
>      14650K .......... .......... .......... .......... ..........  0% 3.22M 5m35s
>      14700K .......... .......... .......... .......... ..........  0% 3.24M 5m35s
>      14750K .......... .......... .......... .......... ..........  0% 3.28M 5m36s
>      14800K .......... .......... .......... .......... ..........  0% 3.26M 5m36s
>      14850K .......... .......... .......... .......... ..........  0%  128M 5m35s
>      14900K .......... .......... .......... .......... ..........  1% 4.01M 5m35s
>      14950K .......... .......... .......... .......... ..........  1% 3.15M 5m35s
>      15000K .......... .......... .......... .......... ..........  1% 4.21M 5m35s
>      15050K .......... .......... .......... .......... ..........  1%  189M 5m34s
>      15100K .......... .......... .......... .......... ..........  1% 4.26M 5m34s
>      15150K .......... .......... .......... .......... ..........  1% 3.17M 5m35s
>      15200K .......... .......... .......... .......... ..........  1% 3.20M 5m35s
>      15250K .......... .......... .......... .......... ..........  1% 3.68M 5m35s
>      15300K .......... .......... .......... .......... ..........  1%  185M 5m34s
>      15350K .......... .......... .......... .......... ..........  1% 4.88M 5m34s
>      15400K .......... .......... .......... .......... ..........  1% 4.48M 5m34s
>      15450K .......... .......... .......... .......... ..........  1% 3.80M 5m34s
>      15500K .......... .......... .......... .......... ..........  1%  165M 5m33s
>      15550K .......... .......... .......... .......... ..........  1% 4.16M 5m33s
>
>     *** WARNING: skipped 2292666 bytes of output ***
>
>     1474000K .......... .......... .......... .......... .......... 98% 4.32M 4s
>     1474050K .......... .......... .......... .......... .......... 98%  140M 4s
>     1474100K .......... .......... .......... .......... .......... 98% 4.11M 4s
>     1474150K .......... .......... .......... .......... .......... 98% 3.05M 4s
>     1474200K .......... .......... .......... .......... .......... 98% 3.47M 4s
>     1474250K .......... .......... .......... .......... .......... 98%  124M 4s
>     1474300K .......... .......... .......... .......... .......... 98% 3.21M 4s
>     1474350K .......... .......... .......... .......... .......... 98% 3.16M 4s
>     1474400K .......... .......... .......... .......... .......... 98% 3.18M 4s
>     1474450K .......... .......... .......... .......... .......... 98% 3.65M 3s
>     1474500K .......... .......... .......... .......... .......... 98%  127M 3s
>     1474550K .......... .......... .......... .......... .......... 98% 3.19M 3s
>     1474600K .......... .......... .......... .......... .......... 98% 3.24M 3s
>     1474650K .......... .......... .......... .......... .......... 98% 3.23M 3s
>     1474700K .......... .......... .......... .......... .......... 98%  157M 3s
>     1474750K .......... .......... .......... .......... .......... 98% 3.24M 3s
>     1474800K .......... .......... .......... .......... .......... 98% 3.26M 3s
>     1474850K .......... .......... .......... .......... .......... 98% 3.28M 3s
>     1474900K .......... .......... .......... .......... .......... 98% 3.28M 3s
>     1474950K .......... .......... .......... .......... .......... 98%  126M 3s
>     1475000K .......... .......... .......... .......... .......... 98% 3.54M 3s
>     1475050K .......... .......... .......... .......... .......... 98% 3.57M 3s
>     1475100K .......... .......... .......... .......... .......... 98% 4.01M 3s
>     1475150K .......... .......... .......... .......... .......... 98% 2.86M 3s
>     1475200K .......... .......... .......... .......... .......... 99%  142M 3s
>     1475250K .......... .......... .......... .......... .......... 99% 3.38M 3s
>     1475300K .......... .......... .......... .......... .......... 99% 3.02M 3s
>     1475350K .......... .......... .......... .......... .......... 99% 3.74M 3s
>     1475400K .......... .......... .......... .......... .......... 99%  108M 3s
>     1475450K .......... .......... .......... .......... .......... 99% 4.64M 3s
>     1475500K .......... .......... .......... .......... .......... 99% 3.24M 3s
>     1475550K .......... .......... .......... .......... .......... 99% 2.98M 3s
>     1475600K .......... .......... .......... .......... .......... 99% 3.06M 3s
>     1475650K .......... .......... .......... .......... .......... 99%  134M 3s
>     1475700K .......... .......... .......... .......... .......... 99% 4.15M 3s
>     1475750K .......... .......... .......... .......... .......... 99% 3.80M 3s
>     1475800K .......... .......... .......... .......... .......... 99% 4.16M 3s
>     1475850K .......... .......... .......... .......... .......... 99%  154M 3s
>     1475900K .......... .......... .......... .......... .......... 99% 3.88M 3s
>     1475950K .......... .......... .......... .......... .......... 99% 4.28M 3s
>     1476000K .......... .......... .......... .......... .......... 99% 3.10M 3s
>     1476050K .......... .......... .......... .......... .......... 99% 4.47M 3s
>     1476100K .......... .......... .......... .......... .......... 99%  165M 3s
>     1476150K .......... .......... .......... .......... .......... 99% 3.14M 3s
>     1476200K .......... .......... .......... .......... .......... 99% 3.62M 3s
>     1476250K .......... .......... .......... .......... .......... 99% 3.84M 3s
>     1476300K .......... .......... .......... .......... .......... 99%  209M 3s
>     1476350K .......... .......... .......... .......... .......... 99% 4.15M 3s
>     1476400K .......... .......... .......... .......... .......... 99% 3.13M 3s
>     1476450K .......... .......... .......... .......... .......... 99% 4.19M 3s
>     1476500K .......... .......... .......... .......... .......... 99% 3.02M 3s
>     1476550K .......... .......... .......... .......... .......... 99%  114M 3s
>     1476600K .......... .......... .......... .......... .......... 99% 2.70M 3s
>     1476650K .......... .......... .......... .......... .......... 99% 2.84M 3s
>     1476700K .......... .......... .......... .......... .......... 99% 3.62M 3s
>     1476750K .......... .......... .......... .......... .......... 99% 2.98M 3s
>     1476800K .......... .......... .......... .......... .......... 99%  123M 3s
>     1476850K .......... .......... .......... .......... .......... 99% 2.98M 3s
>     1476900K .......... .......... .......... .......... .......... 99% 2.98M 3s
>     1476950K .......... .......... .......... .......... .......... 99% 4.13M 3s
>     1477000K .......... .......... .......... .......... .......... 99%  144M 3s
>     1477050K .......... .......... .......... .......... .......... 99% 4.27M 3s
>     1477100K .......... .......... .......... .......... .......... 99% 3.65M 3s
>     1477150K .......... .......... .......... .......... .......... 99% 3.99M 3s
>     1477200K .......... .......... .......... .......... .......... 99% 2.76M 3s
>     1477250K .......... .......... .......... .......... .......... 99%  145M 3s
>     1477300K .......... .......... .......... .......... .......... 99% 4.00M 3s
>     1477350K .......... .......... .......... .......... .......... 99% 5.87M 3s
>     1477400K .......... .......... .......... .......... .......... 99% 3.71M 3s
>     1477450K .......... .......... .......... .......... .......... 99%  194M 3s
>     1477500K .......... .......... .......... .......... .......... 99% 3.75M 3s
>     1477550K .......... .......... .......... .......... .......... 99% 4.40M 3s
>     1477600K .......... .......... .......... .......... .......... 99% 3.71M 3s
>     1477650K .......... .......... .......... .......... .......... 99% 3.71M 3s
>     1477700K .......... .......... .......... .......... .......... 99%  140M 3s
>     1477750K .......... .......... .......... .......... .......... 99% 4.33M 3s
>     1477800K .......... .......... .......... .......... .......... 99% 3.01M 3s
>     1477850K .......... .......... .......... .......... .......... 99% 4.95M 3s
>     1477900K .......... .......... .......... .......... .......... 99%  149M 3s
>     1477950K .......... .......... .......... .......... .......... 99% 3.02M 3s
>     1478000K .......... .......... .......... .......... .......... 99% 3.08M 3s
>     1478050K .......... .......... .......... .......... .......... 99% 3.11M 3s
>     1478100K .......... .......... .......... .......... .......... 99% 3.03M 3s
>     1478150K .......... .......... .......... .......... .......... 99%  123M 3s
>     1478200K .......... .......... .......... .......... .......... 99% 3.01M 3s
>     1478250K .......... .......... .......... .......... .......... 99% 3.18M 3s
>     1478300K .......... .......... .......... .......... .......... 99% 2.93M 3s
>     1478350K .......... .......... .......... .......... .......... 99% 3.03M 3s
>     1478400K .......... .......... .......... .......... .......... 99%  144M 3s
>     1478450K .......... .......... .......... .......... .......... 99% 3.81M 3s
>     1478500K .......... .......... .......... .......... .......... 99% 3.10M 3s
>     1478550K .......... .......... .......... .......... .......... 99% 3.08M 3s
>     1478600K .......... .......... .......... .......... .......... 99%  132M 3s
>     1478650K .......... .......... .......... .......... .......... 99% 3.18M 3s
>     1478700K .......... .......... .......... .......... .......... 99% 3.14M 3s
>     1478750K .......... .......... .......... .......... .......... 99% 3.16M 3s
>     1478800K .......... .......... .......... .......... .......... 99% 3.06M 3s
>     1478850K .......... .......... .......... .......... .......... 99%  167M 3s
>     1478900K .......... .......... .......... .......... .......... 99% 4.10M 3s
>     1478950K .......... .......... .......... .......... .......... 99% 2.89M 2s
>     1479000K .......... .......... .......... .......... .......... 99% 2.96M 2s
>     1479050K .......... .......... .......... .......... .......... 99%  143M 2s
>     1479100K .......... .......... .......... .......... .......... 99% 3.61M 2s
>     1479150K .......... .......... .......... .......... .......... 99% 3.86M 2s
>     1479200K .......... .......... .......... .......... .......... 99% 4.52M 2s
>     1479250K .......... .......... .......... .......... .......... 99% 3.61M 2s
>     1479300K .......... .......... .......... .......... .......... 99%  143M 2s
>     1479350K .......... .......... .......... .......... .......... 99% 4.01M 2s
>     1479400K .......... .......... .......... .......... .......... 99% 3.20M 2s
>     1479450K .......... .......... .......... .......... .......... 99% 3.88M 2s
>     1479500K .......... .......... .......... .......... .......... 99%  208M 2s
>     1479550K .......... .......... .......... .......... .......... 99% 3.73M 2s
>     1479600K .......... .......... .......... .......... .......... 99% 3.11M 2s
>     1479650K .......... .......... .......... .......... .......... 99% 4.10M 2s
>     1479700K .......... .......... .......... .......... .......... 99% 3.15M 2s
>     1479750K .......... .......... .......... .......... .......... 99%  140M 2s
>     1479800K .......... .......... .......... .......... .......... 99% 3.02M 2s
>     1479850K .......... .......... .......... .......... .......... 99% 3.05M 2s
>     1479900K .......... .......... .......... .......... .......... 99% 3.01M 2s
>     1479950K .......... .......... .......... .......... .......... 99% 3.58M 2s
>     1480000K .......... .......... .......... .......... .......... 99%  148M 2s
>     1480050K .......... .......... .......... .......... .......... 99% 2.95M 2s
>     1480100K .......... .......... .......... .......... .......... 99% 3.23M 2s
>     1480150K .......... .......... .......... .......... .......... 99% 3.26M 2s
>     1480200K .......... .......... .......... .......... .......... 99%  216M 2s
>     1480250K .......... .......... .......... .......... .......... 99% 4.17M 2s
>     1480300K .......... .......... .......... .......... .......... 99% 4.15M 2s
>     1480350K .......... .......... .......... .......... .......... 99% 3.64M 2s
>     1480400K .......... .......... .......... .......... .......... 99% 2.49M 2s
>     1480450K .......... .......... .......... .......... .......... 99%  131M 2s
>     1480500K .......... .......... .......... .......... .......... 99% 3.71M 2s
>     1480550K .......... .......... .......... .......... .......... 99% 3.16M 2s
>     1480600K .......... .......... .......... .......... .......... 99% 3.05M 2s
>     1480650K .......... .......... .......... .......... .......... 99%  160M 2s
>     1480700K .......... .......... .......... .......... .......... 99% 3.54M 2s
>     1480750K .......... .......... .......... .......... .......... 99% 2.00M 2s
>     1480800K .......... .......... .......... .......... .......... 99% 3.49M 2s
>     1480850K .......... .......... .......... .......... .......... 99% 2.98M 2s
>     1480900K .......... .......... .......... .......... .......... 99%  137M 2s
>     1480950K .......... .......... .......... .......... .......... 99% 3.82M 2s
>     1481000K .......... .......... .......... .......... .......... 99% 4.41M 2s
>     1481050K .......... .......... .......... .......... .......... 99% 5.45M 2s
>     1481100K .......... .......... .......... .......... .......... 99%  189M 2s
>     1481150K .......... .......... .......... .......... .......... 99% 3.35M 2s
>     1481200K .......... .......... .......... .......... .......... 99% 4.40M 2s
>     1481250K .......... .......... .......... .......... .......... 99% 4.99M 2s
>     1481300K .......... .......... .......... .......... .......... 99% 3.61M 2s
>     1481350K .......... .......... .......... .......... .......... 99%  123M 2s
>     1481400K .......... .......... .......... .......... .......... 99% 4.13M 2s
>     1481450K .......... .......... .......... .......... .......... 99% 4.15M 2s
>     1481500K .......... .......... .......... .......... .......... 99% 4.17M 2s
>     1481550K .......... .......... .......... .......... .......... 99% 2.08M 2s
>     1481600K .......... .......... .......... .......... .......... 99%  146M 2s
>     1481650K .......... .......... .......... .......... .......... 99% 3.60M 2s
>     1481700K .......... .......... .......... .......... .......... 99% 3.55M 2s
>     1481750K .......... .......... .......... .......... .......... 99% 4.10M 2s
>     1481800K .......... .......... .......... .......... .......... 99%  135M 2s
>     1481850K .......... .......... .......... .......... .......... 99% 3.69M 2s
>     1481900K .......... .......... .......... .......... .......... 99% 3.60M 2s
>     1481950K .......... .......... .......... .......... .......... 99% 3.05M 2s
>     1482000K .......... .......... .......... .......... .......... 99% 3.03M 2s
>     1482050K .......... .......... .......... .......... .......... 99%  122M 2s
>     1482100K .......... .......... .......... .......... .......... 99% 2.85M 2s
>     1482150K .......... .......... .......... .......... .......... 99% 2.97M 2s
>     1482200K .......... .......... .......... .......... .......... 99% 2.93M 2s
>     1482250K .......... .......... .......... .......... .......... 99%  177M 2s
>     1482300K .......... .......... .......... .......... .......... 99% 4.90M 2s
>     1482350K .......... .......... .......... .......... .......... 99% 3.44M 2s
>     1482400K .......... .......... .......... .......... .......... 99% 3.91M 2s
>     1482450K .......... .......... .......... .......... .......... 99% 2.90M 2s
>     1482500K .......... .......... .......... .......... .......... 99%  139M 2s
>     1482550K .......... .......... .......... .......... .......... 99% 2.97M 2s
>     1482600K .......... .......... .......... .......... .......... 99% 3.02M 2s
>     1482650K .......... .......... .......... .......... .......... 99% 3.40M 2s
>     1482700K .......... .......... .......... .......... .......... 99%  147M 2s
>     1482750K .......... .......... .......... .......... .......... 99% 3.87M 2s
>     1482800K .......... .......... .......... .......... .......... 99% 5.92M 2s
>     1482850K .......... .......... .......... .......... .......... 99% 2.90M 2s
>     1482900K .......... .......... .......... .......... .......... 99% 3.98M 2s
>     1482950K .......... .......... .......... .......... .......... 99%  134M 2s
>     1483000K .......... .......... .......... .......... .......... 99% 4.82M 2s
>     1483050K .......... .......... .......... .......... .......... 99% 4.05M 2s
>     1483100K .......... .......... .......... .......... .......... 99% 3.63M 2s
>     1483150K .......... .......... .......... .......... .......... 99% 3.52M 2s
>     1483200K .......... .......... .......... .......... .......... 99%  154M 2s
>     1483250K .......... .......... .......... .......... .......... 99% 3.95M 2s
>     1483300K .......... .......... .......... .......... .......... 99% 3.99M 2s
>     1483350K .......... .......... .......... .......... .......... 99% 3.32M 2s
>     1483400K .......... .......... .......... .......... .......... 99%  147M 1s
>     1483450K .......... .......... .......... .......... .......... 99% 5.69M 1s
>     1483500K .......... .......... .......... .......... .......... 99% 3.62M 1s
>     1483550K .......... .......... .......... .......... .......... 99% 2.80M 1s
>     1483600K .......... .......... .......... .......... .......... 99% 4.37M 1s
>     1483650K .......... .......... .......... .......... .......... 99%  124M 1s
>     1483700K .......... .......... .......... .......... .......... 99% 3.03M 1s
>     1483750K .......... .......... .......... .......... .......... 99% 2.96M 1s
>     1483800K .......... .......... .......... .......... .......... 99% 3.96M 1s
>     1483850K .......... .......... .......... .......... .......... 99%  161M 1s
>     1483900K .......... .......... .......... .......... .......... 99% 3.05M 1s
>     1483950K .......... .......... .......... .......... .......... 99% 3.07M 1s
>     1484000K .......... .......... .......... .......... .......... 99% 3.14M 1s
>     1484050K .......... .......... .......... .......... .......... 99% 3.14M 1s
>     1484100K .......... .......... .......... .......... .......... 99%  119M 1s
>     1484150K .......... .......... .......... .......... .......... 99% 2.96M 1s
>     1484200K .......... .......... .......... .......... .......... 99% 3.05M 1s
>     1484250K .......... .......... .......... .......... .......... 99% 3.05M 1s
>     1484300K .......... .......... .......... .......... .......... 99%  154M 1s
>     1484350K .......... .......... .......... .......... .......... 99% 2.74M 1s
>     1484400K .......... .......... .......... .......... .......... 99% 3.04M 1s
>     1484450K .......... .......... .......... .......... .......... 99% 3.06M 1s
>     1484500K .......... .......... .......... .......... .......... 99% 2.99M 1s
>     1484550K .......... .......... .......... .......... .......... 99%  110M 1s
>     1484600K .......... .......... .......... .......... .......... 99% 2.99M 1s
>     1484650K .......... .......... .......... .......... .......... 99% 3.04M 1s
>     1484700K .......... .......... .......... .......... .......... 99% 4.33M 1s
>     1484750K .......... .......... .......... .......... .......... 99% 3.58M 1s
>     1484800K .......... .......... .......... .......... .......... 99%  142M 1s
>     1484850K .......... .......... .......... .......... .......... 99% 3.95M 1s
>     1484900K .......... .......... .......... .......... .......... 99% 4.10M 1s
>     1484950K .......... .......... .......... .......... .......... 99% 3.15M 1s
>     1485000K .......... .......... .......... .......... .......... 99%  128M 1s
>     1485050K .......... .......... .......... .......... .......... 99% 4.71M 1s
>     1485100K .......... .......... .......... .......... .......... 99% 3.68M 1s
>     1485150K .......... .......... .......... .......... .......... 99% 2.96M 1s
>     1485200K .......... .......... .......... .......... .......... 99% 2.82M 1s
>     1485250K .......... .......... .......... .......... .......... 99%  133M 1s
>     1485300K .......... .......... .......... .......... .......... 99% 2.92M 1s
>     1485350K .......... .......... .......... .......... .......... 99% 2.75M 1s
>     1485400K .......... .......... .......... .......... .......... 99% 2.80M 1s
>     1485450K .......... .......... .......... .......... .......... 99%  156M 1s
>     1485500K .......... .......... .......... .......... .......... 99% 2.96M 1s
>     1485550K .......... .......... .......... .......... .......... 99% 3.49M 1s
>     1485600K .......... .......... .......... .......... .......... 99% 2.66M 1s
>     1485650K .......... .......... .......... .......... .......... 99% 3.72M 1s
>     1485700K .......... .......... .......... .......... .......... 99%  142M 1s
>     1485750K .......... .......... .......... .......... .......... 99% 4.77M 1s
>     1485800K .......... .......... .......... .......... .......... 99% 3.85M 1s
>     1485850K .......... .......... .......... .......... .......... 99% 3.88M 1s
>     1485900K .......... .......... .......... .......... .......... 99%  150M 1s
>     1485950K .......... .......... .......... .......... .......... 99% 2.85M 1s
>     1486000K .......... .......... .......... .......... .......... 99% 3.84M 1s
>     1486050K .......... .......... .......... .......... .......... 99% 3.78M 1s
>     1486100K .......... .......... .......... .......... .......... 99% 3.81M 1s
>     1486150K .......... .......... .......... .......... .......... 99%  136M 1s
>     1486200K .......... .......... .......... .......... .......... 99% 3.05M 1s
>     1486250K .......... .......... .......... .......... .......... 99% 2.79M 1s
>     1486300K .......... .......... .......... .......... .......... 99% 3.68M 1s
>     1486350K .......... .......... .......... .......... .......... 99% 3.85M 1s
>     1486400K .......... .......... .......... .......... .......... 99%  119M 1s
>     1486450K .......... .......... .......... .......... .......... 99% 2.22M 1s
>     1486500K .......... .......... .......... .......... .......... 99% 3.81M 1s
>     1486550K .......... .......... .......... .......... .......... 99% 3.67M 1s
>     1486600K .......... .......... .......... .......... .......... 99%  133M 1s
>     1486650K .......... .......... .......... .......... .......... 99% 3.52M 1s
>     1486700K .......... .......... .......... .......... .......... 99% 3.78M 1s
>     1486750K .......... .......... .......... .......... .......... 99% 4.63M 1s
>     1486800K .......... .......... .......... .......... .......... 99% 3.46M 1s
>     1486850K .......... .......... .......... .......... .......... 99%  195M 1s
>     1486900K .......... .......... .......... .......... .......... 99% 2.67M 1s
>     1486950K .......... .......... .......... .......... .......... 99% 3.46M 1s
>     1487000K .......... .......... .......... .......... .......... 99% 2.87M 1s
>     1487050K .......... .......... .......... .......... .......... 99%  191M 1s
>     1487100K .......... .......... .......... .......... .......... 99% 1.45M 1s
>     1487150K .......... .......... .......... .......... .......... 99% 2.74M 1s
>     1487200K .......... .......... .......... .......... .......... 99% 2.76M 1s
>     1487250K .......... .......... .......... .......... .......... 99% 2.95M 1s
>     1487300K .......... .......... .......... .......... .......... 99%  194M 1s
>     1487350K .......... .......... .......... .......... .......... 99% 2.55M 1s
>     1487400K .......... .......... .......... .......... .......... 99% 2.77M 1s
>     1487450K .......... .......... .......... .......... .......... 99% 2.86M 1s
>     1487500K .......... .......... .......... .......... .......... 99%  220M 1s
>     1487550K .......... .......... .......... .......... .......... 99% 3.23M 1s
>     1487600K .......... .......... .......... .......... .......... 99% 2.77M 1s
>     1487650K .......... .......... .......... .......... .......... 99% 2.73M 1s
>     1487700K .......... .......... .......... .......... .......... 99% 2.87M 1s
>     1487750K .......... .......... .......... .......... .......... 99%  133M 1s
>     1487800K .......... .......... .......... .......... .......... 99% 2.82M 1s
>     1487850K .......... .......... .......... .......... .......... 99% 2.93M 1s
>     1487900K .......... .......... .......... .......... .......... 99% 2.79M 0s
>     1487950K .......... .......... .......... .......... .......... 99% 3.66M 0s
>     1488000K .......... .......... .......... .......... .......... 99%  144M 0s
>     1488050K .......... .......... .......... .......... .......... 99% 3.04M 0s
>     1488100K .......... .......... .......... .......... .......... 99% 3.02M 0s
>     1488150K .......... .......... .......... .......... .......... 99% 2.92M 0s
>     1488200K .......... .......... .......... .......... .......... 99%  174M 0s
>     1488250K .......... .......... .......... .......... .......... 99% 2.79M 0s
>     1488300K .......... .......... .......... .......... .......... 99% 3.82M 0s
>     1488350K .......... .......... .......... .......... .......... 99% 2.41M 0s
>     1488400K .......... .......... .......... .......... .......... 99% 3.82M 0s
>     1488450K .......... .......... .......... .......... .......... 99%  135M 0s
>     1488500K .......... .......... .......... .......... .......... 99% 3.01M 0s
>     1488550K .......... .......... .......... .......... .......... 99% 3.92M 0s
>     1488600K .......... .......... .......... .......... .......... 99% 3.94M 0s
>     1488650K .......... .......... .......... .......... .......... 99%  152M 0s
>     1488700K .......... .......... .......... .......... .......... 99% 4.03M 0s
>     1488750K .......... .......... .......... .......... .......... 99% 2.93M 0s
>     1488800K .......... .......... .......... .......... .......... 99% 2.79M 0s
>     1488850K .......... .......... .......... .......... .......... 99% 3.55M 0s
>     1488900K .......... .......... .......... .......... .......... 99%  138M 0s
>     1488950K .......... .......... .......... .......... .......... 99% 2.89M 0s
>     1489000K .......... .......... .......... .......... .......... 99% 4.48M 0s
>     1489050K .......... .......... .......... .......... .......... 99% 3.68M 0s
>     1489100K .......... .......... .......... .......... .......... 99%  174M 0s
>     1489150K .......... .......... .......... .......... .......... 99% 4.37M 0s
>     1489200K .......... .......... .......... .......... .......... 99% 3.65M 0s
>     1489250K .......... .......... .......... .......... .......... 99% 3.78M 0s
>     1489300K .......... .......... .......... .......... .......... 99% 3.01M 0s
>     1489350K .......... .......... .......... .......... .......... 99%  119M 0s
>     1489400K .......... .......... .......... .......... .......... 99% 2.86M 0s
>     1489450K .......... .......... .......... .......... .......... 99% 2.98M 0s
>     1489500K .......... .......... .......... .......... .......... 99% 3.00M 0s
>     1489550K .......... .......... .......... .......... .......... 99% 2.97M 0s
>     1489600K .......... .......... .......... .......... .......... 99%  151M 0s
>     1489650K .......... .......... .......... .......... .......... 99% 2.00M 0s
>     1489700K .......... .......... .......... .......... .......... 99% 4.04M 0s
>     1489750K .......... .......... .......... .......... .......... 99% 2.82M 0s
>     1489800K .......... .......... .......... .......... .......... 99%  161M 0s
>     1489850K .......... .......... .......... .......... .......... 99% 3.14M 0s
>     1489900K .......... .......... .......... .......... .......... 99% 3.00M 0s
>     1489950K .......... .......... .......... .......... .......... 99% 3.03M 0s
>     1490000K .......... .......... .......... .......... .......... 99% 2.93M 0s
>     1490050K .......... .......... .......... .......... .......... 99%  135M 0s
>     1490100K .......... .......... .......... .........            100% 2.35M=5m33s
>
>     2020-11-30 09:22:07 (4.36 MB/s) - ‘svwiki-20201120-pages-articles.xml.bz2’ saved [1525903265/1525903265]

  

Extract the .xml-file

In [ ]:
ls
bzip2 -dk svwiki-20201120-pages-articles.xml.bz2

  

>     conf
>     derby.log
>     eventlogs
>     ganglia
>     logs
>     svwiki-20201120-pages-articles.xml.bz2

  

Copy file to DBFS cluster

In [ ]:

val localpath="file:/databricks/driver/svwiki-20201120-pages-articles.xml"

dbutils.fs.mkdirs("dbfs:/datasets/")

dbutils.fs.cp(localpath, "dbfs:/datasets/")

display(dbutils.fs.ls("dbfs:/datasets/svwiki-20201120-pages-articles.xml"))

  

[TABLE]

In [ ]:
// Verify Dataset is in cluster
display(dbutils.fs.ls("dbfs:/datasets/svwiki-20201120-pages-articles.xml"))

  

[TABLE]

  

We now have a wikipedia dump ready to be processed

This page has some resources that can be used for Wikipedia parsing:
https://mindfulmachines.io/blog/2018/3/18/wikipedia-data-in-spark-and-scala-updated

In [ ]:
//---------------spark_import
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.spark.sql.SQLContext

//----------------xml_loader_import
import org.apache.hadoop.io.LongWritable
import org.apache.hadoop.io.Text
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.io.{ LongWritable, Text }
import com.databricks.spark.xml._

import org.apache.hadoop.conf.Configuration

  

>     import org.apache.spark.SparkContext
>     import org.apache.spark.SparkConf
>     import org.apache.spark.sql.SQLContext
>     import org.apache.hadoop.io.LongWritable
>     import org.apache.hadoop.io.Text
>     import org.apache.hadoop.conf.Configuration
>     import org.apache.hadoop.io.{LongWritable, Text}
>     import com.databricks.spark.xml._
>     import org.apache.hadoop.conf.Configuration

In [ ]:

def readWikiDump(sc: SparkContext, file: String) : RDD[(Long, String)] = {
  val conf = new Configuration()
  conf.set(XmlInputFormat.START_TAG_KEY, "<page>")
  conf.set(XmlInputFormat.END_TAG_KEY, "</page>")
  val rdd = sc.newAPIHadoopFile(file, 
                                classOf[XmlInputFormat], 
                                classOf[LongWritable], 
                                classOf[Text], 
                                conf)
  rdd.map{case (k,v) => (k.get(), new String(v.copyBytes()))}
}

  

>     readWikiDump: (sc: org.apache.spark.SparkContext, file: String)org.apache.spark.rdd.RDD[(Long, String)]

In [ ]:
// Read the wiki dump

val dump = readWikiDump(sc, "dbfs:/datasets/svwiki-20201120-pages-articles.xml")

//dump.collect() This throws a memory error

dump.take(5)

  

>     dump: org.apache.spark.rdd.RDD[(Long, String)] = MapPartitionsRDD[563] at map at command-738986526041623:11
>     res0: Array[(Long, String)] =
>     Array((10671,<page>
>         <title>Amager</title>
>         <ns>0</ns>
>         <id>1</id>
>         <revision>
>           <id>48450724</id>
>           <parentid>48155628</parentid>
>           <timestamp>2020-11-15T15:50:09Z</timestamp>
>           <contributor>
>             <username>Nalle&amp;Lisa</username>
>             <id>348723</id>
>           </contributor>
>           <comment>Länkfix</comment>
>           <model>wikitext</model>
>           <format>text/x-wiki</format>
>           <text bytes="7485" xml:space="preserve">{{källor|datum=2013-05}}
>
>     {{Geobox|Island
>     &lt;!-- *** Mallhuvud *** --&gt;
>     | name                  = Amager
>     | native_name           =
>     | other_name            =
>     | category              = Ö
>     &lt;!-- *** Namn **** --&gt;
>     | etymology             =
>     | nickname              =
>     &lt;!-- *** Bild *** --&gt;
>     | image                 = Amager.jpg
>     | image_caption         =
>     &lt;!-- *** Länder, regioner mm *** --&gt;
>     | country               = Danmark
>     | country_flag          = true
>     | state                 = | state_type =
>     | region                = [[Region Hovedstaden]]| region_type =
>     | district              = | district_type =
>     | municipality          = [[Dragørs kommun]]| municipality_type =
>     | municipality1         = [[Köpenhamns kommun]]
>     | municipality2         = [[Tårnby kommun]]
>     &lt;!-- *** Tillhörighet, föremål inom området mm *** --&gt;
>     | parent                =
>     | range                 = | range_type = Arkipelag
>     | border                =
>     | part                  =
>     | city                  =
>     | capital               =
>     | landmark              =
>     | river                 =
>     &lt;!-- *** Läge *** --&gt;
>     | location              = [[Öresund]]
>     | lat_d                 = 55| lat_m = 37| lat_s = 0| lat_NS = N
>     | long_d                = 12| long_m = 37| long_s = 0| long_EW = E
>     | highest               =
>     | highest_location      = | highest_region = | highest_country =
>     | highest_elevation     =
>     | highest_lat_d         = | highest_lat_m = | highest_lat_s = | highest_lat_NS =
>     | highest_long_d        = | highest_long_m = | highest_long_s = | highest_long_EW =
>     &lt;!-- *** Dimensioner *** --&gt;
>     | length                = 16.1| length_orientation =
>     | width                 = 11.1| width_orientation =
>     | height                = 4
>     | area                  = 96.29
>     &lt;!-- *** Befolkning *** --&gt;
>     | population            = 196047| population_date = 1 januari 2018 | population_note =
>     | population_density    = auto| population_density_note =
>     &lt;!-- *** Tillgänglighet *** --&gt;
>     | public                =
>     | access                =
>     &lt;!-- *** Koder ***  --&gt;
>     | timezone              = CET| utc_offset = +1
>     | timezone_DST          = CEST| utc_offset_DST = +2
>     | postal_code           =
>     | area_code             =
>     | code                  =
>     &lt;!-- *** Fria fält *** --&gt;
>     | free                  =
>     | free_type             =
>     &lt;!-- *** Kartor *** --&gt;
>     | map                   = Denmark physical map.svg
>     | map_caption           = Amagers läge i Danmark.
>     | map_locator           = Danmark
>     | map_locator_x         =
>     | map_locator_y         =
>     &lt;!-- *** Fotnoter *** --&gt;
>     | footnotes             =
>     }}
>
>     '''Amager''' är en [[Danmark|dansk]] [[ö (landområde)|ö]] i [[Öresund]]. Öns norra och västra delar tillhör [[Köpenhamn]], medan övriga delar upptas av [[Tårnby kommun]] och [[Dragørs kommun]].
>     Amager har en yta på 96,29 km² och befolkningen uppgår till 196&amp;nbsp;047 personer (1/1 2018). På den östra delen av ön finns [[Kastrups flygplats]]. Amager har också flera militära anläggningar (t.ex. Kastrup fort och Dragör fort), strandparksområdena [[Amager Strandpark]] och Kastrup Strandpark, flera restauranger i världsklass (t.ex. Noma), ett stort naturområde ([[Amager Fælled]]), ett av Nordens största köpcentra (Field's), och mycket annat. Amager förkortas ofta Ama'r för att återspegla det danska uttalet.
>
>     Amager är delvis en [[konstgjord ö]], delvis en naturlig sådan. T.ex. så anlades en konstgjord ö i anslutning till det redan existerande Amager Strandpark, när denna rustades upp väsentligen 2005. Ön är mycket låg och vissa delar ligger under havsytan, framför allt det genom fördämning och utfyllnad skapade området [[Kalvebod Fælled]] (Vestamager). Ön började uppodlas i större skala under 1500-talets första hälft då kung [[Kristian II]] bjöd in nederländska bönder till att odla upp ön för kronans räkning. Flertalet slog sig ned i fiskeläget [[Dragør]] där många än idag har namn med nederländskt ursprung som Neels och Tønnes.
>
>     Den stora utvecklingsfasen började på 1800-talet då flera fabriker, bostadsområden och militär verksamhet utvecklade sig på ön. Det fanns ett antal stora och kända fabriker på ön och den industriella prägeln höll sig väldigt tydlig långt in på 1900-talet. Det förekommer också områden som tidigare varit sommarstugeområden och där det än idag finns kolonilottområden, t.ex. i Sundbyöster ut mot kusten. Under 1900-talet började militären att utveckla stora övningsområden på ön vilket resulterade i Vestamager som idag innefattar ett stort naturområde endast några kilometer från Köpenhamns centrum. Under senare delen av 1900-talet har flygplatsen [[Kastrups flygplats|Kastrup]] som ligger på ön expanderat kraftigt och i samband med byggandet av [[Öresundsförbindelsen]] började även ett stort område byggas som kallas för [[Ørestad]].
>
>     Ön, som är förbunden med övriga Köpenhamn genom flera broar, [[järnväg]] och sedan 2017 även [[Köpenhamns tunnelbana|Metro]], har under de senaste 10-20 åren förändrats mycket. Dels har flera områden exploaterats med nya moderna kvarter lägenheter, t.ex. i områdena Örestad, [[Islands Brygge]] och Amager Strand, och dels så har Amagers närhet till Köpenhamn raderat dess tidigare rykte som en industriell förstad till Köpenhamn. Idag finns det områden av alla sorter på ön - tack vare flygplatsen finns ett aktivt näringsliv kvar, samtidigt som de nya områdena och inte minst upprustningen av t.ex. Amager Strand har förändrat bilden av ön. Idag är flera delar av ön mycket attraktiva bostadsområden.
>
>     Ön återfick i samband med Öresundsbron även en järnvägsförbindelse vilket inte funnits sedan 1960-talet. En [[motorväg]] byggdes samtidigt över öns mellersta del. [[Köpenhamns metro]] går till Kastrup och Vestamager med ett antal stationer på ön. Kring metro- och regionaltågsstationen Ørestad har ett stort kontors-, handels- och bostadsområde uppförts under [[2000-talet (decennium)|00-talet]]. Detta sträcker sig från [[Bella Center]] i norr till metrostationen [[Vestamager station|Vestamager]] i söder. Även om området inte ligger vid kusten har Ørestad delvis fått karaktären av en hamnstad genom anläggandet av konstgjorda kanaler och dammar. Vid [[Islands Brygge]] finns en &quot;äkta&quot; sjöstad.
>
>     Före [[Öresundsförbindelsen]]s uppförande hade småstaden [[Dragør]] en färjeförbindelse med [[Limhamn]] i södra [[Malmö]]. På grund av denna fanns i Dragør tidigare en betydande gränshandel. Denna har under 2000-talet helt upphört liksom den genomgående trafiken.
>
>     Öns västra del, som utvanns från Öresund under 1920- och 30-talen, är med undantag för Bella Center, Ørestad samt ett koloniområde, avsatt som natur- och rekreationsområde. Marken är ängsmark (odlad mark finns dock på den naturliga delen av öns södra del).
>     Längst i söder finns den flerhundraåriga skogen [[Kongelunden]]. På öns östra del, vid Öresundskusten, finns [[Amager Strandpark]] med en populär sandstrand. Området har omgestaltats, med en [[konstgjord ö]] och en [[lagun]] innanför. Nyinvigningen av parken, som funnits sedan 1934, ägde rum 2005.
>
>     Amager är den tätast befolkade ön i Danmark. Den näst mest tätbefolkade är [[Thurø]].
>
>     == Se även ==
>     *[[Amagerbro]]
>     *[[Amagerport]]
>     *[[Amagerbroderi]]
>
>     ==Externa länkar==
>     {{commonscat|Amager}}
>
>     {{Öar i Danmark}}
>
>     {{auktoritetsdata}}
>
>     [[Kategori:Amager| ]]
>     [[Kategori:Öar i Region Hovedstaden]]</text>
>           <sha1>qom8qpk70qa0alvfb4b4ha9or4ftohh</sha1>
>         </revision>
>       </page>), (52187,<page>
>         <title>Afrika</title>
>         <ns>0</ns>
>         <id>2</id>
>         <revision>
>           <id>48459142</id>
>           <parentid>48374249</parentid>
>           <timestamp>2020-11-18T06:30:03Z</timestamp>
>           <contributor>
>             <username>LarskeBot</username>
>             <id>310602</id>
>           </contributor>
>           <minor />
>           <comment>/* Geografi */mellan X - Y → mellan X och Y med [[Project:AWB|AWB]]</comment>
>           <model>wikitext</model>
>           <format>text/x-wiki</format>
>           <text bytes="37806" xml:space="preserve">{{templateStyles-tagg Faktamall}}
>     {| class=&quot;infobox&quot; style=&quot;font-size:90%;&quot; width=&quot;300&quot;
>     |+ &lt;big&gt;'''Afrika'''&lt;/big&gt;
>     |-
>     | align=&quot;center&quot; colspan=&quot;2&quot; style=&quot;background:#f9f9f9;&quot; | [[Fil:Africa (orthographic projection).svg|280px|Afrika i världen.]]
>     ----
>     |-
>     | align=&quot;center&quot; colspan=&quot;2&quot; style=&quot;background:#f9f9f9;&quot; | [[Fil:Afrika politisk.png|280px|Politisk karta.]]
>     ----
>     |-
>     | colspan=&quot;2&quot; |
>     |-
>     | '''Area:''' &lt;br /&gt; '''Folkmängd:''' &lt;br /&gt; '''Antal stater:''' &lt;br /&gt; '''Autonoma regioner:''' &lt;br /&gt; '''Högsta punkt:''' &lt;br /&gt; '''Lägsta punkt:''' || 30&amp;nbsp;221&amp;nbsp;532&amp;nbsp;km² &lt;br /&gt; 1&amp;nbsp;022&amp;nbsp;234&amp;nbsp;000&lt;ref name=&quot;esa.un.org&quot;/&gt; (2010) &lt;br /&gt; 54 &lt;br /&gt; 8 &lt;br /&gt; [[Kilimanjaro]]&amp;nbsp;5&amp;nbsp;895&amp;nbsp;[[m ö.h.|m&amp;nbsp;ö.h.]] &lt;br /&gt; [[Assalsjön]]&amp;nbsp;156&amp;nbsp;[[Meter över havet|m&amp;nbsp;u.h.]]
>     |-
>     | colspan=&quot;2&quot; |
>     ----
>     |-
>     | colspan=&quot;2&quot; | '''Stater'''
>     |-
>     | {{flaggbild2|Algeriet}} &lt;br /&gt; {{flaggbild2|Angola}} &lt;br /&gt; {{flaggbild2|Benin}} &lt;br /&gt; {{flaggbild2|Botswana}} &lt;br /&gt; {{flaggbild2|Burkina Faso}} &lt;br /&gt; {{flaggbild2|Burundi}} &lt;br /&gt; {{flaggbild2|Centralafrikanska republiken}} &lt;br /&gt; {{flaggbild2|Djibouti}} &lt;br /&gt; {{flaggbild2|Egypten}} &lt;br /&gt; {{flaggbild2|Ekvatorialguinea}} &lt;br /&gt; {{flaggbild2|Elfenbenskusten}} &lt;br /&gt; {{flaggbild2|Eritrea}} &lt;br /&gt; {{flaggbild2|Etiopien}} &lt;br /&gt; {{flaggbild2|Gabon}} &lt;br /&gt; {{flaggbild2|Gambia}} &lt;br /&gt; {{flaggbild2|Ghana}} &lt;br /&gt; {{flaggbild2|Guinea}} &lt;br /&gt; {{flaggbild2|Guinea-Bissau}} &lt;br /&gt; {{flaggbild2|Kamerun}} &lt;br /&gt; {{flaggbild2|Kap Verde}} &lt;br /&gt; {{flaggbild2|Kenya}} &lt;br /&gt; {{flaggbild2|Komorerna}} &lt;br /&gt; {{flaggbild2|Kongo-Brazzaville}} &lt;br /&gt; {{flaggbild2|Kongo-Kinshasa}} &lt;br /&gt; {{flaggbild2|Lesotho}} &lt;br /&gt; {{flaggbild2|Liberia}} &lt;br /&gt;{{flaggbild2|Libyen}} ||{{flaggbild2|Madagaskar}} &lt;br /&gt; {{flaggbild2|Malawi}} &lt;br /&gt; {{flaggbild2|Mali}} &lt;br /&gt; {{flaggbild2|Marocko}} &lt;br /&gt; {{flaggbild2|Mauretanien}} &lt;br /&gt; {{flaggbild2|Mauritius}} &lt;br /&gt; {{flaggbild2|Moçambique}} &lt;br /&gt; {{flaggbild2|Namibia}} &lt;br /&gt; {{flaggbild2|Niger}} &lt;br /&gt; {{flaggbild2|Nigeria}} &lt;br /&gt; {{flaggbild2|Rwanda}} &lt;br /&gt; {{flaggbild2|São Tomé och Príncipe}} &lt;br /&gt; {{flaggbild2|Senegal}} &lt;br /&gt; {{flaggbild2|Seychellerna}} &lt;br /&gt; {{flaggbild2|Sierra Leone}} &lt;br /&gt; {{flaggbild2|Somalia}} &lt;br /&gt; {{flaggbild2|Sudan}} &lt;br /&gt; {{flaggbild2|Sydafrika}} &lt;br /&gt; {{flaggbild2|Sydsudan}} &lt;br /&gt; {{flaggbild2|Swaziland}} &lt;br /&gt; {{flaggbild2|Tanzania}} &lt;br /&gt; {{flaggbild2|Tchad}} &lt;br /&gt; {{flaggbild2|Togo}} &lt;br /&gt; {{flaggbild2|Tunisien}} &lt;br /&gt; {{flaggbild2|Uganda}} &lt;br /&gt; {{flaggbild2|Zambia}} &lt;br /&gt; {{flaggbild2|Zimbabwe}}
>     |-
>     | colspan=&quot;2&quot; |
>     ----
>     '''Stater som har delar av ytan i Afrika'''
>     |-
>     | {{flaggbild2|Egypten}} &lt;br /&gt; {{flaggbild2|Frankrike}} &lt;br /&gt; {{flaggbild2|Jemen}} &lt;br /&gt; {{flaggbild2|Portugal}} &lt;br /&gt; {{flaggbild2|Spanien}}
>     |-
>     | colspan=&quot;2&quot; |
>     ----
>     '''Särskilda områden'''
>     &lt;br /&gt; {{flaggbild|Ceuta}} [[Ceuta]] ([[Spanien]])
>
>     {{flaggbild|Kanarieöarna}} [[Kanarieöarna]] ([[Spanien]])
>
>     {{flaggbild|Madeira}} [[Madeira]] ([[Portugal]])
>
>     {{flaggbild|Frankrike}} [[Mayotte]] ([[Frankrike]])
>
>     {{flaggbild|Melilla}} [[Melilla]] ([[Spanien]])
>
>     {{flaggbild|Frankrike}} [[Réunion]] ([[Frankrike]])
>
>     {{flaggbild|Sankta Helena}} [[Sankta Helena]] ([[Storbritannien]])
>
>     [[Fil:Flag of Somaliland.svg|22px|Somaliland]] [[Somaliland]]
>
>     {{flaggbild2|Västsahara}} ([[Marocko]])
>     ----
>     '''Största städer (över 10 miljoner invånare med förorter):'''
>     &lt;br /&gt;{{flaggbild|Egypten}} [[Kairo]]
>     &lt;br /&gt;{{flaggbild|Kongo-Kinshasa}} [[Kinshasa]]
>     &lt;br /&gt;{{flaggbild|Nigeria}} [[Lagos]]
>     ----
>     [[Fil:Africa satellite.jpg|280px|Satellitfoto]]
>     |}
>     '''Afrika''' är [[jorden]]s näst största [[kontinent]] (efter [[Eurasien]]) och även jordens näst största världsdel efter [[Asien]], både vad gäller [[areal]] och [[folkmängd]]. Med världsdelens öar inräknade mäter Afrika 30 244 050&amp;nbsp;km², vilket motsvarar 20,3 % av jordens landmassa eller cirka 6 % av jordens totala area. Omkring 22 miljoner km² av dessa ligger i [[tropikerna]] vilket gör den afrikanska kontinenten till världens varmaste kontinent. I Afrika bor det ungefär 1 miljard människor i 55 länder – en sjundedel av jordens befolkning. Dess längd i nordlig-sydlig riktning är omkring 8 000&amp;nbsp;km och dess största bredd omkring 7&amp;nbsp;800&amp;nbsp;km.
>
>     == Etymologi ==
>     Ordet &quot;Afrika&quot;s [[etymologi]]ska bakgrund är dunkel, och ordet har dessutom haft olika betydelse under olika [[epok]]er.
>
>     I äldre tider kallades allt land väster om [[Egypten]] för &quot;[[Libyen]]&quot;. Enligt den [[Grekland|grek]]iska historieskrivaren [[Herodotos]] berättelse seglade [[fenicisk]]a sjömän på uppdrag av den egyptiska kungen Neko runt Libyen, det vill säga [[kontinent]]en Afrika redan omkring 600 f.Kr.
>
>     När [[Romerska riket|romarna]] först använde termen ''[[Africa terra]]'' avsåg de före [[andra puniska kriget]] endast trakten kring [[Karthago]], det vill säga motsvarande ungefär dagens Tunisien där själva staden Karthago låg samt stora delar av Libyen och Algeriet. Romarna uppfattade först [[Egypten]] som en del av [[Asien]], och det var först geografen [[Klaudios Ptolemaios|Ptolemaios]] som drog gränsen vid [[Suez]]. Under kejsartiden kom ''Africa'' att avse hela det området romarna kände till av kontinenten, och allt eftersom européerna har upptäckt mer och mer av kontinenten har termen avsett ett allt större område. Namnet ska ha avsett &quot;landet där ''afri''&quot; (singularis: ''afer'') bodde. ''Afer'' avsåg namnet på ett folk som kan motsvara dagens [[berber]]. [[Viking]]arna under 800- och 900-talen kallade norra Afrika för &quot;Blåland&quot;.
>
>     == Geografi ==
>     {{Fördjupning|Afrikas geografi}}
>     Geologiskt är Afrika en mycket gammal kontinent, i huvudsak uppbyggd av [[urberg]]. Under jordens forntid var den en del av jättekontinenten [[Gondwana]]land. Denna gled isär under [[Jura (period)|juraperioden]] och [[Krita (geologi)|kritaperioden]] för 100-150 miljoner år sedan, och så fick Afrika sin nuvarande form.
>
>     Inom sina regelbundna kuster rymmer den en total yta på omkring 30 244 050&amp;nbsp;km² inklusive öarna. Afrika åtskiljs från [[Europa]] av [[Medelhavet]] och förenas med [[Asien]] i sitt nordöstra hörn genom [[näs]]et vid [[Suez]], omkring 130&amp;nbsp;km i bredd. Omkring 8 000&amp;nbsp;km skiljer den nordligaste punkten, [[Ras ben Sakka]] i [[Tunisien]], från dess sydligaste, [[Kap Agulhas]] i [[Sydafrika]]. Från [[Kap Verde]], dess västligaste punkt, till [[Ras Hafun]] i [[Somalia]] är det omkring 7 400&amp;nbsp;km. Afrikas kuster mäter omkring 28 000&amp;nbsp;km (1&amp;nbsp;km kust på 1 057&amp;nbsp;km²) varav [[Medelhavet]] står för 5 400&amp;nbsp;km, [[Atlanten]] 11 000&amp;nbsp;km, [[Indiska oceanen]] 8 700&amp;nbsp;km och [[Röda havet]] 2 900&amp;nbsp;km, och avsaknaden av djupare [[bukt]]er och betydande [[halvö]]ar blir uppenbar om man jämför med Europas 9 700 000&amp;nbsp;km² som omgärdas av 32 000&amp;nbsp;km kust (1&amp;nbsp;km kust på 278&amp;nbsp;km² land), eller för det [[fjord]]rika [[Norge]], där motsvarande siffra 385 000&amp;nbsp;km² land och en kuststräcka på 20 000&amp;nbsp;km (1&amp;nbsp;km kust per 19&amp;nbsp;km² land).
>
>     Afrikas genomsnittliga höjd över havet är omkring 600 m, vilket motsvarar de båda amerikanska kontinenternas höjd men är lägre än den asiatiska på omkring 950 m. Jämfört med de andra kontinenterna utmärker sig Afrika med en liten andel landområden på mycket hög respektive mycket låg höjd. Landområdena under 180 [[Meter över havet|m ö.h.]] upptar en osedvanligt liten del av den totala landmassan, och även de högsta delarna, som är betydligt lägre än motsvarande i Asien, utgörs av isolerade berg och bergskedjor. Generellt ligger de högre [[platå]]erna i öst och söder och de lägre delarna åt väst och norr och med en linje från ungefär vid [[Röda havet]]s mitt till [[Kongofloden]]s mynning kan kontinenten delas upp i ett lågland i norr och ett högland i söder. Man kan således dela upp Afrika i fyra övergripande landområden:
>     # Slättlanden vid kusterna – ofta med [[Mangrove|mangroveträsk]] insprängda – når aldrig långt in från kusterna utom vid vattendragens nedre delar. Slamrika slättland finns egentligen bara vid de större flodernas deltan, och annars utgör kustområdena det lägsta steget i en serie terrasser som bildar stigningen upp till de inre platåerna.
>     # [[Atlasbergen]] i norr som ligger isolerat av [[Sahara]]öknen som bitvis ligger under havsnivån.
>     # De södra och östra höglanden som ligger på mellan 600 och 1 000 m ö.h.
>     # De nordliga och västra slättlanden, inklusive Sahara, som kantas och genomfars av stråk med högra landområden, men i allmänhet håller sig under 600 m ö.h.
>
>     Skog täcker cirka en femtedel av Afrikas yta.
>
>     Hela Afrika har ett [[Tropiskt klimat|tropiskt]] eller [[Subtropiskt klimat]].
>
>     === Fauna ===
>
>     {{Fördjupning|Afrikas fauna}}
>
>     Afrika är huvudsakligen känt för den öppna [[savann]]en med väldiga hjordar av hovdjur som [[afrikansk buffel]], gnuer, [[antiloper]], [[giraff]]er och [[zebror]]. Dessa jagas av stora rovdjur som [[lejon]], [[hyenor]] och [[Gepard|geparder]]. Den afrikanska djungeln befolkas av [[ormar]] och [[primater]] samt av vattenlevande djur som [[krokodiler]] och [[groddjur]]. Afrika har även det största antalet arter som kan räknas till [[megafauna]]n, bland annat [[afrikanska elefanter]], [[trubbnoshörning]] och [[flodhäst]], på grund av att kontinenten bara i mindre utsträckning var påverkad av det [[massutdöende]] som skedde under [[pleistocen]].
>
>     == Historia ==
>     {{Fördjupning|Afrikas historia}}
>     Den första mänskliga migrationen, från kontinenten men även inom Afrikas kuster, kan spåras [[lingvistisk]]t, kulturellt och genom datoranalyser av [[genetik|genetiskt]] material. Någonstans i [[Östafrika]] tror dagens forskare att de första [[hominider]] uppkom. I [[Etiopien]] har man hittat ett två miljoner år gammalt skelett som uppkallades [[Australopithecus afarensis|Lucy]], den första kända mänskliga [[primater|primaten]].
>
>     === Tidigare civilisationer och handel ===
>     [[Fil:African-civilizations-map-pre-colonial.svg|miniatyr|höger|Några av de mest kända afrikanska civilisationerna. Civilisationer från olika tidsepoker visas parallellt.]]Lingvistiska spår tyder på att [[Bantu]]folk emigrerade från [[Västafrika]] in i [[khoisan]]folkets (de som tidigare kallades [[bushmän]] och [[hottentott]]er) område och undanträngde dem. Bantufolken använde en uppsättning grödor, bland annat [[kassava]] och [[jams]], som var mer lämpade för jordbruk i det [[tropik|tropiska]] Afrika, och med sitt jordbruk kunde bantu försörja en betydligt större folkmängd än de undanträngde jägare eller samlare. Bantufolkens traditionella bosättningsområde sträcker sig från [[Sahara]]öknen till de [[Tempererat klimat|tempererade regionerna]] i söder. Historiskt har deras vapen varit spjut, sköld, pil och båge. Det för [[khoisanspråk]]ens karaktäristiska klickljuden återfinns i [[nguni (språk)|nguni]]- och [[bantu]]språk som till exempel [[xhosa]] och [[zulu]]. Khoisanspråken talas idag främst av utspridda minoritetsgrupper i södra Afrika i länderna kring [[Kalahariöknen]].
>
>     Afrika söder om Sahara har bestått olika stater och riken, mindre stadsstater samt [[stam (samhällsvetenskap)|stammar]] och [[klan]]er. Det forna egyptiska riket, är en av världens äldsta stater, och den mest kända av de äldre afrikanska [[civilisation]]erna. Det fanns även statsbildningar i [[Nubien]], [[Ghana]], [[Timbuktu]], [[Kanem-Bornu]] och i nuvarande [[Zimbabwe]].
>
>     [[Nordafrika]] kom från och med 600-talet under arabiskt inflytande – en tid av kulturell blomstring – och med kamelens hjälp spreds [[Islam]] och den arabiska kulturen genom Sahara och vidare söderut. Arabiska och persiska handelsmän spred likaså Islam längs med den östafrikanska kusten parallellt med en omfattande svart slavhandel. Detta östafrikanska handelsutbyte med araber och perser sträckte sig ända bort till Java, därmed den etniska, kulturella och språkliga mångfalden på Östafrikas kust, [[Madagaskar]] och [[Komorerna]].
>
>     === Arabvärlden och Europa utforskar kontinenten ===
>     Det arabiska intresset för Afrika går långt tillbaka, men handel med slavar från centra som Lamu och Zanzibar vid östkusten har pågått åtminstone sedan senare delen av 1000-talet e.Kr. De miljoner slavar som exporterades till arabvärlden och österut under den [[arabiska slavhandeln]] finns idag kvar som en stor etnisk grupp där, i stor utsträckning integrerad med den ursprungliga arabiska befolkningen. Slavhandel med svarta pågår fortfarande i vissa arabländer, till exempel Sudan.&lt;ref&gt;[http://www.hrw.org/backgrounder/africa/sudanupdate.htm Human Rights Watch] [http://news.bbc.co.uk/2/hi/programmes/this_world/6458377.stm BBC World]&lt;/ref&gt;
>
>     Under [[1300-talet]] började vissa europeiska handelsexpeditioner få kontakt med Nord- och Västafrika.
>
>     Från 1500-talets mitt fram till förbudet mot slavhandel i Storbritanniens parlament 1807, som kom att få full kraft först något årtionde senare, handlades slavar från lokala stamledare i främst Väst- och Centralafrika. Flera miljoner afrikaner såldes på detta vis som slavar till främst de Europeiska kolonierna i Amerika.
>
>     Tidiga kolonisatörer var [[araber]], [[Portugal|portugiser]] och [[Spanien|spanjorer]]. Därefter koloniserades kontinenten av framförallt av [[Storbritannien]], [[Frankrike]], [[Spanien]], [[Italien]], [[Belgien]], [[Nederländerna]], [[Tyskland]] och [[Portugal]].
>
>     === Koloniseringen ===
>     {{Fördjupning|Kapplöpningen om Afrika}}
>     [[Fil:Map_of_Africa_from_Encyclopaedia_Britannica_1890.jpg|miniatyr|Politisk karta över Afrika från 1890.]] Från slutet av 1000-talet koloniserade arabiska köpmän den afrikanska östkusten. Holländarna koloniserade på 1600-talet den då relativt folktomma södra udden av Afrika. Portugiserna grundade flera kolonier i västra Afrika och vid nuvarande Moçambique. I mitten av [[1800-talet]] leddes flera expeditioner in i de centrala delarna av Afrika, bland annat av Livingstone och Speke, men också den svenske upptäcktsresanden Andersson. Denna kolonisering förändrade i grunden det ekonomiska livet på kontinenten, och innebar i flera fall ensidig export av naturresurser. [[Leopold II av Belgien|Kung Leopolds]] rovdrift på sin privata koloni [[Kongostaten]] är ökänd och kostade miljontals afrikanska liv.
>
>     I Afrika fanns efter första världskriget endast tre självständiga stater nämligen konungariket Egypten, republiken Liberia och konungariket Abessinien. Tanger bildade med sitt område en internationell zon. För övrigt var hela världsdelen uppdelad på kolonier åt europeiska makter.&lt;ref&gt;{{runeberg.org|nfcn|0245.html Supplement Afrika}}&lt;/ref&gt;
>
>     Flertalet kolonier erhöll [[självständighet]] under slutet av 1950-talet och början av 1960-talet. Avkoloniseringen resulterade i flertalet fall med katastrof, där diktatur och etnisk rensning vanligen blev följden, till exempel i Rhodesia (nuvarande Zimbabwe), Rwanda, Uganda, Nigeria m.fl. stater.
>
>     == Politik ==
>     {{Fördjupning|Afrikas politik}}
>     [[Fil:Electoral_Democracies_in_Africa,_Freedom_House_2007.png|miniatyr|De blå länderna är demokratier enligt tankesmedjan Freedom House.]]
>     Det postkoloniala Afrikas politik har i hög grad präglats av diktatur, statskupper och brist på [[mänskliga rättigheter]].
>
>     [[Nation]]erna i Afrika har ofta visat sig vara instabila [[stat]]er som efter självständigheten hemsökts av korruption, våldsamma [[statskupp]]er, [[despotism]] och [[dikatur|militärdiktaturer]]. Under perioden [[1960]]-[[1989]] genomfördes över 70 kupper och 13 [[mord]] på [[president]]er. Under det [[kalla kriget]] kämpade [[USA]] och [[Sovjetunionen]] om makten i Afrika, och tillsammans med länder som [[Frankrike]] och [[Kuba]], stödde de aktivt diktaturer och iscensatte flera av dessa kupper och illgärningar mot demokratiskt valda regeringar. Enligt en spridd uppfattning beror många [[militär]]a konflikter om [[territorium|territorier]] och [[gräns]]er under Afrikas moderna historia på de gränser från kolonial tid som huvudsakligen drogs upp efter [[Kongokonferensen]], men detta kan inte visas eftersom något jämförelseområde inte finns. Konflikter i Afrika har ofta drivits av tillgången till kontinentens rika [[råvara|råvarutillgångar]] – [[diamant]]er och [[metall]]er som [[guld]], [[platina]], [[uran]] och [[koppar]].
>
>     Korruption och havererade politiska program har lett till utbredd [[undernäring]] och undermålig infrastruktur orsakar alltjämt brist på mat och rent vatten. Sanitetsproblemen har också orsakat en akut spridning av bland annat [[HIV]]-viruset som leder till [[sjukdom]]en [[AIDS]].
>
>     Trots nöd och umbäranden finns vissa positiva tecken – [[demokrati]]skt valda [[regering]]ar växer i antal även om de inte utgör någon majoritet ännu. Genom påtryckningar från till exempel [[Internationella valutafonden]] (IMF) har många afrikanska stater lyckats vända en nedåtgående ekonomisk spiral som varat i flera decennier till en positiv tillväxt. Om man beaktar IMF:s katastrofala misslyckanden i länder som [[Argentina]] och [[Filippinerna]] är det dock oklart i vilken omfattning dessa konventionella ekonomiska kriterier ger en rättvis bild av vad Afrika faktiskt behöver.
>
>     De afrikanska ledarna har i olika omgångar sökt former för samarbete sinsemellan. Hittills har dock detta inte lyckats särskilt väl. Under [[inbördeskrig]]et i [[Kongo-Kinshasa|Kongo]] var det grannstaterna som spädde på konflikten snarare än utomafrikanska stater, och motivet var främst att skaffa kortsiktiga vinster genom att utvinna diamanter och andra mineraler. Antalet döda i konflikten har uppskattats till 3,5 miljoner under en femårsperiod.
>
>     De politiska samarbetsorganisationer man skapat har inte kunnat leva upp till förväntningarna. Organisationen för afrikansk enighet (OAU 1963-2002) kallades till exempel diktatorsklubben för dess tendens att alltid stödja de sittande politiska ledarna och ägna väldigt lite energi åt att främja den vanlige afrikanens mänskliga rättigheter. 2002 skapades i stället ''[[Afrikanska unionen]]'', som är tänkt att underlätta ett framtida samarbete mellan kontinentens stater. Hittills har denna organisation inte klarat att hantera till exempel utrotningskriget i Darfur-provinsen i Sudan, där en arabisk statsledning tillåtit slakt på hundratusentals av bantubefolkningen i sydväst. En avgörande skillnad mellan såväl OAU som AU å ena sidan och EU å den andra, är att EU startades av en grupp länder som satte demokrati och mänskliga rättigheter i fokus för den egna politiken, och som krävt detta av tillkommande medlemsstater. OAU och AU har från början i stället varit territoriellt inriktade, och visserligen haft demokrati och mänskliga rättigheter som skrivelser i sina stadgar, men ytterst sällan satt kraft bakom dessa ord. På så sätt liknar AU mer FN än EU.
>
>     == Ekonomi ==
>     {{Fördjupning|Afrikas ekonomi}}
>     [[Fil:Bakweri cocoyam farmer from Cameroon.jpg|250px|miniatyr|Jordbrukare på en [[jams]]odling i Kamerun]]
>     Afrikas ekonomi omfattar ungefär 887 miljoner människor i 54 olika stater ([[Juli 2005]]). Afrika är världens fattigaste kontinent, och den är, i medeltal, fattigare än den var för 25 år sedan. På [[FN]]:s Human Development Report från [[2003]] (som rankade världens 175 länder enligt en rad kriterier) var plats mellan 151 och 175 intagna av länder i Afrika.
>
>     Det finns dock länder som lyckas bra ekonomiskt, bland andra [[Botswana]], [[Ghana]] och [[Sydafrika]]. [[Etiopien]] har också en relativ bra ekonomi tack vare den höga tillväxten i landet på cirka 11-15% per år. Sydafrika har en väl fungerande inhemsk aktiehandel och har stora naturtillgångar i bland annat [[guld]] och [[diamanter]] och ett väl fungerande rättsväsen. Det finns i Sydafrika även tillgång till investeringskapital, stora marknader och utbildad arbetskraft. Det finns även andra afrikanska länder, exempelvis [[Egypten]], som länge haft en väl fungerande handelssektor och ekonomi.
>
>     Kontinentens kust mot [[Atlanten]] återfinns stora fyndigheter av [[Petroleum|olja]]. I [[Nigeria]] finns en av världens mest betydande oljereserver och landets ekonomi är bland de snabbast växande i världen.
>
>     Afrikas jordbruk sysselsätter drygt hälften av Afrikas befolkning och är den viktigaste näringen. Cirka 6 % (ungefär 1 814 600&amp;nbsp;km²) av världsdelens yta är uppodlad, mest i savannområdena. I stora delar av de brukningsbara områdena är jordens bördighet ganska låg. De bästa jordarna återfinns i begränsade höglandsområden som i till exempel [[Kamerun]] och [[Östafrika]].
>
>     == Demografi ==
>     {{Fördjupning|Afrikas demografi}}
>     ===Befolkning===
>     Människan, ''[[Homo sapiens|Homo sapiens sapiens]]'', antas härstamma från Afrika och spred sig härifrån till de omgivande kontinenterna. Den svarta befolkningen söder om Sahara tillhör den ursprungliga befolkningen som hela tiden levat i Afrika.
>
>     Norr om Sahara präglar samröret med övriga [[Medelhavet|Medelhavsområdet]] och inte minst sydvästra [[Asien]] både befolkning, språk och kultur.
>
>     På ön [[Madagaskar]], som länge var obebodd, härstammar majoriteten av befolkningen från människor som emigrerat från [[Indonesien]]. Under kolonialtiden skedde inflyttning från [[Europa]] och [[Asien]] till södra Afrika, där den icke-svarta befolkningen idag utgör betydande minoritetsgrupper.
>
>     Afrikas befolkning uppvisar många för utvecklingsregioner typiska drag, som höga [[födelsetal]] i kombination med sjunkande [[Mortalitet (demografi)|dödstal]] med snabb befolkningsökning och låg medelålder (45 % är under 15 år) som följd. Även om befolkningstätheten varierar kraftigt mellan olika regioner är Afrika generellt en glestbefolkad kontinent, med i genomsnitt 27,8 invånare per km².
>
>     Afrika är den kontinent som har den lägsta urbaniseringsgraden, med endast 25% boende i städer. Även här är variationerna stora, och i norra och östra Afrika finns en lång urban tradition.
>
>     === Språk ===
>     {{Fördjupning|Afrikas språk}}
>     [[Fil:African languages SV.png|miniatyr|300px|Språkfamiljer i Afrika]]
>     Uppskattningsvis finns det ungefär 1800 talade [[språk]] i Afrika idag. Vissa av dessa, till exempel [[Swahili]], [[Hausa]], och [[Yoruba (språk)|Yoruba]], talas av många miljoner människor, medan det finns andra som bara talas av några hundra eller ännu färre. Afrika har även många olika [[teckenspråk]]. De flesta afrikaner talar som modersmål ett språk tillhörande någon av de fyra språkfamiljerna [[afroasiatiska språk]], [[nilo-sahariska språk]], [[Niger-Kongospråk]] eller [[khoisanspråk]].
>     Med några få nämnvärda undantag i östra Afrika, har nästan alla afrikanska länder språk som har sitt ursprung utanför världsdelen som officiellt språk, på grund av kolonialismen och mänsklig migration. Till exempel används i ett stort antal afrikanska länder [[franska]] eller [[engelska]] som administrativt språk för stat, handel, utbildning och media. [[Arabiska]], [[portugisiska]], [[afrikaans]] och [[malagassiska]] är andra exempel på språk med ursprung utanför Afrika som idag används av miljoner afrikaner, både i offentliga och privata sammanhang.
>
>     === Religion ===
>     {{Fördjupning|Afrikas religion}}{{Se även|Kristendomen i Afrika}}
>     [[Kristendom]] och [[islam]] är de största religionerna i Afrika, 46,3 % av alla afrikaner är kristna och 40,5 % är muslimer. Resten 11,8 % tillhör olika lokala afrikanska religioner som funnits före den europeiska kolonisationen. Ett mindre antal afrikaner är [[Hinduism|hinduer]], eller har tro från [[Judendom|judisk]] tradition. Exempel på [[Afrikanska judar|afrikanska folk som tillhör judendomen]] är [[Beta Israel]], [[Lemba]]folk och [[Abayudaya]] i östra [[Uganda]]. Sydafrika och Angola har över 55% kristna katoliker.
>
>     == Kultur ==
>     {{Fördjupning|Afrikas kultur}}
>     Afrika har ett stort antal överlappande kulturer, och flera tusen olika etniska grupper. Den mest konventionella distinktionen är den mellan området söder om Sahara och området norr om öknen. De nordliga länderna associerar sig främst med arabkulturen, medan de sydliga länderna består av i sin tur många olika kulturella områden. En av de största kulturella identiteterna är den som innefattar alla folk som talar något [[bantuspråk]].
>
>     Det går också att göra en indelning i gamla franska Västafrika å ena sidan, och resten av Afrika (mer specifikt de brittiska eller portugisiska kolonierna i södra och östra Afrika) å andra sidan. En annan kulturell linje går mellan de afrikaner som håller kvar vid sina gamla traditionella livsstilar, och de som har anpassat sig till en mer modern livsstil. Det går även att dela upp traditionalisterna i sådana som ägnar sig åt uppfödning av boskap, och sådana som ägnar sig åt jordbruk.
>
>     [[Afrikansk konst]] och [[afrikansk arkitektur]] reflekterar de afrikanska kulturernas diversitet. De äldsta exemplen på afrikansk konst är 75 000 år gamla pärlsmycken gjorda av [[nassarius]]skal, dessa hittades i [[Blombosgrottan]]. [[Pyramiderna i Giza]] var världens högsta byggnader under 4 000 år. [[Klippkyrkorna i Lalibela]] i [[Etiopien]], bland vilka ''[[Bete Giyorgis]]'' (&quot;St. Göranskyrkan&quot;) är mest välkänd, är också kända exempel på afrikansk arkitektur.
>
>     == Regioner och territorier ==
>     [[Fil:Africa-regions.png|miniatyr|Regioner i Afrika:
>     {{legend|#0000ff|[[Nordafrika]]}}
>     {{legend|#00ff00|[[Västafrika]]}}
>     {{legend|#ff00ff|[[Centralafrika]]}}
>     {{legend|#ffa500|[[Östafrika]]}}
>     {{legend|#ff0000|[[Södra Afrika]]}}]]
>     {| class=&quot;wikitable sortable&quot;
>     |+
>     |- bgcolor=&quot;#ECECEC&quot;
>     ! Regioners namn&lt;ref&gt;Enligt [[:Fil:United Nations geographical subregions.png|FN:s regionindelning]] av världen.&lt;br /&gt;&lt;/ref&gt; och&lt;br /&gt; territorier, med flagga
>     ! data-sort-type=number | [[Area]]&lt;br /&gt;(km²)
>     ! data-sort-type=number |[[Folkmängd]]&lt;br /&gt;(2010)&lt;ref name=&quot;esa.un.org1&quot;/&gt;
>     ! data-sort-type=number |[[Befolkningstäthet]]&lt;br /&gt;(per km²)
>     ! [[Huvudstad]]
>     |-
>     | colspan=5 style=&quot;background:#eee;&quot; | '''[[Centralafrika]]''':
>     |-
>     | {{flaggbild2|Angola}}
>     | align=&quot;right&quot; | 1 246 700
>     | align=&quot;right&quot; | 19&amp;nbsp;081&amp;nbsp;912
>     | align=&quot;right&quot; | 15,3
>     | [[Luanda]]
>     |-
>     | {{flaggbild2|Kamerun}}
>     | align=&quot;right&quot; | 475 440
>     | align=&quot;right&quot; | 19 598 889
>     | align=&quot;right&quot; | 41,2
>     | [[Yaoundé]]
>     |-
>     | {{flaggbild2|Centralafrikanska republiken}}
>     | align=&quot;right&quot; | 622 984
>     | align=&quot;right&quot; | 4 401 051
>     | align=&quot;right&quot; | 7,1
>     | [[Bangui]]
>     |-
>     | {{flaggbild2|Tchad}}
>     | align=&quot;right&quot; | 1 284 000
>     | align=&quot;right&quot; | 11 227 208
>     | align=&quot;right&quot; | 8,7
>     | [[N'Djamena]]
>     |-
>     | {{flaggbild2|Kongo-Brazzaville}}
>     | align=&quot;right&quot; | 342 000
>     | align=&quot;right&quot; | 4 042 899
>     | align=&quot;right&quot; | 12,9
>     | [[Brazzaville]]
>     |-
>     | {{flaggbild2|Kongo-Kinshasa}}
>     | align=&quot;right&quot; | 2 345 410
>     | align=&quot;right&quot; | 65 965 795
>     | align=&quot;right&quot; | 28,1
>     | [[Kinshasa]]
>     |-
>     | {{flaggbild2|Ekvatorialguinea}}
>     | align=&quot;right&quot; | 28 051
>     | align=&quot;right&quot; | 700 401
>     | align=&quot;right&quot; | 24,9
>     | [[Malabo]]
>     |-
>     | {{flaggbild2|Gabon}}
>     | align=&quot;right&quot; | 267 667
>     | align=&quot;right&quot; | 1 505 463
>     | align=&quot;right&quot; | 5,6
>     | [[Libreville]]
>     |-
>     | {{flaggbild2|São Tomé och Príncipe}}
>     | align=&quot;right&quot; | 1 001
>     | align=&quot;right&quot; | 165 397
>     | align=&quot;right&quot; | 165,0
>     | [[São Tomé]]
>     |-
>     | colspan=5 style=&quot;background:#eee;&quot; | '''[[Nordafrika]]''':
>     |-
>     | {{flaggbild2|Algeriet}}
>     | align=&quot;right&quot; | 2 381 740
>     | align=&quot;right&quot; | 35 468 000
>     | align=&quot;right&quot; | 14,9
>     | [[Alger (stad)|Alger]]
>     |-
>     | {{flaggbild2|Egypten}}&lt;ref&gt;[[Egypten]] brukar anses ligga i två världsdelar, Nordafrika (enligt FN:s regionindelning) och Västasien; befolknings- och arealuppgifterna avser endast den afrikanska delen, väster om [[Suezkanalen]].&lt;/small&gt;&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 1 001 450
>     | align=&quot;right&quot; | 81 121 000
>     | align=&quot;right&quot; | 81,0
>     | [[Kairo]]
>     |-
>     | {{flaggbild2|Libyen}}
>     | align=&quot;right&quot; | 1 759 540
>     | align=&quot;right&quot; | 6 355 112
>     | align=&quot;right&quot; | 3,6
>     | [[Tripoli]]
>     |-
>     | {{flaggbild2|Marocko}}
>     | align=&quot;right&quot; | 446 550
>     | align=&quot;right&quot; | 31 951 412
>     | align=&quot;right&quot; | 71,5
>     | [[Rabat]]
>     |-
>     | {{flaggbild2|Sudan}}
>     | align=&quot;right&quot; | 1 886 068
>     | align=&quot;right&quot; | 33 551 941
>     | align=&quot;right&quot; | 17,8
>     | [[Khartoum]]
>     |-
>     | {{flaggbild2|Tunisien}}
>     | align=&quot;right&quot; | 163 610
>     | align=&quot;right&quot; | 10 480 934
>     | align=&quot;right&quot; | 64,1
>     | [[Tunis]]
>     |-
>     | {{flaggbild2|Västsahara}} ([[Marocko]])&lt;ref&gt;[[Västsahara]] är ockuperat av [[Marocko]], som också har anspråk på området.&lt;/small&gt;&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 266 000
>     | align=&quot;right&quot; | 530 500
>     | align=&quot;right&quot; | 2,0
>     | [[El Aaiún]]
>     |-
>     | colspan=5 | ''Områden som är likställda delar av europeiska stater'':
>     |-
>     | [[Fil:Flag of the Canary Islands.svg|22x20px|border|Kanarieöarna]] [[Kanarieöarna]] ([[Spanien]])&lt;ref&gt;De till [[Spanien]] hörande [[Kanarieöarna]] brukar geografiskt räknas till Nordafrika på grund av deras relativa närhet till [[Marocko]] och [[Västsahara]]; befolknings- och arealuppgifterna är från år 2001.&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 7 492
>     | align=&quot;right&quot; | 2 117 519
>     | align=&quot;right&quot; | 282,6
>     | [[Las Palmas de Gran Canaria]],&lt;br /&gt;[[Santa Cruz de Tenerife]]
>     |-
>     | [[Fil:Flag Ceuta.svg|22x20px|border|Ceuta]] [[Ceuta]] ([[Spanien]])&lt;ref&gt;Den [[Spanien|spanska]] [[exklav]]en [[Ceuta]] omges på landsidan av Marocko i Nordafrika; befolknings- och arealuppgifterna är från år 2001.&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 20
>     | align=&quot;right&quot; | 78 674
>     | align=&quot;right&quot; | 3933,7
>     | –
>     |-
>     | [[Fil:Flag of Madeira.svg|22x20px|Madeira]] [[Madeira]] ([[Portugal]])&lt;ref&gt;Det till [[Portugal]] hörande [[Madeira]] brukar geografiskt räknas till Nordafrika på grund av deras relativa närhet till [[Marocko]]; befolknings- och arealuppgifterna är från år 2001.&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 797
>     | align=&quot;right&quot; | 267 302
>     | align=&quot;right&quot; | 335,4
>     | [[Funchal]]
>     |-
>     | [[Fil:Flag of Melilla.svg|22x20px|Melilla]] [[Melilla]] ([[Spanien]])&lt;ref&gt;Den [[Spanien|spanska]] [[exklav]]en [[Melilla]] omges på landsidan av Marocko i Nordafrika; befolknings- och arealuppgifterna är från år 2001.&lt;br /&gt;&lt;/ref&gt;
>     | align=&quot;right&quot; | 12
>     | align=&quot;right&quot; | 73 460
>     | align=&quot;right&quot; | 6121,6
>     | –
>     |-
>     | colspan=5 style=&quot;background:#eee;&quot; | '''[[Södra Afrika]]''':
>     |-
>     | {{flaggbild2|Botswana}}
>     | align=&quot;right&quot; | 600 370
>     | align=&quot;right&quot; | 2 006 945
>     | align=&quot;right&quot; | 3,3
>     | [[Gaborone]]
>     |-
>     | {{flaggbild2|Lesotho}}
>     | align=&quot;right&quot; | 30 355
>     | align=&quot;right&quot; | 2 171 318
>     | align=&quot;right&quot; | 71,5
>     | [[Maseru]]
>     |-
>     | {{flaggbild2|Namibia}}
>     | align=&quot;right&quot; | 825 418
>     | align=&quot;right&quot; | 2 283 289
>     | align=&quot;right&quot; | 2,7
>     | [[Windhoek]]
>     |-
>     | {{flaggbild2|Sydafrika}}
>     | align=&quot;right&quot; | 1 219 912
>     | align=&quot;right&quot; | 50 132 817
>     | align=&quot;right&quot; | 41,1
>     | [[Bloemfontein]], [[Kapstaden]], [[Pretoria]] (Tshwane)&lt;ref&gt;[[Bloemfontein]] är [[Sydafrika]]s judiciella huvudstad, medan [[Kapstaden]] är säte för den lagstiftande församlingen och [[Pretoria]] är administrativ huvudstad. Staden Pretoria har inlett en process för att eventuellt byta namn till Tshwane.&lt;br /&gt;&lt;/ref&gt;
>     |-
>     | {{flaggbild2|Swaziland}}
>     | align=&quot;right&quot; | 17 363
>     | align=&quot;right&quot; | 1 186 056
>     | align=&quot;right&quot; | 68,3
>     | [[Mbabane]]
>     |-
>     | colspan=5 style=&quot;background:#eee;&quot; | '''[[Västafrika]]''':
>     |-
>     | {{flaggbild2|Benin}}
>     | align=&quot;right&quot; | 112 620
>     | align=&quot;right&quot; | 8 849 892
>     | align=&quot;right&quot; | 78,6
>     | [[Porto-Novo]]
>     |-
>     | {{flaggbild2|Burkina Faso}}
>     | align=&quot;right&quot; | 274 200
>     | align=&quot;right&quot; | 16 468 714
>     | align=&quot;right&quot; | 60,1
>     | [[Ouagadougou]]
>     |-
>     | {{flaggbild2|Kap Verde}}
>     | align=&quot;right&quot; | 4 033
>     | align=&quot;right&quot; | 495 999
>     | align=&quot;right&quot; | 123,0
>     | [[Praia]]
>     |-
>     | {{flaggbild2|Elfenbenskusten}}
>     | align=&quot;right&quot; | 322 460
>     | align=&quot;right&quot; | 19 737 800
>     | align=&quot;right&quot; | 61,2
>     | [[Abidjan]], [[Yamoussoukro]]&lt;ref&gt;[[Yamoussoukro]] är [[Elfenbenskusten]]s officiella huvudstad, men [[Abidjan]] fungerar ''[[de facto]]'' som huvudstad.&lt;br /&gt;&lt;/ref&gt;
>     |-
>     | {{flaggbild2|Gambia}}
>     | align=&quot;right&quot; | 11 300
>     | align=&quot;right&quot; | 1 728 394
>     | align=&quot;right&quot; | 152,9
>     | [[Banjul]]
>     |-
>     | {{flaggbild2|Ghana}}
>     | align=&quot;right&quot; | 239 460
>     | align=&quot;right&quot; | 24 391 823
>     | align=&quot;right&quot; | 101,9
>     | [[Accra]]
>     |-
>     | {{flaggbild2|Guinea}}
>     | align=&quot;right&quot; | 245 857
>     | align=&quot;right&quot; | 9 981 590
>     | align=&quot;right&quot; | 40,6
>     | [[Conakry]]
>     |-
>     | {{flaggbild2|Guinea-Bissau}}
>     | align=&quot;right&quot; | 36 120
>     | align=&quot;right&quot; | 1 515 224
>     | align=&quot;right&quot; | 41,9
>     | [[Bissau]]
>     |-
>     | {{flaggbild2|Liberia}}
>     | align=&quot;right&quot; | 111 370
>     | align=&quot;right&quot; | 3 994 122
>     | align=&quot;right&quot; | 35,9
>     | [[Monrovia]]
>     |-
>     | {{flaggbild2|Mali}}
>     | align=&quot;right&quot; | 1 240 000
>     | align=&quot;right&quot; | 15 369 809
>     | align=&quot;right&quot; | 9,1
>     | [[Bamako]]
>     |-
>     | {{flaggbild2|Mauretanien}}
>     | align=&quot;right&quot; | 1 030 700
>     | align=&quot;right&quot; | 3 459 773
>     | align=&quot;right&quot; | 3,4
>     | [[Nouakchott]]
>     |-
>     | {{flaggbild2|Niger}}
>     | align=&quot;right&quot; | 1 267 000
>     | align=&quot;right&quot; | 15 511 953
>     | align=&quot;right&quot; | 12,2
>     | [[Niamey]]
>     |-
>     | {{flaggbild2|Nigeria}}
>     | align=&quot;right&quot; | 923 768
>     | align=&quot;right&quot; | 158 423 182
>     | align=&quot;right&quot; | 171,5
>     | [[Abuja]]
>     |-
>     | [[Fil:Flag of Saint Helena.svg|22x20px|Sankta Helena]] [[Sankta Helena (ö)|Sankta Helena]] ([[Storbritannien]])
>     | align=&quot;right&quot; | 410
>     | align=&quot;right&quot; | 4 118
>     | align=&quot;right&quot; | 10,0
>     | [[Jamestown, Sankta Helena|Jamestown]]
>     |-
>     | {{flaggbild2|Senegal}}
>     | align=&quot;right&quot; | 196 190
>     | align=&quot;right&quot; | 12 433 728
>     | align=&quot;right&quot; | 63,4
>     | [[Dakar]]
>     |-
>     | {{flaggbild2|Sierra Leone}}
>     | align=&quot;right&quot; | 71 740
>     | align=&quot;right&quot; | 5 867 536
>     | align=&quot;right&quot; | 81,8
>     | [[Freetown]]
>     |-
>     | {{flaggbild2|Togo}}
>     | align=&quot;right&quot; | 56 785
>     | align=&quot;right&quot; | 6 027 798
>     | align=&quot;right&quot; | 106,1
>     | [[Lomé]]
>     |-
>     | colspan=5 style=&quot;background:#eee;&quot; | '''[[Östafrika]]''':
>     |-
>     | {{flaggbild2|Burundi}}
>     | align=&quot;right&quot; | 27 830
>     | align=&quot;right&quot; | 8 382 849
>     | align=&quot;right&quot; | 301,2
>     | [[Bujumbura]]
>     |-
>     | {{flaggbild2|Komorerna}}
>     | align=&quot;right&quot; | 2 170
>     | align=&quot;right&quot; | 734 750
>     | align=&quot;right&quot; | 338,6
>     | [[Moroni]]
>     |-
>     | {{flaggbild2|Djibouti}}
>     | align=&quot;right&quot; | 23 000
>     | align=&quot;right&quot; | 888 716
>     | align=&quot;right&quot; | 38,6
>     | [[Djibouti (stad)|Djibouti]]
>     |-
>     | {{flaggbild2|Eritrea}}
>     | align=&quot;right&quot; | 121 320
>     | align=&quot;right&quot; | 5 253 676
>     | align=&quot;right&quot; | 43,3
>     | [[Asmara]]
>     |-
>     | {{flaggbild2|Etiopien}}
>     | align=&quot;right&quot; | 1 127 127
>     | align=&quot;right&quot; | 82 949 541
>     | align=&quot;right&quot; | 73,6
>     | [[Addis Abeba]]
>     |-
>     | {{flaggbild2|Kenya}}
>     | align=&quot;right&quot; | 582 650
>     | align=&quot;right&quot; | 40 512 682
>     | align=&quot;right&quot; | 69,5
>     | [[Nairobi]]
>     |-
>     |{{flaggbild2|Madagaskar}}
>     | align=&quot;right&quot; | 587 040
>     | align=&quot;right&quot; | 20 713 819
>     | align=&quot;right&quot; | 35,3
>     | [[Antananarivo]]
>     |-
>     | {{flaggbild2|Malawi}}
>     | align=&quot;right&quot; | 118 480
>     | align=&quot;right&quot; | 14 900 841
>     | align=&quot;right&quot; | 90,3
>     | [[Lilongwe]]
>     |-
>     | {{flaggbild2|Mauritius}}
>     | align=&quot;right&quot; | 2 040
>     | align=&quot;right&quot; | 1 299 172
>     | align=&quot;right&quot; | 636,8
>     | [[Port Louis]]
>     |-
>     | {{flaggbild|Frankrike}} [[Mayotte]] ([[Frankrike]])
>     | align=&quot;right&quot; | 374
>     | align=&quot;right&quot; | 204 114
>     | align=&quot;right&quot; | 545,8
>     | [[Mamoudzou]]
>     |-
>     | {{flaggbild|Moçambique}} [[Moçambique]]
>     | align=&quot;right&quot; | 801 590
>     | align=&quot;right&quot; | 23 390 765
>     | align=&quot;right&quot; | 29,2
>     | [[Maputo]]
>     |-
>     | {{flaggbild|Frankrike}} [[Réunion]] ([[Frankrike]])
>     | align=&quot;right&quot; | 2 512
>     | align=&quot;right&quot; | 846 068
>     | align=&quot;right&quot; | 336,8
>     | [[Saint-Denis (Réunion)|Saint-Denis]]
>     |-
>     | {{flaggbild2|Rwanda}}
>     | align=&quot;right&quot; | 26 338
>     | align=&quot;right&quot; | 10 624 005
>     | align=&quot;right&quot; | 403,4
>     | [[Kigali]]
>     |-
>     | {{flaggbild2|Seychellerna}}
>     | align=&quot;right&quot; | 455
>     | align=&quot;right&quot; | 86 518
>     | align=&quot;right&quot; | 190,1
>     | [[Victoria, Seychellerna|Victoria]]
>     |-
>     | {{flaggbild2|Somalia}}
>     | align=&quot;right&quot; | 637 657
>     | align=&quot;right&quot; | 9 330 872
>     | align=&quot;right&quot; | 14,6
>     | [[Mogadishu]]
>     |-
>     | {{flaggbild2|Sydsudan}}
>     | align=&quot;right&quot; | 633 783
>     | align=&quot;right&quot; | 8 207 607
>     | align=&quot;right&quot; | 13
>     | [[Juba]]
>     |-
>     | {{flaggbild2|Tanzania}}
>     | align=&quot;right&quot; | 945 087
>     | align=&quot;right&quot; | 44 841 226
>     | align=&quot;right&quot; | 47,4
>     | [[Dodoma]]
>     |-
>     | {{flaggbild2|Uganda}}
>     | align=&quot;right&quot; | 236 040
>     | align=&quot;right&quot; | 33 424 683
>     | align=&quot;right&quot; | 141,6
>     | [[Kampala]]
>     |-
>     | {{flaggbild2|Zambia}}
>     | align=&quot;right&quot; | 752 614
>     | align=&quot;right&quot; | 13 088 570
>     | align=&quot;right&quot; | 17,4
>     | [[Lusaka]]
>     |-
>     | {{flaggbild2|Zimbabwe}}
>     | align=&quot;right&quot; | 390 580
>     | align=&quot;right&quot; | 12 571 454
>     | align=&quot;right&quot; | 32,2
>     | [[Harare]]
>     |- style=&quot; font-weight:bold; &quot;
>     | Total
>     | align=&quot;right&quot; | 30&amp;nbsp;305&amp;nbsp;053
>     | align=&quot;right&quot; | 1&amp;nbsp;022&amp;nbsp;234&amp;nbsp;400
>     | align=&quot;right&quot; | 33,7
>     |}
>
>     == Se även ==
>     * [[Nordafrika]]
>     * [[Västafrika]]
>     * [[Östafrika]]
>     * [[Centralafrika]]
>     * [[Södra Afrika]]
>     * [[Afrikas klimat]]
>
>     == Referenser ==
>
>     === Noter ===
>     &lt;references&gt;
>     &lt;ref name=&quot;esa.un.org&quot;&gt;[http://esa.un.org/unpd/wpp/Excel-Data/population.htm/ &quot;World Population Prospects: The 2010 Revision&quot;] {{Wayback|url=http://esa.un.org/unpd/wpp/Excel-Data/population.htm/ |date=20120606170530 }} [[Förenta nationerna|United Nations]] (Department of Economic and Social Affairs, population division)&lt;/ref&gt;
>     &lt;ref name=&quot;esa.un.org1&quot;&gt;[http://esa.un.org/unpd/wpp/Excel-Data/population.htm/ &quot;World Population Prospects: The 2010 Revision&quot;] {{Wayback|url=http://esa.un.org/unpd/wpp/Excel-Data/population.htm/ |date=20120606170530 }} [[United Nations]] (Department of Economic and Social Affairs, population division)&lt;/ref&gt;
>     &lt;/references&gt;
>
>     === Källor ===
>     {{enwp|url=https://en.wikipedia.org/w/index.php?title=Africa&amp;oldid=81702686|artikel=Africa}}
>     * [https://www.courses.psu.edu/test/test100_hkr/AFIM/Main_HTML/Intro_1.html Pennsylvania State University – ''African Ethnic Groups'' och andra kartor]
>     * [http://www.pygmies.info/ Pygméerna i Afrika] Kultur och musik av de första urinvånarna i Afrika
>     * Afrika i skolböckerna, M. Palmberg 1987
>
>     == Externa länkar ==
>     {{commonscat|Africa}}
>     {{Wikinews|Afrika}}
>     {{wikiatlas|Africa}}
>
>     {{Afrika}}
>     {{Världens regioner}}
>     {{auktoritetsdata}}
>
>     [[Kategori:Afrika| ]]
>     [[Kategori:Wikipedia:Basartiklar]]</text>
>           <sha1>1ueywtz2sm89v81ujno96unvkkmxpoa</sha1>
>         </revision>
>       </page>), (55447,<page>
>         <title>Amerika</title>
>         <ns>0</ns>
>         <id>3</id>
>         <revision>
>           <id>48343172</id>
>           <parentid>48187017</parentid>
>           <timestamp>2020-10-11T13:36:09Z</timestamp>
>           <contributor>
>             <username>Olympicbandy</username>
>             <id>564328</id>
>           </contributor>
>           <model>wikitext</model>
>           <format>text/x-wiki</format>
>           <text bytes="2728" xml:space="preserve">{{för|Amerika som kortnamn på '''Amerikas förenta stater'''|USA|andra betydelser|Amerika (olika betydelser)}}
>     [[Fil:Americas (orthographic projection).svg|miniatyr|250px|Karta som visar Nord- och Sydamerika, tillsammans benämnda Amerika]]
>     '''Amerika''' (engelska: ''the Americas'') består av landmassorna mellan [[Atlanten]] och [[Stilla havet]] på [[västra halvklotet]], namngivna efter den italienske upptäcktsresanden [[Amerigo Vespucci]]. Amerika uppfattas vanligen idag som två världsdelar, [[Nordamerika]] och [[Sydamerika]].&lt;ref&gt;[https://snl.no/Amerika Store norske leksikon/ Amerika]&lt;/ref&gt; Amerika indelas geografiskt i Nordamerika (med [[Centralamerika]]) och Sydamerika. [[Grönland]] ingår geografiskt i Nordamerika, men räknas politiskt och kulturellt till Europa. Kulturgeografiskt avgränsas också ofta [[Latinamerika]], då inklusive både Central- och...

  

Keep only articles
------------------

In [ ]:
import scala.util.matching.UnanchoredRegex
var regex = s"<ns>0</ns>".r
val articles_dump = dump.filter({ case (id, source) => regex.findFirstIn(source.toString()) != None})
articles_dump.take(1)

  

>     import scala.util.matching.UnanchoredRegex
>     regex: scala.util.matching.Regex = <ns>0</ns>
>     articles_dump: org.apache.spark.rdd.RDD[(Long, String)] = MapPartitionsRDD[564] at filter at command-2158711750341777:3
>     res1: Array[(Long, String)] =
>     Array((10671,<page>
>         <title>Amager</title>
>         <ns>0</ns>
>         <id>1</id>
>         <revision>
>           <id>48450724</id>
>           <parentid>48155628</parentid>
>           <timestamp>2020-11-15T15:50:09Z</timestamp>
>           <contributor>
>             <username>Nalle&amp;Lisa</username>
>             <id>348723</id>
>           </contributor>
>           <comment>Länkfix</comment>
>           <model>wikitext</model>
>           <format>text/x-wiki</format>
>           <text bytes="7485" xml:space="preserve">{{källor|datum=2013-05}}
>
>     {{Geobox|Island
>     &lt;!-- *** Mallhuvud *** --&gt;
>     | name                  = Amager
>     | native_name           =
>     | other_name            =
>     | category              = Ö
>     &lt;!-- *** Namn **** --&gt;
>     | etymology             =
>     | nickname              =
>     &lt;!-- *** Bild *** --&gt;
>     | image                 = Amager.jpg
>     | image_caption         =
>     &lt;!-- *** Länder, regioner mm *** --&gt;
>     | country               = Danmark
>     | country_flag          = true
>     | state                 = | state_type =
>     | region                = [[Region Hovedstaden]]| region_type =
>     | district              = | district_type =
>     | municipality          = [[Dragørs kommun]]| municipality_type =
>     | municipality1         = [[Köpenhamns kommun]]
>     | municipality2         = [[Tårnby kommun]]
>     &lt;!-- *** Tillhörighet, föremål inom området mm *** --&gt;
>     | parent                =
>     | range                 = | range_type = Arkipelag
>     | border                =
>     | part                  =
>     | city                  =
>     | capital               =
>     | landmark              =
>     | river                 =
>     &lt;!-- *** Läge *** --&gt;
>     | location              = [[Öresund]]
>     | lat_d                 = 55| lat_m = 37| lat_s = 0| lat_NS = N
>     | long_d                = 12| long_m = 37| long_s = 0| long_EW = E
>     | highest               =
>     | highest_location      = | highest_region = | highest_country =
>     | highest_elevation     =
>     | highest_lat_d         = | highest_lat_m = | highest_lat_s = | highest_lat_NS =
>     | highest_long_d        = | highest_long_m = | highest_long_s = | highest_long_EW =
>     &lt;!-- *** Dimensioner *** --&gt;
>     | length                = 16.1| length_orientation =
>     | width                 = 11.1| width_orientation =
>     | height                = 4
>     | area                  = 96.29
>     &lt;!-- *** Befolkning *** --&gt;
>     | population            = 196047| population_date = 1 januari 2018 | population_note =
>     | population_density    = auto| population_density_note =
>     &lt;!-- *** Tillgänglighet *** --&gt;
>     | public                =
>     | access                =
>     &lt;!-- *** Koder ***  --&gt;
>     | timezone              = CET| utc_offset = +1
>     | timezone_DST          = CEST| utc_offset_DST = +2
>     | postal_code           =
>     | area_code             =
>     | code                  =
>     &lt;!-- *** Fria fält *** --&gt;
>     | free                  =
>     | free_type             =
>     &lt;!-- *** Kartor *** --&gt;
>     | map                   = Denmark physical map.svg
>     | map_caption           = Amagers läge i Danmark.
>     | map_locator           = Danmark
>     | map_locator_x         =
>     | map_locator_y         =
>     &lt;!-- *** Fotnoter *** --&gt;
>     | footnotes             =
>     }}
>
>     '''Amager''' är en [[Danmark|dansk]] [[ö (landområde)|ö]] i [[Öresund]]. Öns norra och västra delar tillhör [[Köpenhamn]], medan övriga delar upptas av [[Tårnby kommun]] och [[Dragørs kommun]].
>     Amager har en yta på 96,29 km² och befolkningen uppgår till 196&amp;nbsp;047 personer (1/1 2018). På den östra delen av ön finns [[Kastrups flygplats]]. Amager har också flera militära anläggningar (t.ex. Kastrup fort och Dragör fort), strandparksområdena [[Amager Strandpark]] och Kastrup Strandpark, flera restauranger i världsklass (t.ex. Noma), ett stort naturområde ([[Amager Fælled]]), ett av Nordens största köpcentra (Field's), och mycket annat. Amager förkortas ofta Ama'r för att återspegla det danska uttalet.
>
>     Amager är delvis en [[konstgjord ö]], delvis en naturlig sådan. T.ex. så anlades en konstgjord ö i anslutning till det redan existerande Amager Strandpark, när denna rustades upp väsentligen 2005. Ön är mycket låg och vissa delar ligger under havsytan, framför allt det genom fördämning och utfyllnad skapade området [[Kalvebod Fælled]] (Vestamager). Ön började uppodlas i större skala under 1500-talets första hälft då kung [[Kristian II]] bjöd in nederländska bönder till att odla upp ön för kronans räkning. Flertalet slog sig ned i fiskeläget [[Dragør]] där många än idag har namn med nederländskt ursprung som Neels och Tønnes.
>
>     Den stora utvecklingsfasen började på 1800-talet då flera fabriker, bostadsområden och militär verksamhet utvecklade sig på ön. Det fanns ett antal stora och kända fabriker på ön och den industriella prägeln höll sig väldigt tydlig långt in på 1900-talet. Det förekommer också områden som tidigare varit sommarstugeområden och där det än idag finns kolonilottområden, t.ex. i Sundbyöster ut mot kusten. Under 1900-talet började militären att utveckla stora övningsområden på ön vilket resulterade i Vestamager som idag innefattar ett stort naturområde endast några kilometer från Köpenhamns centrum. Under senare delen av 1900-talet har flygplatsen [[Kastrups flygplats|Kastrup]] som ligger på ön expanderat kraftigt och i samband med byggandet av [[Öresundsförbindelsen]] började även ett stort område byggas som kallas för [[Ørestad]].
>
>     Ön, som är förbunden med övriga Köpenhamn genom flera broar, [[järnväg]] och sedan 2017 även [[Köpenhamns tunnelbana|Metro]], har under de senaste 10-20 åren förändrats mycket. Dels har flera områden exploaterats med nya moderna kvarter lägenheter, t.ex. i områdena Örestad, [[Islands Brygge]] och Amager Strand, och dels så har Amagers närhet till Köpenhamn raderat dess tidigare rykte som en industriell förstad till Köpenhamn. Idag finns det områden av alla sorter på ön - tack vare flygplatsen finns ett aktivt näringsliv kvar, samtidigt som de nya områdena och inte minst upprustningen av t.ex. Amager Strand har förändrat bilden av ön. Idag är flera delar av ön mycket attraktiva bostadsområden.
>
>     Ön återfick i samband med Öresundsbron även en järnvägsförbindelse vilket inte funnits sedan 1960-talet. En [[motorväg]] byggdes samtidigt över öns mellersta del. [[Köpenhamns metro]] går till Kastrup och Vestamager med ett antal stationer på ön. Kring metro- och regionaltågsstationen Ørestad har ett stort kontors-, handels- och bostadsområde uppförts under [[2000-talet (decennium)|00-talet]]. Detta sträcker sig från [[Bella Center]] i norr till metrostationen [[Vestamager station|Vestamager]] i söder. Även om området inte ligger vid kusten har Ørestad delvis fått karaktären av en hamnstad genom anläggandet av konstgjorda kanaler och dammar. Vid [[Islands Brygge]] finns en &quot;äkta&quot; sjöstad.
>
>     Före [[Öresundsförbindelsen]]s uppförande hade småstaden [[Dragør]] en färjeförbindelse med [[Limhamn]] i södra [[Malmö]]. På grund av denna fanns i Dragør tidigare en betydande gränshandel. Denna har under 2000-talet helt upphört liksom den genomgående trafiken.
>
>     Öns västra del, som utvanns från Öresund under 1920- och 30-talen, är med undantag för Bella Center, Ørestad samt ett koloniområde, avsatt som natur- och rekreationsområde. Marken är ängsmark (odlad mark finns dock på den naturliga delen av öns södra del).
>     Längst i söder finns den flerhundraåriga skogen [[Kongelunden]]. På öns östra del, vid Öresundskusten, finns [[Amager Strandpark]] med en populär sandstrand. Området har omgestaltats, med en [[konstgjord ö]] och en [[lagun]] innanför. Nyinvigningen av parken, som funnits sedan 1934, ägde rum 2005.
>
>     Amager är den tätast befolkade ön i Danmark. Den näst mest tätbefolkade är [[Thurø]].
>
>     == Se även ==
>     *[[Amagerbro]]
>     *[[Amagerport]]
>     *[[Amagerbroderi]]
>
>     ==Externa länkar==
>     {{commonscat|Amager}}
>
>     {{Öar i Danmark}}
>
>     {{auktoritetsdata}}
>
>     [[Kategori:Amager| ]]
>     [[Kategori:Öar i Region Hovedstaden]]</text>
>           <sha1>qom8qpk70qa0alvfb4b4ha9or4ftohh</sha1>
>         </revision>
>       </page>))

In [ ]:
import scala.util.matching.UnanchoredRegex
var regex = s"<redirect title".r
val filtered_dump = articles_dump.filter({ case (id, source) => regex.findFirstIn(source.toString()) == None})
filtered_dump.take(1)

  

>     import scala.util.matching.UnanchoredRegex
>     regex: scala.util.matching.Regex = <redirect title
>     filtered_dump: org.apache.spark.rdd.RDD[(Long, String)] = MapPartitionsRDD[565] at filter at command-2158711750341794:3
>     res2: Array[(Long, String)] =
>     Array((10671,<page>
>         <title>Amager</title>
>         <ns>0</ns>
>         <id>1</id>
>         <revision>
>           <id>48450724</id>
>           <parentid>48155628</parentid>
>           <timestamp>2020-11-15T15:50:09Z</timestamp>
>           <contributor>
>             <username>Nalle&amp;Lisa</username>
>             <id>348723</id>
>           </contributor>
>           <comment>Länkfix</comment>
>           <model>wikitext</model>
>           <format>text/x-wiki</format>
>           <text bytes="7485" xml:space="preserve">{{källor|datum=2013-05}}
>
>     {{Geobox|Island
>     &lt;!-- *** Mallhuvud *** --&gt;
>     | name                  = Amager
>     | native_name           =
>     | other_name            =
>     | category              = Ö
>     &lt;!-- *** Namn **** --&gt;
>     | etymology             =
>     | nickname              =
>     &lt;!-- *** Bild *** --&gt;
>     | image                 = Amager.jpg
>     | image_caption         =
>     &lt;!-- *** Länder, regioner mm *** --&gt;
>     | country               = Danmark
>     | country_flag          = true
>     | state                 = | state_type =
>     | region                = [[Region Hovedstaden]]| region_type =
>     | district              = | district_type =
>     | municipality          = [[Dragørs kommun]]| municipality_type =
>     | municipality1         = [[Köpenhamns kommun]]
>     | municipality2         = [[Tårnby kommun]]
>     &lt;!-- *** Tillhörighet, föremål inom området mm *** --&gt;
>     | parent                =
>     | range                 = | range_type = Arkipelag
>     | border                =
>     | part                  =
>     | city                  =
>     | capital               =
>     | landmark              =
>     | river                 =
>     &lt;!-- *** Läge *** --&gt;
>     | location              = [[Öresund]]
>     | lat_d                 = 55| lat_m = 37| lat_s = 0| lat_NS = N
>     | long_d                = 12| long_m = 37| long_s = 0| long_EW = E
>     | highest               =
>     | highest_location      = | highest_region = | highest_country =
>     | highest_elevation     =
>     | highest_lat_d         = | highest_lat_m = | highest_lat_s = | highest_lat_NS =
>     | highest_long_d        = | highest_long_m = | highest_long_s = | highest_long_EW =
>     &lt;!-- *** Dimensioner *** --&gt;
>     | length                = 16.1| length_orientation =
>     | width                 = 11.1| width_orientation =
>     | height                = 4
>     | area                  = 96.29
>     &lt;!-- *** Befolkning *** --&gt;
>     | population            = 196047| population_date = 1 januari 2018 | population_note =
>     | population_density    = auto| population_density_note =
>     &lt;!-- *** Tillgänglighet *** --&gt;
>     | public                =
>     | access                =
>     &lt;!-- *** Koder ***  --&gt;
>     | timezone              = CET| utc_offset = +1
>     | timezone_DST          = CEST| utc_offset_DST = +2
>     | postal_code           =
>     | area_code             =
>     | code                  =
>     &lt;!-- *** Fria fält *** --&gt;
>     | free                  =
>     | free_type             =
>     &lt;!-- *** Kartor *** --&gt;
>     | map                   = Denmark physical map.svg
>     | map_caption           = Amagers läge i Danmark.
>     | map_locator           = Danmark
>     | map_locator_x         =
>     | map_locator_y         =
>     &lt;!-- *** Fotnoter *** --&gt;
>     | footnotes             =
>     }}
>
>     '''Amager''' är en [[Danmark|dansk]] [[ö (landområde)|ö]] i [[Öresund]]. Öns norra och västra delar tillhör [[Köpenhamn]], medan övriga delar upptas av [[Tårnby kommun]] och [[Dragørs kommun]].
>     Amager har en yta på 96,29 km² och befolkningen uppgår till 196&amp;nbsp;047 personer (1/1 2018). På den östra delen av ön finns [[Kastrups flygplats]]. Amager har också flera militära anläggningar (t.ex. Kastrup fort och Dragör fort), strandparksområdena [[Amager Strandpark]] och Kastrup Strandpark, flera restauranger i världsklass (t.ex. Noma), ett stort naturområde ([[Amager Fælled]]), ett av Nordens största köpcentra (Field's), och mycket annat. Amager förkortas ofta Ama'r för att återspegla det danska uttalet.
>
>     Amager är delvis en [[konstgjord ö]], delvis en naturlig sådan. T.ex. så anlades en konstgjord ö i anslutning till det redan existerande Amager Strandpark, när denna rustades upp väsentligen 2005. Ön är mycket låg och vissa delar ligger under havsytan, framför allt det genom fördämning och utfyllnad skapade området [[Kalvebod Fælled]] (Vestamager). Ön började uppodlas i större skala under 1500-talets första hälft då kung [[Kristian II]] bjöd in nederländska bönder till att odla upp ön för kronans räkning. Flertalet slog sig ned i fiskeläget [[Dragør]] där många än idag har namn med nederländskt ursprung som Neels och Tønnes.
>
>     Den stora utvecklingsfasen började på 1800-talet då flera fabriker, bostadsområden och militär verksamhet utvecklade sig på ön. Det fanns ett antal stora och kända fabriker på ön och den industriella prägeln höll sig väldigt tydlig långt in på 1900-talet. Det förekommer också områden som tidigare varit sommarstugeområden och där det än idag finns kolonilottområden, t.ex. i Sundbyöster ut mot kusten. Under 1900-talet började militären att utveckla stora övningsområden på ön vilket resulterade i Vestamager som idag innefattar ett stort naturområde endast några kilometer från Köpenhamns centrum. Under senare delen av 1900-talet har flygplatsen [[Kastrups flygplats|Kastrup]] som ligger på ön expanderat kraftigt och i samband med byggandet av [[Öresundsförbindelsen]] började även ett stort område byggas som kallas för [[Ørestad]].
>
>     Ön, som är förbunden med övriga Köpenhamn genom flera broar, [[järnväg]] och sedan 2017 även [[Köpenhamns tunnelbana|Metro]], har under de senaste 10-20 åren förändrats mycket. Dels har flera områden exploaterats med nya moderna kvarter lägenheter, t.ex. i områdena Örestad, [[Islands Brygge]] och Amager Strand, och dels så har Amagers närhet till Köpenhamn raderat dess tidigare rykte som en industriell förstad till Köpenhamn. Idag finns det områden av alla sorter på ön - tack vare flygplatsen finns ett aktivt näringsliv kvar, samtidigt som de nya områdena och inte minst upprustningen av t.ex. Amager Strand har förändrat bilden av ön. Idag är flera delar av ön mycket attraktiva bostadsområden.
>
>     Ön återfick i samband med Öresundsbron även en järnvägsförbindelse vilket inte funnits sedan 1960-talet. En [[motorväg]] byggdes samtidigt över öns mellersta del. [[Köpenhamns metro]] går till Kastrup och Vestamager med ett antal stationer på ön. Kring metro- och regionaltågsstationen Ørestad har ett stort kontors-, handels- och bostadsområde uppförts under [[2000-talet (decennium)|00-talet]]. Detta sträcker sig från [[Bella Center]] i norr till metrostationen [[Vestamager station|Vestamager]] i söder. Även om området inte ligger vid kusten har Ørestad delvis fått karaktären av en hamnstad genom anläggandet av konstgjorda kanaler och dammar. Vid [[Islands Brygge]] finns en &quot;äkta&quot; sjöstad.
>
>     Före [[Öresundsförbindelsen]]s uppförande hade småstaden [[Dragør]] en färjeförbindelse med [[Limhamn]] i södra [[Malmö]]. På grund av denna fanns i Dragør tidigare en betydande gränshandel. Denna har under 2000-talet helt upphört liksom den genomgående trafiken.
>
>     Öns västra del, som utvanns från Öresund under 1920- och 30-talen, är med undantag för Bella Center, Ørestad samt ett koloniområde, avsatt som natur- och rekreationsområde. Marken är ängsmark (odlad mark finns dock på den naturliga delen av öns södra del).
>     Längst i söder finns den flerhundraåriga skogen [[Kongelunden]]. På öns östra del, vid Öresundskusten, finns [[Amager Strandpark]] med en populär sandstrand. Området har omgestaltats, med en [[konstgjord ö]] och en [[lagun]] innanför. Nyinvigningen av parken, som funnits sedan 1934, ägde rum 2005.
>
>     Amager är den tätast befolkade ön i Danmark. Den näst mest tätbefolkade är [[Thurø]].
>
>     == Se även ==
>     *[[Amagerbro]]
>     *[[Amagerport]]
>     *[[Amagerbroderi]]
>
>     ==Externa länkar==
>     {{commonscat|Amager}}
>
>     {{Öar i Danmark}}
>
>     {{auktoritetsdata}}
>
>     [[Kategori:Amager| ]]
>     [[Kategori:Öar i Region Hovedstaden]]</text>
>           <sha1>qom8qpk70qa0alvfb4b4ha9or4ftohh</sha1>
>         </revision>
>       </page>))

In [ ]:
val corpus_dump = filtered_dump

  

>     corpus_dump: org.apache.spark.rdd.RDD[(Long, String)] = MapPartitionsRDD[565] at filter at command-2158711750341794:3

  

Filtering
=========

In [ ]:
def filterSourceFail(source: String) : String = {
  
  // There is an &amp;nbsp; where the &amp; provides the & in &nbsp;
  // I should therefore replace &amp; first
  val noAmp = raw"&amp;".r.replaceAllIn(source, "&")
  
  // Loop to remove some specific sections
  val sections = Seq("Se även", "Referenser", "Noter", "Källor", "Externa länkar")
  var current = noAmp
  for (section <- sections) {
    val regexp = s"[=]+ *$section *[=]+".r
    if (regexp.findAllIn(current).hasNext)
    {
      val a = regexp.findAllIn(current).start
      val pre = current.substring(0, a)
      var post = current.substring(a, current.length)
      post = regexp.replaceAllIn(post, "")
      if ("^=".r.findAllIn(post).hasNext) {
        val b = "^=".r.findAllIn(post).start
        post = post.substring(b, post.length)
      }
      else {
        post = ""        
      }
      current = pre + post

    }
  }

  val lowerCase = current.toLowerCase  
  
  // Loop to remove some specific tags
  val tags = Seq("id", "ns", "parentid", "timestamp", "contributor", "comment", "model", "format", "sha1")
  current = lowerCase
  for (tag <- tags) {
    val start = s"<$tag>".r
    val end = s"</$tag>".r
    while (start.findAllIn(current).hasNext) {
      val a = start.findAllIn(current).start
      val b = end.findAllIn(current).start
      val len = current.length
      current = current.substring(0, a) + current.substring(b+tag.length+3, current.length)
      assert (current.length < len)
    }

  }
  val removedSpecificTags = current

  val noTags = raw"<[^<>]*>".r.replaceAllIn(removedSpecificTags, "")
  
  // A loop to remove innermost curly braces, then outer such things.
  val curly = raw"\{[^\{\}]*\}".r
  current = noTags
  while (curly.findAllIn(current).hasNext)
  {
     current = curly.replaceAllIn(current, "")
  }
  val noCurly = current
  
  // val noNumbers = raw"\d".r.replaceAllIn(noCurly, "")
  
  // Unnamed internal links appear in figure captions, confusing later matching
  val noUnnamedInternalLinks = raw"\[\[([^\|\]]*)\]\]".r.replaceAllIn(noCurly, m => m.group(1))

  // Loop to remove internal links and their properties
  current = noUnnamedInternalLinks
  val regexp = raw"\[\[[^\|\[]*\|([^\[\]]*)\]\]".r 
  while (regexp.findAllIn(current).hasNext) {
    val len = current.length
    current = regexp.replaceAllIn(current, m => "[[" + m.group(1) + "]]")
    assert (current.length < len)
  }
  val noNamedInternalLinks = current
  
  val noInternalLinks = raw"\[\[([^\]]*)\]\]".r.replaceAllIn(noNamedInternalLinks, m => m.group(1))
  val noNamedExternalLinks = raw"\[[^ \[]*\ ([^\]]*)\]".r.replaceAllIn(noInternalLinks, m => m.group(1))
  val noExternalLinks = raw"\[([^\]]*)\]".r.replaceAllIn(noNamedExternalLinks, m => m.group(1))
  
  val nbsp = raw"&nbsp;".r.replaceAllIn(noExternalLinks, " ")
  val noQuotes = raw"&quot;".r.replaceAllIn(nbsp, "")
  current = noQuotes
  /*
  val noQuoteChars = raw"'".r.replaceAllIn(noQuotes, "")
  // Loop to remove double whitespace characters
  current = noQuoteChars
  val doubleWhitespace = raw"\s(\s)".r
  while (doubleWhitespace.findAllIn(current).hasNext) {
    current = doubleWhitespace.replaceAllIn(current, m => m.group(1))
  }
  */
  val noDoubleWhitespace = current
  noDoubleWhitespace
}

  

>     filterSourceFail: (source: String)String

In [ ]:
import java.util.regex.Matcher;
import java.io.{PrintWriter, StringWriter}

def getStackTraceAsString(t: Throwable) = {
    val sw = new StringWriter
    t.printStackTrace(new PrintWriter(sw))
    sw.toString
}

/**
  * Represents a parsed Wikipedia page from the Wikipedia XML dump
  *
  * https://en.wikipedia.org/wiki/Wikipedia:Database_download
  * https://meta.wikimedia.org/wiki/Data_dump_torrents#enwiki
  *
  * @param title Title of the current page
  * @param text Text of the current page including markup
  * @param isCategory Is the page a category page, not perfectly accurate
  * @param isFile Is the page a file page, not perfectly accurate
  * @param isTemplate Is the page a template page, not perfectly accurate
  */
case class Page(title: String, text: String, status: String, space: String) // isCategory: Boolean , isFile: Boolean, isTemplate: Boolean)

def filterSource(source: String) : Page = {
  var current = source
  var regex = "none".r
  var out = ""
  var title = ""
  var status = ""
  var space = ""
  try {
    var tag = "title"
    regex = s"<$tag[^>]*>((.*:)?.*)</$tag>".r
    var m = regex.findAllIn(current)
    title = m.group(1)
    space = m.group(2)

    tag = "text"
    regex = s"(?s)<$tag[^>]*>(.*?)</$tag>".r
    current = regex.findAllIn(current).group(1)
    val noXML = current
    
    // There is an &amp;nbsp; where the &amp; provides the & in &nbsp;
    // I should therefore replace &amp; first
    val noAmp = raw"&amp;".r.replaceAllIn(current, "&")
    current = noAmp

    // Loop to remove some specific sections
    val sections = Seq("Se även", "Referenser", "Noter", "Källor", "Externa länkar")
    for (section <- sections) {
      val regexp = s"[=]+ *$section *[=]+".r
      if (regexp.findAllIn(current).hasNext)
      {
        val a = regexp.findAllIn(current).start
        val pre = current.substring(0, a)
        var post = current.substring(a, current.length)
        post = regexp.replaceAllIn(post, "")
        if ("^=".r.findAllIn(post).hasNext) {
          val b = "^=".r.findAllIn(post).start
          post = post.substring(b, post.length)
        }
        else {
          post = ""        
        }
        current = pre + post
      }
    }

    val lowerCase = current.toLowerCase  

    val noTags = raw"<[^<>]*>".r.replaceAllIn(lowerCase, "")

    // A loop to remove innermost curly braces, then outer such things.
    val curly = raw"\{[^\{\}]*\}".r
    current = noTags
    while (curly.findAllIn(current).hasNext)
    {
       current = curly.replaceAllIn(current, "")
    }
    val noCurly = current

    // val noNumbers = raw"\d".r.replaceAllIn(noCurly, "")

    // Unnamed internal links appear in figure captions, confusing later matching
    val noUnnamedInternalLinks = raw"\[\[([^\|\]]*)\]\]".r.replaceAllIn(noCurly, m => Matcher.quoteReplacement(m.group(1)))

    // Loop to remove internal links and their properties
    current = noUnnamedInternalLinks
    val regexp = raw"\[\[[^\]\[]*\|([^\[\]]*)\]\]".r 
    while (regexp.findAllIn(current).hasNext) {
      val len = current.length
      current = regexp.replaceAllIn(current, m => Matcher.quoteReplacement(m.group(1)))
      assert (current.length < len)
    }
    val noNamedInternalLinks = current

    val noInternalLinks = raw"\[\[([^\]]*)\]\]".r.replaceAllIn(noNamedInternalLinks, m => Matcher.quoteReplacement(m.group(1)))
    current = noInternalLinks
    val noNamedExternalLinks = raw"\[[^ \[]*\ ([^\]]*)\]".r.replaceAllIn(noInternalLinks, m => Matcher.quoteReplacement(m.group(1)))
    current = noNamedExternalLinks
    val noExternalLinks = raw"\[([^\]]*)\]".r.replaceAllIn(noNamedExternalLinks, m => Matcher.quoteReplacement(m.group(1)))
    current = noExternalLinks

    val nbsp = raw"&nbsp;".r.replaceAllIn(noExternalLinks, " ")
    val noQuotes = raw"&quot;".r.replaceAllIn(nbsp, "")
    current = noQuotes
    val noQuoteChars = raw"''+".r.replaceAllIn(noQuotes, "")
    // Loop to remove double whitespace characters
    current = noQuoteChars
    /*
    val doubleWhitespace = raw"\s(\s)".r
    while (doubleWhitespace.findAllIn(current).hasNext) {
      current = doubleWhitespace.replaceAllIn(current, m => Matcher.quoteReplacement(m.group(1)))
    }
    */
    val noDoubleWhitespace = current
  }
  catch {
    case anyException: Throwable => {out += s"**EXCEPTION**\n$regex\n"; status += getStackTraceAsString(anyException) + "\n"}
  }
  finally {
    out += current
  }
  return Page(title,out,status,space)
}


  

>     import java.util.regex.Matcher
>     import java.io.{PrintWriter, StringWriter}
>     getStackTraceAsString: (t: Throwable)String
>     defined class Page
>     filterSource: (source: String)Page

In [ ]:
val corpus = corpus_dump.map({ case (id, source) => (id, filterSource(source))}).cache() // let's cache


  

>     corpus: org.apache.spark.rdd.RDD[(Long, Page)] = MapPartitionsRDD[566] at map at command-738986526041628:1

In [ ]:
corpus.take(5)

  

>     res3: Array[(Long, Page)] =
>     Array((10671,Page(Amager,
>
>
>
>     amager är en dansk ö i öresund. öns norra och västra delar tillhör köpenhamn, medan övriga delar upptas av tårnby kommun och dragørs kommun.
>     amager har en yta på 96,29 km² och befolkningen uppgår till 196 047 personer (1/1 2018). på den östra delen av ön finns kastrups flygplats. amager har också flera militära anläggningar (t.ex. kastrup fort och dragör fort), strandparksområdena amager strandpark och kastrup strandpark, flera restauranger i världsklass (t.ex. noma), ett stort naturområde (amager fælled), ett av nordens största köpcentra (field's), och mycket annat. amager förkortas ofta ama'r för att återspegla det danska uttalet.
>
>     amager är delvis en konstgjord ö, delvis en naturlig sådan. t.ex. så anlades en konstgjord ö i anslutning till det redan existerande amager strandpark, när denna rustades upp väsentligen 2005. ön är mycket låg och vissa delar ligger under havsytan, framför allt det genom fördämning och utfyllnad skapade området kalvebod fælled (vestamager). ön började uppodlas i större skala under 1500-talets första hälft då kung kristian ii bjöd in nederländska bönder till att odla upp ön för kronans räkning. flertalet slog sig ned i fiskeläget dragør där många än idag har namn med nederländskt ursprung som neels och tønnes.
>
>     den stora utvecklingsfasen började på 1800-talet då flera fabriker, bostadsområden och militär verksamhet utvecklade sig på ön. det fanns ett antal stora och kända fabriker på ön och den industriella prägeln höll sig väldigt tydlig långt in på 1900-talet. det förekommer också områden som tidigare varit sommarstugeområden och där det än idag finns kolonilottområden, t.ex. i sundbyöster ut mot kusten. under 1900-talet började militären att utveckla stora övningsområden på ön vilket resulterade i vestamager som idag innefattar ett stort naturområde endast några kilometer från köpenhamns centrum. under senare delen av 1900-talet har flygplatsen kastrup som ligger på ön expanderat kraftigt och i samband med byggandet av öresundsförbindelsen började även ett stort område byggas som kallas för ørestad.
>
>     ön, som är förbunden med övriga köpenhamn genom flera broar, järnväg och sedan 2017 även metro, har under de senaste 10-20 åren förändrats mycket. dels har flera områden exploaterats med nya moderna kvarter lägenheter, t.ex. i områdena örestad, islands brygge och amager strand, och dels så har amagers närhet till köpenhamn raderat dess tidigare rykte som en industriell förstad till köpenhamn. idag finns det områden av alla sorter på ön - tack vare flygplatsen finns ett aktivt näringsliv kvar, samtidigt som de nya områdena och inte minst upprustningen av t.ex. amager strand har förändrat bilden av ön. idag är flera delar av ön mycket attraktiva bostadsområden.
>
>     ön återfick i samband med öresundsbron även en järnvägsförbindelse vilket inte funnits sedan 1960-talet. en motorväg byggdes samtidigt över öns mellersta del. köpenhamns metro går till kastrup och vestamager med ett antal stationer på ön. kring metro- och regionaltågsstationen ørestad har ett stort kontors-, handels- och bostadsområde uppförts under 00-talet. detta sträcker sig från bella center i norr till metrostationen vestamager i söder. även om området inte ligger vid kusten har ørestad delvis fått karaktären av en hamnstad genom anläggandet av konstgjorda kanaler och dammar. vid islands brygge finns en äkta sjöstad.
>
>     före öresundsförbindelsens uppförande hade småstaden dragør en färjeförbindelse med limhamn i södra malmö. på grund av denna fanns i dragør tidigare en betydande gränshandel. denna har under 2000-talet helt upphört liksom den genomgående trafiken.
>
>     öns västra del, som utvanns från öresund under 1920- och 30-talen, är med undantag för bella center, ørestad samt ett koloniområde, avsatt som natur- och rekreationsområde. marken är ängsmark (odlad mark finns dock på den naturliga delen av öns södra del).
>     längst i söder finns den flerhundraåriga skogen kongelunden. på öns östra del, vid öresundskusten, finns amager strandpark med en populär sandstrand. området har omgestaltats, med en konstgjord ö och en lagun innanför. nyinvigningen av parken, som funnits sedan 1934, ägde rum 2005.
>
>     amager är den tätast befolkade ön i danmark. den näst mest tätbefolkade är thurø.
>
>     ,,null)), (52187,Page(Afrika,
>
>     afrika är jordens näst största kontinent (efter eurasien) och även jordens näst största världsdel efter asien, både vad gäller areal och folkmängd. med världsdelens öar inräknade mäter afrika 30 244 050 km², vilket motsvarar 20,3 % av jordens landmassa eller cirka 6 % av jordens totala area. omkring 22 miljoner km² av dessa ligger i tropikerna vilket gör den afrikanska kontinenten till världens varmaste kontinent. i afrika bor det ungefär 1 miljard människor i 55 länder – en sjundedel av jordens befolkning. dess längd i nordlig-sydlig riktning är omkring 8 000 km och dess största bredd omkring 7 800 km.
>
>     == etymologi ==
>     ordet afrikas etymologiska bakgrund är dunkel, och ordet har dessutom haft olika betydelse under olika epoker.
>
>     i äldre tider kallades allt land väster om egypten för libyen. enligt den grekiska historieskrivaren herodotos berättelse seglade feniciska sjömän på uppdrag av den egyptiska kungen neko runt libyen, det vill säga kontinenten afrika redan omkring 600 f.kr.
>
>     när romarna först använde termen africa terra avsåg de före andra puniska kriget endast trakten kring karthago, det vill säga motsvarande ungefär dagens tunisien där själva staden karthago låg samt stora delar av libyen och algeriet. romarna uppfattade först egypten som en del av asien, och det var först geografen ptolemaios som drog gränsen vid suez. under kejsartiden kom africa att avse hela det området romarna kände till av kontinenten, och allt eftersom européerna har upptäckt mer och mer av kontinenten har termen avsett ett allt större område. namnet ska ha avsett landet där afri (singularis: afer) bodde. afer avsåg namnet på ett folk som kan motsvara dagens berber. vikingarna under 800- och 900-talen kallade norra afrika för blåland.
>
>     == geografi ==
>
>     geologiskt är afrika en mycket gammal kontinent, i huvudsak uppbyggd av urberg. under jordens forntid var den en del av jättekontinenten gondwanaland. denna gled isär under juraperioden och kritaperioden för 100-150 miljoner år sedan, och så fick afrika sin nuvarande form.
>
>     inom sina regelbundna kuster rymmer den en total yta på omkring 30 244 050 km² inklusive öarna. afrika åtskiljs från europa av medelhavet och förenas med asien i sitt nordöstra hörn genom näset vid suez, omkring 130 km i bredd. omkring 8 000 km skiljer den nordligaste punkten, ras ben sakka i tunisien, från dess sydligaste, kap agulhas i sydafrika. från kap verde, dess västligaste punkt, till ras hafun i somalia är det omkring 7 400 km. afrikas kuster mäter omkring 28 000 km (1 km kust på 1 057 km²) varav medelhavet står för 5 400 km, atlanten 11 000 km, indiska oceanen 8 700 km och röda havet 2 900 km, och avsaknaden av djupare bukter och betydande halvöar blir uppenbar om man jämför med europas 9 700 000 km² som omgärdas av 32 000 km kust (1 km kust på 278 km² land), eller för det fjordrika norge, där motsvarande siffra 385 000 km² land och en kuststräcka på 20 000 km (1 km kust per 19 km² land).
>
>     afrikas genomsnittliga höjd över havet är omkring 600 m, vilket motsvarar de båda amerikanska kontinenternas höjd men är lägre än den asiatiska på omkring 950 m. jämfört med de andra kontinenterna utmärker sig afrika med en liten andel landområden på mycket hög respektive mycket låg höjd. landområdena under 180 m ö.h. upptar en osedvanligt liten del av den totala landmassan, och även de högsta delarna, som är betydligt lägre än motsvarande i asien, utgörs av isolerade berg och bergskedjor. generellt ligger de högre platåerna i öst och söder och de lägre delarna åt väst och norr och med en linje från ungefär vid röda havets mitt till kongoflodens mynning kan kontinenten delas upp i ett lågland i norr och ett högland i söder. man kan således dela upp afrika i fyra övergripande landområden:
>     # slättlanden vid kusterna – ofta med mangroveträsk insprängda – når aldrig långt in från kusterna utom vid vattendragens nedre delar. slamrika slättland finns egentligen bara vid de större flodernas deltan, och annars utgör kustområdena det lägsta steget i en serie terrasser som bildar stigningen upp till de inre platåerna.
>     # atlasbergen i norr som ligger isolerat av saharaöknen som bitvis ligger under havsnivån.
>     # de södra och östra höglanden som ligger på mellan 600 och 1 000 m ö.h.
>     # de nordliga och västra slättlanden, inklusive sahara, som kantas och genomfars av stråk med högra landområden, men i allmänhet håller sig under 600 m ö.h.
>
>     skog täcker cirka en femtedel av afrikas yta.
>
>     hela afrika har ett tropiskt eller subtropiskt klimat.
>
>     === fauna ===
>
>
>
>     afrika är huvudsakligen känt för den öppna savannen med väldiga hjordar av hovdjur som afrikansk buffel, gnuer, antiloper, giraffer och zebror. dessa jagas av stora rovdjur som lejon, hyenor och geparder. den afrikanska djungeln befolkas av ormar och primater samt av vattenlevande djur som krokodiler och groddjur. afrika har även det största antalet arter som kan räknas till megafaunan, bland annat afrikanska elefanter, trubbnoshörning och flodhäst, på grund av att kontinenten bara i mindre utsträckning var påverkad av det massutdöende som skedde under pleistocen.
>
>     == historia ==
>
>     den första mänskliga migrationen, från kontinenten men även inom afrikas kuster, kan spåras lingvistiskt, kulturellt och genom datoranalyser av genetiskt material. någonstans i östafrika tror dagens forskare att de första hominider uppkom. i etiopien har man hittat ett två miljoner år gammalt skelett som uppkallades lucy, den första kända mänskliga primaten.
>
>     === tidigare civilisationer och handel ===
>     några av de mest kända afrikanska civilisationerna. civilisationer från olika tidsepoker visas parallellt.lingvistiska spår tyder på att bantufolk emigrerade från västafrika in i khoisanfolkets (de som tidigare kallades bushmän och hottentotter) område och undanträngde dem. bantufolken använde en uppsättning grödor, bland annat kassava och jams, som var mer lämpade för jordbruk i det tropiska afrika, och med sitt jordbruk kunde bantu försörja en betydligt större folkmängd än de undanträngde jägare eller samlare. bantufolkens traditionella bosättningsområde sträcker sig från saharaöknen till de tempererade regionerna i söder. historiskt har deras vapen varit spjut, sköld, pil och båge. det för khoisanspråkens karaktäristiska klickljuden återfinns i nguni- och bantuspråk som till exempel xhosa och zulu. khoisanspråken talas idag främst av utspridda minoritetsgrupper i södra afrika i länderna kring kalahariöknen.
>
>     afrika söder om sahara har bestått olika stater och riken, mindre stadsstater samt stammar och klaner. det forna egyptiska riket, är en av världens äldsta stater, och den mest kända av de äldre afrikanska civilisationerna. det fanns även statsbildningar i nubien, ghana, timbuktu, kanem-bornu och i nuvarande zimbabwe.
>
>     nordafrika kom från och med 600-talet under arabiskt inflytande – en tid av kulturell blomstring – och med kamelens hjälp spreds islam och den arabiska kulturen genom sahara och vidare söderut. arabiska och persiska handelsmän spred likaså islam längs med den östafrikanska kusten parallellt med en omfattande svart slavhandel. detta östafrikanska handelsutbyte med araber och perser sträckte sig ända bort till java, därmed den etniska, kulturella och språkliga mångfalden på östafrikas kust, madagaskar och komorerna.
>
>     === arabvärlden och europa utforskar kontinenten ===
>     det arabiska intresset för afrika går långt tillbaka, men handel med slavar från centra som lamu och zanzibar vid östkusten har pågått åtminstone sedan senare delen av 1000-talet e.kr. de miljoner slavar som exporterades till arabvärlden och österut under den arabiska slavhandeln finns idag kvar som en stor etnisk grupp där, i stor utsträckning integrerad med den ursprungliga arabiska befolkningen. slavhandel med svarta pågår fortfarande i vissa arabländer, till exempel sudan.&lt;ref&gt;human rights watch bbc world&lt;/ref&gt;
>
>     under 1300-talet började vissa europeiska handelsexpeditioner få kontakt med nord- och västafrika.
>
>     från 1500-talets mitt fram till förbudet mot slavhandel i storbritanniens parlament 1807, som kom att få full kraft först något årtionde senare, handlades slavar från lokala stamledare i främst väst- och centralafrika. flera miljoner afrikaner såldes på detta vis som slavar till främst de europeiska kolonierna i amerika.
>
>     tidiga kolonisatörer var araber, portugiser och spanjorer. därefter koloniserades kontinenten av framförallt av storbritannien, frankrike, spanien, italien, belgien, nederländerna, tyskland och portugal.
>
>     === koloniseringen ===
>
>     politisk karta över afrika från 1890. från slutet av 1000-talet koloniserade arabiska köpmän den afrikanska östkusten. holländarna koloniserade på 1600-talet den då relativt folktomma södra udden av afrika. portugiserna grundade flera kolonier i västra afrika och vid nuvarande moçambique. i mitten av 1800-talet leddes flera expeditioner in i de centrala delarna av afrika, bland annat av livingstone och speke, men också den svenske upptäcktsresanden andersson. denna kolonisering förändrade i grunden det ekonomiska livet på kontinenten, och innebar i flera fall ensidig export av naturresurser. kung leopolds rovdrift på sin privata koloni kongostaten är ökänd och kostade miljontals afrikanska liv.
>
>     i afrika fanns efter första världskriget endast tre självständiga stater nämligen konungariket egypten, republiken liberia och konungariket abessinien. tanger bildade med sitt område en internationell zon. för övrigt var hela världsdelen uppdelad på kolonier åt europeiska makter.&lt;ref&gt;&lt;/ref&gt;
>
>     flertalet kolonier erhöll självständighet under slutet av 1950-talet och början av 1960-talet. avkoloniseringen resulterade i flertalet fall med katastrof, där diktatur och etnisk rensning vanligen blev följden, till exempel i rhodesia (nuvarande zimbabwe), rwanda, uganda, nigeria m.fl. stater.
>
>     == politik ==
>
>     de blå länderna är demokratier enligt tankesmedjan freedom house.
>     det postkoloniala afrikas politik har i hög grad präglats av diktatur, statskupper och brist på mänskliga rättigheter.
>
>     nationerna i afrika har ofta visat sig vara instabila stater som efter självständigheten hemsökts av korruption, våldsamma statskupper, despotism och militärdiktaturer. under perioden 1960-1989 genomfördes över 70 kupper och 13 mord på presidenter. under det kalla kriget kämpade usa och sovjetunionen om makten i afrika, och tillsammans med länder som frankrike och kuba, stödde de aktivt diktaturer och iscensatte flera av dessa kupper och illgärningar mot demokratiskt valda regeringar. enligt en spridd uppfattning beror många militära konflikter om territorier och gränser under afrikas moderna historia på de gränser från kolonial tid som huvudsakligen drogs upp efter kongokonferensen, men detta kan inte visas eftersom något jämförelseområde inte finns. konflikter i afrika har ofta drivits av tillgången till kontinentens rika råvarutillgångar – diamanter och metaller som guld, platina, uran och koppar.
>
>     korruption och havererade politiska program har lett till utbredd undernäring och undermålig infrastruktur orsakar alltjämt brist på mat och rent vatten. sanitetsproblemen har också orsakat en akut spridning av bland annat hiv-viruset som leder till sjukdomen aids.
>
>     trots nöd och umbäranden finns vissa positiva tecken – demokratiskt valda regeringar växer i antal även om de inte utgör någon majoritet ännu. genom påtryckningar från till exempel internationella valutafonden (imf) har många afrikanska stater lyckats vända en nedåtgående ekonomisk spiral som varat i flera decennier till en positiv tillväxt. om man beaktar imf:s katastrofala misslyckanden i länder som argentina och filippinerna är det dock oklart i vilken omfattning dessa konventionella ekonomiska kriterier ger en rättvis bild av vad afrika faktiskt behöver.
>
>     de afrikanska ledarna har i olika omgångar sökt former för samarbete sinsemellan. hittills har dock detta inte lyckats särskilt väl. under inbördeskriget i kongo var det grannstaterna som spädde på konflikten snarare än utomafrikanska stater, och motivet var främst att skaffa kortsiktiga vinster genom att utvinna diamanter och andra mineraler. antalet döda i konflikten har uppskattats till 3,5 miljoner under en femårsperiod.
>
>     de politiska samarbetsorganisationer man skapat har inte kunnat leva upp till förväntningarna. organisationen för afrikansk enighet (oau 1963-2002) kallades till exempel diktatorsklubben för dess tendens att alltid stödja de sittande politiska ledarna och ägna väldigt lite energi åt att främja den vanlige afrikanens mänskliga rättigheter. 2002 skapades i stället afrikanska unionen, som är tänkt att underlätta ett framtida samarbete mellan kontinentens stater. hittills har denna organisation inte klarat att hantera till exempel utrotningskriget i darfur-provinsen i sudan, där en arabisk statsledning tillåtit slakt på hundratusentals av bantubefolkningen i sydväst. en avgörande skillnad mellan såväl oau som au å ena sidan och eu å den andra, är att eu startades av en grupp länder som satte demokrati och mänskliga rättigheter i fokus för den egna politiken, och som krävt detta av tillkommande medlemsstater. oau och au har från början i stället varit territoriellt inriktade, och visserligen haft demokrati och mänskliga rättigheter som skrivelser i sina stadgar, men ytterst sällan satt kraft bakom dessa ord. på så sätt liknar au mer fn än eu.
>
>     == ekonomi ==
>
>     jordbrukare på en jamsodling i kamerun
>     afrikas ekonomi omfattar ungefär 887 miljoner människor i 54 olika stater (juli 2005). afrika är världens fattigaste kontinent, och den är, i medeltal, fattigare än den var för 25 år sedan. på fn:s human development report från 2003 (som rankade världens 175 länder enligt en rad kriterier) var plats mellan 151 och 175 intagna av länder i afrika.
>
>     det finns dock länder som lyckas bra ekonomiskt, bland andra botswana, ghana och sydafrika. etiopien har också en relativ bra ekonomi tack vare den höga tillväxten i landet på cirka 11-15% per år. sydafrika har en väl fungerande inhemsk aktiehandel och har stora naturtillgångar i bland annat guld och diamanter och ett väl fungerande rättsväsen. det finns i sydafrika även tillgång till investeringskapital, stora marknader och utbildad arbetskraft. det finns även andra afrikanska länder, exempelvis egypten, som länge haft en väl fungerande handelssektor och ekonomi.
>
>     kontinentens kust mot atlanten återfinns stora fyndigheter av olja. i nigeria finns en av världens mest betydande oljereserver och landets ekonomi är bland de snabbast växande i världen.
>
>     afrikas jordbruk sysselsätter drygt hälften av afrikas befolkning och är den viktigaste näringen. cirka 6 % (ungefär 1 814 600 km²) av världsdelens yta är uppodlad, mest i savannområdena. i stora delar av de brukningsbara områdena är jordens bördighet ganska låg. de bästa jordarna återfinns i begränsade höglandsområden som i till exempel kamerun och östafrika.
>
>     == demografi ==
>
>     ===befolkning===
>     människan, homo sapiens sapiens, antas härstamma från afrika och spred sig härifrån till de omgivande kontinenterna. den svarta befolkningen söder om sahara tillhör den ursprungliga befolkningen som hela tiden levat i afrika.
>
>     norr om sahara präglar samröret med övriga medelhavsområdet och inte minst sydvästra asien både befolkning, språk och kultur.
>
>     på ön madagaskar, som länge var obebodd, härstammar majoriteten av befolkningen från människor som emigrerat från indonesien. under kolonialtiden skedde inflyttning från europa och asien till södra afrika, där den icke-svarta befolkningen idag utgör betydande minoritetsgrupper.
>
>     afrikas befolkning uppvisar många för utvecklingsregioner typiska drag, som höga födelsetal i kombination med sjunkande dödstal med snabb befolkningsökning och låg medelålder (45 % är under 15 år) som följd. även om befolkningstätheten varierar kraftigt mellan olika regioner är afrika generellt en glestbefolkad kontinent, med i genomsnitt 27,8 invånare per km².
>
>     afrika är den kontinent som har den lägsta urbaniseringsgraden, med endast 25% boende i städer. även här är variationerna stora, och i norra och östra afrika finns en lång urban tradition.
>
>     === språk ===
>
>     språkfamiljer i afrika
>     uppskattningsvis finns det ungefär 1800 talade språk i afrika idag. vissa av dessa, till exempel swahili, hausa, och yoruba, talas av många miljoner människor, medan det finns andra som bara talas av några hundra eller ännu färre. afrika har även många olika teckenspråk. de flesta afrikaner talar som modersmål ett språk tillhörande någon av de fyra språkfamiljerna afroasiatiska språk, nilo-sahariska språk, niger-kongospråk eller khoisanspråk.
>     med några få nämnvärda undantag i östra afrika, har nästan alla afrikanska länder språk som har sitt ursprung utanför världsdelen som officiellt språk, på grund av kolonialismen och mänsklig migration. till exempel används i ett stort antal afrikanska länder franska eller engelska som administrativt språk för stat, handel, utbildning och media. arabiska, portugisiska, afrikaans och malagassiska är andra exempel på språk med ursprung utanför afrika som idag används av miljoner afrikaner, både i offentliga och privata sammanhang.
>
>     === religion ===
>
>     kristendom och islam är de största religionerna i afrika, 46,3 % av alla afrikaner är kristna och 40,5 % är muslimer. resten 11,8 % tillhör olika lokala afrikanska religioner som funnits före den europeiska kolonisationen. ett mindre antal afrikaner är hinduer, eller har tro från judisk tradition. exempel på afrikanska folk som tillhör judendomen är beta israel, lembafolk och abayudaya i östra uganda. sydafrika och angola har över 55% kristna katoliker.
>
>     == kultur ==
>
>     afrika har ett stort antal överlappande kulturer, och flera tusen olika etniska grupper. den mest konventionella distinktionen är den mellan området söder om sahara och området norr om öknen. de nordliga länderna associerar sig främst med arabkulturen, medan de sydliga länderna består av i sin tur många olika kulturella områden. en av de största kulturella identiteterna är den som innefattar alla folk som talar något bantuspråk.
>
>     det går också att göra en indelning i gamla franska västafrika å ena sidan, och resten av afrika (mer specifikt de brittiska eller portugisiska kolonierna i södra och östra afrika) å andra sidan. en annan kulturell linje går mellan de afrikaner som håller kvar vid sina gamla traditionella livsstilar, och de som har anpassat sig till en mer modern livsstil. det går även att dela upp traditionalisterna i sådana som ägnar sig åt uppfödning av boskap, och sådana som ägnar sig åt jordbruk.
>
>     afrikansk konst och afrikansk arkitektur reflekterar de afrikanska kulturernas diversitet. de äldsta exemplen på afrikansk konst är 75 000 år gamla pärlsmycken gjorda av nassariusskal, dessa hittades i blombosgrottan. pyramiderna i giza var världens högsta byggnader under 4 000 år. klippkyrkorna i lalibela i etiopien, bland vilka bete giyorgis (st. göranskyrkan) är mest välkänd, är också kända exempel på afrikansk arkitektur.
>
>     == regioner och territorier ==
>     regioner i afrika:
>
>
>
>
>
>
>
>     ,,null)), (55447,Page(Amerika,
>     karta som visar nord- och sydamerika, tillsammans benämnda amerika
>     amerika (engelska: the americas) består av landmassorna mellan atlanten och stilla havet på västra halvklotet, namngivna efter den italienske upptäcktsresanden amerigo vespucci. amerika uppfattas vanligen idag som två världsdelar, nordamerika och sydamerika.&lt;ref&gt;store norske leksikon/ amerika&lt;/ref&gt; amerika indelas geografiskt i nordamerika (med centralamerika) och sydamerika. grönland ingår geografiskt i nordamerika, men räknas politiskt och kulturellt till europa. kulturgeografiskt avgränsas också ofta latinamerika, då inklusive både central- och sydamerika samt mexiko, och angloamerika, som är usa och kanada. områdets totala landyta är ungefär 42 miljoner kvadratkilometer och är således något mindre än asiens.
>
>     ofta används amerika och amerikansk för enbart landet usa, amerikas förenta stater, vilket medför en förväxlingsrisk med dubbelkontinenten som helhet men är jämförbart med hur andra landsnamn förkortas i dagligt tal (exempelvis mexiko, vars fullständiga namn översatt till svenska är mexikos förenta stater). på engelska, portugisiska och spanska kommer man ofta runt problemet genom att de två världsdelarna kallas the americas respektive las américas (två amerikorna, ett uttryck som dock aldrig har etablerats på svenska).
>
>     == namnet ==
>     amerika namngavs efter amerigo vespucci, som var den första som insåg att amerika var en kontinent som nyligen kommit till kännedom för européer, som inte var östindien. namnet används 1507 för första gången i en världskarta som skapades av den tyske geografen martin waldseemüller och beskriver där sydamerika. 1511 omnämnes det i ett engelskt skådespel och 1516 används det av leonardo da vinci på hans världskarta och vann alltså snabb utbredning. spanjorerna brukade i början ha andra namn. de kallade landet för den nya världen eller las indias occidentales (västra indien). för sydamerika behöll de under hela 1500-talet namnen peruana och brasilia. först på 1600-talet blev namnet amerika allmänt antaget.
>
>     == historia ==
>     * se amerikas historia
>
>     ,,null)), (58373,Page(Abba (olika betydelser),
>     abba kan avse:
>
>     * abba – en tidigare svensk popgrupp
>     ** abba (musikalbum) – ett album av gruppen
>     * abba seafood – ett fiskkonservföretag
>     * abba (teologi) – ett arameiskt ord
>     * abba (hederstitel) – en hederstitel och förtroligt tilltal
>
>     ==geografi==
>
>     &lt;!--notera: om platser läggs till, tas bort eller ordningen på platserna ändras, bör också mallen kartposition under samma rubrik korrigeras för att kartan ska förbli rättvisande.--&gt;
>
>     === australien ===
>
>     * abba river
>
>     === centralafrikanska republiken ===
>
>     # abba (subprefektur), préfecture de la nana-mambéré, &lt;!--geonames id 9143482--&gt;
>
>     === etiopien ===
>
>     # abba (periodiskt vattendrag), amhara, &lt;!--geonames id 345835--&gt;
>     # ābaya hāyk', sjö, &lt;!--geonames id 345844--&gt;
>     # abba (skog), oromia, &lt;!--geonames id 6642287--&gt;
>
>
>     kategori:robotskapade förgreningar 2016-01
>     kategori:robotskapade centralafrikanska republikenförgreningar
>     kategori:robotskapade etiopienförgreningar,,null)), (64397,Page(Abbekås,&lt;!-- slut på mall:ortsfakta sverige --&gt;
>
>     abbekås (uttalas abbekå-s)&lt;ref&gt;runeberg.org: svenska ortnamn med uttalsuppgifter sida 4&lt;/ref&gt; är en tätort i skivarps distrikt (skivarps socken) på söderslätt i skurups kommun.
>
>     == historia ==
>     abbekås var från början ett fiskeläge på skånes sydkust mellan ystad och trelleborg, omtalad redan på medeltiden. onsdagen den 13 november 1872 förstördes stora delar av byn av en stormflod kallad backafloden, framkallad av en orkan som drog in över sydkusten från nordost. 34 hus förstördes, alla fiskeredskap, båtar och befolkningens vinterförråd. vattenståndet ökade till 3,6 meter över det normala, vilket fortfarande är svenskt rekord. redan året därpå inleddes med hjälp av statsbidrag arbetet med en riktig hamn, och fisket och kusthandeln kunde åter komma igång. en ny storhetstid för fisket inträffade under andra världskriget.
>
>     2011 renoverades pirar och kajer, och inloppet gjordes om för att öka säkerheten i yttre hamnbassängen.
>
>     === befolkningsutveckling ===
>
>
>     ==namnet==
>     namnet skrevs 1536 abbekassz. efterleden är kås, 'båtplats, mindre hamn'. förleden innehåller förmodligen mansnamnet abbi.&lt;ref&gt;&lt;/ref&gt;.
>
>     ==samhället==
>     abbekås hamn består av en yttre och en inre hamnbassäng, där den inre används av fiskarna och den yttre är till för fritidsseglare och gästbåtar. här finns såväl hamnkrog och hotell som ett modernt tvätt/dusch/toaletthus för besökare. längre upp i byn finns ytterligare en restaurang och en livsmedelsaffär, ett konstgalleri och två bed&breakfast-ställen. strax öster om byn ligger abbekås golfklubb.
>
>     ==abbekås i musiken==
>     abbekås är omsjunget av edvard persson i sången abbekåsagåsen joakim.
>
>     == kända profiler ==
>     *lorens brolin, spelman
>
>     == bilder ==
>     &lt;gallery&gt;
>     abbekås 2.jpg|abbekås hamn
>     hamnen abbekås.jpg|abbekås hamn
>     magasinet abbekås.jpg|magasinet vid abbekås hamn
>     coast line abbekås.jpg|strandlinjen väster om abbekås hamn
>     abbekås 3.jpg|abbekåsagåsen joakim
>     &lt;/gallery&gt;
>
>     ==omgivande orter==
>
>
>     ,,null)))

  

Count the number of articles

Swedish Wikipedia has approximately 3 million articles, so not sure why
the count says 6 million

In [ ]:
// Looks like valida articles can have : in name. They are only few, so no strucural problem hiding there.
val articles = corpus.filter({ case (id, page) => page.space != null})
articles.take(10)

  

>     articles: org.apache.spark.rdd.RDD[(Long, Page)] = MapPartitionsRDD[567] at filter at command-2158711750341780:1
>     res4: Array[(Long, Page)] =
>     Array((50548827,Page(Sverigedemokraterna: den nationella rörelsen,
>     sverigedemokraterna: den nationella rörelsen är en bok av stieg larsson och mikael ekman, som utkom senhösten 2001. den avhandlar sverigedemokraterna och deras historia, samt med ett kortare avsnitt om det då nybildade nationaldemokraterna.
>
>     ,,Sverigedemokraterna:)), (56168560,Page(Lego Creator: Harry Potter och hemligheternas kammare,lego creator: harry potter och hemligheternas kammare är ett datorspel som är baserat på legot som är baserat på den andra filmen om harry potter som är baserad på den andra boken om samme unga trollkarl.
>
>     spelet är tillverkat av qube software och är ämnat att köras på windows.
>
>     målgruppen är personer i åldersgruppen 8-10 år. i spelet styr spelaren en figur eller ett djur runt i världen för att samla chokladgrodor och anta utmaningar. spelaren kan även bygga med legobitar och placera sina skapelser i världen.
>
>
>
>     kategori:windows-spel
>     kategori:harry potter-spel
>     kategori:lego-spel,,Lego Creator:)), (58729122,Page(Gustav III:s kröning i Storkyrkan,gustav iii:s kröning i storkyrkan 29 maj 1772 av porträttmålaren carl gustaf pilo på nationalmuseum i stockholm. nm 1004. målningen var tänkt som en pendang till ehrenstrahls oljemålning av karl xi:s kröning i uppsala domkyrka 1675. till vänster syns kungen gustav iii i juvelprydd vitskimrande sidendräkt, som kröns av ärkebiskop magnus beronius och riksdrotset greve adam horn, som håller kronan över gustavs huvud. bredvid kungen ses även hans bröder, hertig karl och hertig fredrik adolf, i sina furstliga dräkter med sina blåa furstliga mantlar samt de nya hertigkronorna på sina huvuden. till höger ser man drottningen sofia magdalena under kröningsstolens gyllene skulpturer.&lt;br&gt;carl gustaf pilo arbetade med målningen åren 1782–1793.
>
>     gustaf iii:s kröning i storkyrkan är en ofullbordad oljemålning av carl gustaf pilo (1711–1793), föreställande gustav iii:s kröning i storkyrkan i stockholm 29 maj 1772. den jättelika målningen, den är närmare tre meter hög och över fem meter bred, hänger på nationalmuseum i stockholm.
>
>     kung gustav iii beställde målningen 1777&lt;ref name=nationalmuseum i stockholm 1995&gt;nationalmuseum i stockholm, natur och kultur, 1995, sidan 20. .&lt;/ref&gt; och pilo arbetade på den i nio år från 1782 fram till sin död 1793.&lt;ref&gt;konstnärslexikonet amanda.&lt;/ref&gt; konstnären arbetade med detta enorma verk, som mer noggrant mäter 2,93 x 5,31 m. carl gustaf pilo var cirka 70 år när målningen påbörjades 1782. tavlan gör ett levande intryck, kanske bidrar det skissartade och impressionistiska utförandet till det.
>
>     ==några av de personer man ser på den ofullbordade målningen==
>     i förgrunden sitter gustav iii på drottning kristinas silvertron omgiven av riksråden. det äldsta riksrådet är adam horn af ekebyholm (1717–1778) står till vänster om kungen, och ärkebiskopen magnus olai beronius (1692–1775) till höger om kungen sätter varsamt kronan på hans huvud. adam horn ingick 1766 i den ambassad till köpenhamn som förmälde sofia magdalena av danmark per procuram med kronprins gustav. magnus beronius var biskop i kalmar stift och utnämndes till ärkebiskop 1764.
>
>     kungens bröder, hertig karl står till höger om kungen och hertig fredrik adolf står till vänster om kungen. de står vid tronens trappsteg. båda har sina nya furstliga kronor på sina huvuden och bådas kronor, som hade blåa hättor, utfördes inför gustav iii:s kröning av johan adam marcklin, stockholm. kungens bröder bär även sina furstliga dräkter med sina blåa furstliga mantlar.
>     carl gustav pilos förstudie till kröningstavlan. visar tydligt scenen där gustav iii kröns. runt tronen kan man se riksråden i sina mantlar, prinsarna adolf fredrik och hertig karl i sina kronor. härolderna nedanför trappstegen med sina häroldsstavar samt drottning sofia magdalena till höger. i förgrunden nere till höger kan man också se den mörkhyade hovtjänaren gustav badin.
>
>     drottning sofia magdalena sitter litet längre bort till höger i en så kallad kungsstol (1684), som är formgiven och tillverkad av ornamentsbildhuggaren och möbelsnickaren burchardt precht (1651–1758). även predikstolarna i stockholms storkyrka (1701) och i uppsala domkyrka (1709) är utförda av burchard precht. till grund för predikstolen i storkyrkan ligger ritningar av nicodemus tessin den yngre. burchard precht var född i bremen och blev, ofta i samarbete med tessin den yngre, vår mest lysande barockskulptör, enligt professor henrik cornell om stormaktstidens kyrkobyggnadskonst.
>
>     till höger om drottning sofia magdalena står överstemarskalken, riksrådet greve fredrik carl sinclair (1723–1776), som var den högste hovfunktionären. sinclair var kavaljer åt prins gustav från 1747 och han hade anställts av gustavs mor, drottning lovisa ulrika. han hade blivit riksråd 1760 och greve 1771.
>
>     &lt;gallery&gt;
>     file:gustav iii by alexander roslin - torso (nationalmuseum, 15330).png|kung gustav iii i kröningsdräkt, oljemålning av alexander roslin, 1777. målningen är beskuren. nationalmuseum.
>     file:sofia magdalena, queen of sweden (carl gustav pilo) - nationalmuseum - 21938.tif|sofia magdalena, drottning av sverige, gift med gustav iii, oljemålning av carl gustaf pilo, omkring 1765. nationalmuseum.
>     fil:sophie albertine sweden.jpg|sofia albertina, syster till gustav iii, oljemålning av lorens pasch den yngre.
>     file:karl xiii, 1748-1818, konung av sverige och norge - nationalmuseum - 15317.tif|hertig karl i den furstliga dräkt han bar vid kröningen.
>     file:fredrik adolf.jpg|hertig fredrik adolf i den furstliga dräkt han bar vid kröningen.
>     &lt;/gallery&gt;
>
>     runt dessa centralgestalter grupperar sig hela det dåtida officiella sverige. bakom kungen till vänster om pelaren står lantmarskalken axel gabriel leijonhufvud (1717–1789) stödd på sin stav omgiven av adeln. leijonhufvud var kavaljer hos kronprins gustaf 1756–1762, han befordrades 1757 till major och till överstelöjtnant 1763 före sitt avsked 1766. 1757 blev han riddare av svärdsorden. därefter var han överkammarherre hos kungens syster prinsessan sofia albertina under åren 1767–1770.
>
>     de tre personer som står mellan pelarna i mitten bakom kungen är de ofrälse ståndens talmän. till vänster syns borgmästaren carl fredrik sebaldt (1713–1792), han blev senare justitieborgmästare i stockholm 1777. i mitten syns biskop anders forssenius (1708–1788). han var biskop i skara stift åren 1767–1788 och han var med på kröningen i egenskap av prästeståndets talman. till höger mellan pelarna syns bonden josef hansson (1707–1784). josef hansson var medlem av bondeståndet vid nästan alla riksdagar fram till frihetstidens slut.  på böndernas läktare längst till höger har carl gustaf pilo målat sig själv när han bjuder konstnären elias martin (1739–1818) på en pris snus. i bakgrunden är hovdamer och riksrådsfruar placerade på en hög balkong.
>
>     ==carl gustaf pilos bakgrund som konstnär==
>     carl gustaf pilo var född i nyköping. efter sina studier var han under åren 1740–1772 bosatt i danmark. åt 1741 flyttade han till danmark och blev dansk hovmålare. i danmark hade pilo utnämnts till hovmålare, professor och direktör för det kongelige danske kunstakademi i köpenhamn. akademien var en gåva till fredrik v av danmark år 1754, när han fyllde 31 år. i köpenhamn hade pilo på 1740-, 1750- och 1760-talen utfört ett flertal porträtt, bland annat av fredrik v från omkring 1755, drottningarna lovisa, och juliana maria samt de kungliga barnen, såsom exempelvis kristian vii av danmark på 1750-talet. den danska drottningen, louise av storbritannien, (drottning lovisa av danmark, gift med fredrik v av danmark i hans första gifte, var gustav iii:s svärmor och mor till hans blivande maka, sofia magdalena, som carl gustaf pilo hade målat flera porträtt av redan 1765, innan hon kom till sverige. juliana maria var en tysk prinsessa av braunschweig-wolfenbüttel och drottning av danmark i kung fredrik v:s andra gifte. likaså hade pilo målat porträtt av den dansk-norska författaren ludvig holberg samt några figurkompositioner. när pilo återvände till sverige 1772 bosatte han sig i sin barndomsstad nyköping.&lt;ref&gt; gustav iii:s kröning av carl gustaf pilo. &lt;/ref&gt;
>
>     ==beställningen==
>     kung gustav iii sökte upp porträttmålaren carl gustaf pilo med uppdraget att måla kungens kröning. pilo fick en beställning år 1777&lt;ref name=nationalmuseum i stockholm 1995/&gt; av kungen att måla en stor kröningstavla över sig och sin drottning sofia magdalenas kröning i storkyrkan 1772. pilo försökte avsäga sig beställningen för han hade ju inte varit med vid kröningen och hade aldrig tidigare målat en gruppbild. men kungen var envis och han ville ha en motsvarande målning till david klöcker ehrenstrahls målning, som hängde på drottningholms slott och föreställde karl xi:s kröning i uppsala domkyrka 1675, och gustav iii ville att pilo skulle måla den.&lt;ref&gt;gustav iii:s kröning av carl gustaf pilo. &lt;/ref&gt;
>
>     ==uppdraget att måla gustav iii:s och sofia magdalenas kröning==
>     pilo antog till slut uppdraget att måla kröningstavlan och arbetet påbörjades först 1782. de sista åren var pilo direktör för konstakademien i stockholm. pilo arbetade med tavlan till sin död 1793 och målningen förblev ofullbordad. han avled i mars 1793. att tavlan inte blev helt färdig ser man om man studerar den noga, där man på flera ställen på målningen upptäcker flera dubbla ansikten. därtill experimenterade pilo med asfaltsfärg, ett bindemedel med stenkolstjära (bitumen), vilket gör att tavlan uppvisar många större sprickbildningar.
>
>     till vänster på tavlan ser man hur kungen i juvelprydd vitskimrande sidendräkt krönes av svenska kyrkans ärkebiskop magnus beronius och riksdrotset greve adam horn af ekebyholm, som håller kronan över gustavs huvud. adam horn ingick 1766 i den ambassad till köpenhamn som förmälde sofia magdalena av danmark per procuram med dåvarande kronprins gustav. till höger ser man drottningen sofia magdalena under kröningsstolens gyllene skulpturer. adel, präster, borgare och bönder skymtar i bakgrunden, och konstnären har avbildat sig själv i förgrunden, när han räcker sin snusdosa åt en av sina konstnärsvänner.&lt;ref name=carl g 1915&gt;carl g. laurin, konsten i sverige, p.a. norstedt & söners förlag, kungl. hofboktr. iduns tryckeri-a.b., stockholm 1915, sidorna 57-60.&lt;/ref&gt; som en stor blomma glänser den sköna drottningen, sittande på den gyllene kungsstolen i storkyrkan, fram i ljusdunklet, enligt konsthistorikern carl g. laurin i konsten i sverige, tryckt 1915.&lt;ref name=carl g 1915/&gt; vidare säger carl gustaf laurin att den väldiga tavlan blev aldrig fullt färdig. ofullbordad förvaras den nu på nationalmuseum men är en av de skönaste tavlor, som tillkommit i vårt land.
>
>     ==målningen till nationalmuseum 1865==
>     det är främst två personer som har haft betydelse för omfattningen av nationalmuseums samlingar idag. den ene är carl gustaf tessin (1695–1770) som under sin tid som ambassadör i paris på 1740-talet köpte samtida fransk konst av högsta kvalitet. den andre är gustav iii. efter gustaf iii:s död år 1792 instiftades kongl. museum till kungens minne, varvid hans samlingar överfördes i allmän ägo.&lt;ref&gt;nationalmuseets historia. &lt;/ref&gt; den stora ofullbordade oljemålningen överfördes från drottningholms slott till konstsamlingarna på nationalmuseum år 1865.&lt;ref&gt;per bjurström, nationalmuseum 1792–1992, 1992, sidan 120. .&lt;/ref&gt; tavlan gustav iii:s kröning hänger sedan 1865 på nationalmuseum, den är målad i olja på duk och mäter, för att vara exakt, 293 x 531 cm. själva museibyggnaden nationalmuseum på blasieholmen invigdes 15 juni 1866 samtidigt som stockholmsutställningen 1866, men museets historia går alltså tillbaka till 28 juni 1792, då konglig museum instiftades. nationalmuseet är därmed ett av europas äldsta museer. vid tiden när oljemålningen överfördes till nationalmuseum var fritz von dardel överintendent på museet och därmed dess högste chef. året innan hade gunnar wennerberg inlett sitt uppdrag att gå igenom alla statens tillhöriga konstverk i stockholm och på de kungliga lustslotten ute i landet och därefter göra det urval som skulle exponeras i det nya museet. han gick hårt fram och av kongl. museums 1.416 tavlor utgallrade han närmare en tredjedel eller 460 nummer. i stället överförde han till museet 67 tavlor från drottningholms slott, i första hand franska målningar ur lovisa ulrikas, gift med adolf fredrik, och gustav iii:s samlingar. från gripsholm hämtades 40 målningar, från livrustkammaren 2 målningar.&lt;ref name=nationalmuseum i stockholm 1995/&gt;&lt;ref&gt;per bjurström, nationalmuseum 1792–1992, 1992, sid. 118-119. .&lt;/ref&gt;
>
>     kröningstavlan är pilos kanske yppersta arbete och ett mästerverk i svensk konst. kompositionen är väl avvägd, koloriten glänsande harmonisk samt de individuella porträtten briljant utförda.&lt;ref&gt;carl gustaf pilo i konstnärslexikonett amanda.&lt;/ref&gt; konstnären låter sina figurer smälta in i storkyrkans arkitektur, i ett hemlighetsfullt rum av färg och ljus.&lt;ref name=nationalmuseum i stockholm 1995/&gt; även pilo själv är, såsom nämnts ovan, avbildad på denna målning, trots att han inte var med på kröningen. han ses på vänster sida titta bort från kröningen samt bjuda en annan man på snus.
>
>     ==arvfurstekronan, kröningskronan och hertigkronorna==
>     sveriges riksregalier, det vill säga kungakronan, spiran och äpplet såsom de är utställda i skattkammaren på stockholms slott.
>     kung gustav iii bar under processionen inför kröningsakten karl x gustavs arvfurstekrona. karl (x) gustavs arvfurstekrona kallas även tronföljarkronan eller kronprinskronan. kronan har traditionellt burits av den svenske tronföljaren. den sattes samman av jürgen dargeman inför drottning kristinas kröning år 1650 för att bäras av arvfursten karl (x) gustav och förvaras numera i skattkammaren på stockholms slott. kronan nedlades under ceremoniens gång på högaltaret. eftersom det var den enda tillgängliga arvfurstekronan fick hovjuveleraren johan adam marcklin (1737–1804) i stockholm den 6 juni 1771 i uppdrag att till kröningen förfärdiga nya arvfurstekronor till gustavs bröder prinsarna karl och fredrik adolf. även för kungens syster sofia albertina beställdes en krona. som förebild hade man, helt säkert på kungens initiativ, valt karl gustavs stålkrona, vars form nu anpassades efter nya stilideal. de tre kronorna gjordes av guld och på det att rikets kassa ej måtte betungas, nedsmältes för ändamålet en sedan 1743 i kungliga klädkammaren förvarad toalettservis, som ursprungligen tillhört ulrika eleonora den äldre. hertigkronornas utformning gav på en punkt upphov till en livlig tvist mellan gustav iii och hans bröder. kröningskronan som gustav iii bar vid sin kröning var ursprungligen drottning maria eleonoras krona, utförd 1620 av ruprecht miller (född efter 1550 troligen i tyskland, död senast 1630 i stockholm) och delvis förändrad 1650 och 1751. rupecht miller var guldsmed, medaljkonstnär och medaljgravör i sverige från 1706. i samband med gustav iii:s kröning byttes två av de lägre spirorna ut mot svartemaljerade vasar. kronan finns i skattkammaren på stockholms slott. hertig karls krona var tillverkad 1771 för karl, hertig av södermanland inför gustav iii:s kröning året därpå.
>
>     några av kronorna i skattkammaren på stockholms slott
>     &lt;gallery&gt;
>     file:coronet of the heir apparent of sweden 2014 crop.jpg|karl (x) gustavs arvfurstekrona.
>     file:crown of queen mary eleanor of sweden (& some of the kings) 2014.jpg|gustav iii:s kröningskrona.
>     file:coronet of frederick adolph of sweden & subsequent princes 2014.jpg|hertig karls krona.
>     &lt;/gallery&gt;
>
>     ,,Gustav III:)), (59919655,Page(Lista över USA:s presidenter,den här artikeln är en lista över personer som varit usa:s president. presidenterna är numrerade med sitt ämbetsnummer och till exempel donald trump listas och presenteras som den 45:e presidenten. han var då endast den 44:e personen, samtliga män, att bli president. numreringen görs för sammanhängande ämbetsperioder och grover cleveland är den enda president som suttit två perioder som inte varit sammanhängande. han har därför två ordningstal och räknas som usa:s 22:a och 24:e president.&lt;ref&gt;&lt;/ref&gt;
>
>     == lista över presidenter ==
>     ; partier
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>     == nu levande presidenter ==
>     sedan november 2018, då george h.w. bush avled, finns det fem presidenter som fortfarande lever:
>
>
>
>     den senaste presidenten som har avlidit är george h.w. bush (1989–93) som dog den 30 november 2018, 94 år gammal.
>
>     ,,Lista över USA:)), (68326338,Page(Lista över USA:s delstater och territorier,
>
>     den här artikeln är en lista över usa:s 50 delstater och 5 territorier.
>     se anmärkningar för begreppsförklaringar.
>     __toc__
>     ==karta==
>
>
>     ===delstatshuvudstäder===
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>     == delstater ==
>
>     &lt;nowiki&gt;*&lt;/nowiki&gt; årtal när de tretton första delstaterna ratificerade usa:s konstitution
>
>     == territorier ==
>
>
>
>
>     == anmärkningar ==
>     * begreppet ”stat” eller ”delstat” är i huvudsak en politisk indelning.
>     * fyra av delstaterna är formellt ”samvälden” (engelska: commonwealth), i och med att termen commonwealth ingår i delstatens officiella benämning (commonwealth of massachusetts), detta är dock bara en fråga om benämning och dessa fyra stater skiljer sig inte från de övriga på grund av benämningen:
>     ** kentucky,
>     ** massachusetts,
>     ** pennsylvania och
>     ** virginia.
>     * delaware är
>     ** en delstat, delaware,
>     ** ett antal countyn i olika delstater, se delaware county, och
>     ** en flod mellan pennsylvania och new jersey, se delawarefloden.
>     * new york är
>     ** en delstat, new york (delstat),
>     ** en stad i delstaten new york, new york (på engelska city of new york), och
>     ** ett county i delstaten new york, new york county.
>     * washington är
>     ** en delstat i nordvästra usa, washington (delstat),
>     ** en stad i district of columbia, washington, d.c.,
>     ** ett antal andra städer i olika delstater, se washington (olika betydelser), och
>     ** ett antal countyn i olika delstater, se washington county.
>
>     ,,Lista över USA:)), (88734327,Page(Ernst C:son Herslow,ernst georg carlsson herslow, född 24 november 1884 i malmö s:t petri församling, malmö, död 9 augusti 1967 i svedala församling, malmöhus län&lt;ref&gt;sveriges dödbok 1901–2009, dvd-rom, version 5.00, sveriges släktforskarförbund (2010).&lt;/ref&gt;, var en svensk bankman.
>
>     ==biografi==
>     han var son till tidningsmannen, politikern och bankmannen carl herslow och hulda herslow, född tranchell. efter teknisk och kemisk utbildning anställdes herslow inom olika företag i de svenska sockerindustrin och blev 1926 vice vd vid svenska sockerfabriks ab:s huvudkontor i stockholm. år 1930 övergick han till bankverksamhet.&lt;ref&gt;ne, nationalencyklopedin, bokförlaget bra böcker, åttonde bandet, 1992, sidan 571. .&lt;/ref&gt; han blev verkställande direktör 1930 vid skandinaviska bankens kontor i malmö, för att sedan verka som vd vid huvudkontoret i stockholm åren 1933–1946. under sin tid som vd bidrog han verksamt till bankens konsolidering efter kreugerkrisen.
>
>     åren 1946-1956 var han styrelseordförande vid bankens centralkontor i malmö och 1944-1946 ordförande i svenska bankföreningen. han tillhörde förgrundsgestalterna inom svenskt näringsliv ända in på 1950-talet.
>
>     han gifte sig 1910 med emmy kockum (1889–1942)&lt;ref&gt;&lt;/ref&gt;, och blev far till godsägaren carl herslow, industrimannen jan herslow och företagsledaren ernst peter herslow.
>
>     ernst c:son herslow gravsattes den 14 augusti 1967 på östra kyrkogården i malmö.&lt;ref&gt;svenskagravar.se&lt;/ref&gt;
>
>     == utmärkelser ==
>
>     * 50x50px  kommendör med stora korset av nordstjärneorden, 6 juni 1957.&lt;ref&gt;kungl. hovstaterna: kungl. maj:ts ordens arkiv, matriklar (d 1), vol. 12 (1950–1959), p. 119, digital avbildning.&lt;/ref&gt;
>
>     ,,Ernst C:)), (95798936,Page(Magic: The Gathering,ett spelparti av magic: the gathering
>     magic: the gathering (ofta kallat magic eller mtg) är ett samlarkortspel skapat av richard garfield och introducerat av företaget wizards of the coast, vilket senare köptes av hasbro. magic var det första spelet i genren collictible card games (ccgs) och spelas av uppskattningsvis 35 miljoner spelare i över sjuttio länder.&lt;ref&gt;https://www.businessinsider.com/magic-the-gathering-announces-10-million-esports-program-for-2019-2018-12?r=us&ir=t&lt;/ref&gt; magic kan spelas av två eller flera spelare där varje spelare använder en kortlek bestående av tryckta kort som spelaren själv valt ut eller genom det internet-baserade magic: the gathering online på en dator där kortleken (som kan vara identisk med den fysiska) väljs i dataspelet. nya kortuppsättningar publiceras och införlivas med spelet fyra gånger per år, både tryckt på papper och över internet.
>
>     varje spelomgång representerar en strid mellan mäktiga trollkarlar som använder magiska trollformler, föremål och varelser representerat för att besegra sin motståndare. denna strid utkämpas de facto med samlarkorten som på bilderna visar upp fantasidjur, magiska föremål, eldbollar, flygande mattor och annan high fantasy. trots att originalkonceptet hämtar mycket från traditionella fantasy-rollspel såsom dungeons & dragons har magic mycket lite gemensamt med penna-och-papper-rollspelen. magic är snarare mer likt bridge eller poker, men med betydligt fler kort och mer komplexa regler än andra kortspel. spelet utsågs 2019 till världens mest komplexa spel av ett forskarlag med forskare från cambrige, georga institute of technology och university of pennsylvania. &lt;ref&gt;https://www.technologyreview.com/2019/05/07/135482/magic-the-gathering-is-officially-the-worlds-most-complex-game/&lt;/ref&gt;
>
>     spelets popularitet världen över har gett upphov till ett organiserat turneringssystem och en community av professionella magicspelare, såväl som en andrahandsmarknad av magickort. magickort kan vara värdefulla beroende på hur sällsynta de är och hur stor efterfrågan är hos spelarna.&lt;ref&gt;magic - the gathering cards: the unofficial ultimate collector's guide&lt;/ref&gt;
>
>     == historia ==
>     peter adkinson (dåvarande vd för wizards of the coast) träffade richard garfield första gången för att diskutera garfields nya spel roborally. adkinson var inte särskilt entusiastisk över spelet, då brädspel är dyra att tillverka och svåra att sälja men lätta att marknadsföra. han uppskattade dock garfields idéer och nämnde att han sökte efter ett litet, snabbt spel som kunde spelas under pauserna som ofta uppstår under spelkonvent. garfield återkom senare med en prototyp han hade arbetat med lite då och då de senaste åren under namnet mana clash. adkinson såg omedelbart spelets potential och gick med på att tillverka det.
>
>     rollspelare var tidigt entusiastiska till magic, men spelet fick mycket större slagkraft hos strategispelare. magics framgång på marknaden skapade en explosion av samlarkortspel i mitten på 1990-talet där många betraktades som sämre kopior, (och sålde därefter) medan andra betraktades som likvärdiga i kvalité och popularitet som magic.
>
>     1996 etablerade wizards of the coast en så kallad pro tour, en serie av turneringar där spelare kan tävla om priser upp mot 400 000 dollar för en enstaka tvådagars turnering. sanktionerade genom duelists' convocation international, (eller dci) har dessa turneringar ett visst prestigevärde bland spelarna. systemet liknar det som används i flera populära sporter, till exempel golf eller tennis. företaget publicerar de bättre spelarna som vinner oftare, och skapar på så sätt stjärnor. dessa spelare inspirerar och motiverar andra.
>
>     år 2002 släpptes en officiell online-version av spelet. trots att inofficiella metoder att spela spelet fanns online tidigare, blev magic: the gathering online snabbt en framgång för företaget på grund av en kombination av det inbyggda regelsystemet och det faktum att människor enkelt kunde hitta en motståndare vid vilken tid som helst på dygnet.
>
>     == om spelet ==
>     spelet som utspelar sig i bland annat fantasivärlden dominaria simulerar dueller mellan två eller flera magiker som med hjälp av sina besvärjelser försöker ta livet av varandra. magic spelas av två eller flera spelare som har varsin lek. leken innehåller bland annat de besvärjelsekort som används i kampen mot den andre magikern. rent praktiskt går dock spelet till på så vis att spelarna sitter vid ett bord med varsin kortlek som de själva satt ihop, och försöker reducera motståndarens livspoäng till noll.
>
>     ett samlarkortspel som magic är förutom ett spel även ett samlande på kort (ungefär som äldre tiders samlarbilder). varje spelare har sin egen samling som han eller hon bygger olika lekar med. kort från de tidigare upplagorna kan vara mycket värdefulla och säljs för hundratals dollar. i originalreglerna fanns ett moment av hasardspel kallat ante där vinnaren fick ett av motståndarens kort. det togs dock bort i mitten på 90-talet på grund av spellagar i vissa amerikanska delstater.
>
>     == spelets uppbyggnad ==
>     många kort ändrar de skrivna reglerna, när detta händer har alltid kortets text förtur över regeltexten.
>
>     === korttyper ===
>     det finns olika sorters kort som alla behövs i större eller mindre utsträckning i en vinnande lek. grundtyperna är:
>     ; land: grundläggande kort som producerar mana vilket krävs för att spela nästan alla andra kort, enda undantagen är andra land och ett fåtal kort som kostar noll mana. grundländerna är kopplade till varsin färg.
>
>     ; varelser (creatures): varelser som trollkarlen frammanar för att anfalla motståndaren.
>
>     ; besvärjelser (sorceries samt instants): engångsbesvärjelser som skadar motståndaren eller hjälper trollkarlen och hans varelser temporärt.
>     ; förtrollningar (enchantments) : förtrollningars effekter påverkar spelet tills de förstörs.
>     ; artefakter (artifacts): kraftfulla artefakter som trollkarlen hittat.
>     ; planvandrare (planewalkers): avatar av planvandrare. det är i princip en spelare under din kontroll. planewalkers är mycket sällsynta.
>
>     === färger ===
>     det finns fem färger i magic. röd, grön, blå, vit och svart. varje färg har sina egna typiska drag som kan sägas symbolisera elementen. spelaren  tappar sina länder för att kunna använda den magiska kraft som genomsyrar länderna. denna kraft kallas mana och den resurs spelaren behöver för att kunna spela, casta sina andra kort.
>
>     ==== vit ====
>     * är lagen och ordningens färg. vit mana är vanligast på slätterna (plains)
>     * tema: deras formler består mycket av skyddsformler och de har läkare, präster, soldater med mera. även många mytologiska varelser såsom enhörningar, gripar och änglar.
>     vits vanligaste taktik går ut på att anfalla ofta med små varelser, men kan också vinna genom att bara skydda sig.
>
>     ==== blå ====
>     * är intelligensens, illusionens och svekets färg.
>     * hänger ihop med elementen luft och vatten. blå mana kommer främst ur öar (island)
>     * tema: luft- och vattenformler, även temat magi som sådant. de har många formler som stoppar eller förändrar andra formler. varelser från blått är oftast vatten- eller luftvarelser såsom fåglar, sjöjungfrur och liknande, eller trollkarlar.
>     blås vanligaste taktik går ut på att förutse allt motståndaren gör men kan också vinna genom att förinta motståndarens kortlek. blå anses oftast svårast att spela.
>
>     ==== svart ====
>     * är ambitionens och dödens färg. svart mana mynnar ur träsk (swamp)
>     * tema: döden, korruption, galenskap, vandöda. deras formler kan föra tillbaka de döda till livet, skrämma ihjäl andra varelser, få motspelaren att bli galen med mera. svartas varelser består oftast av vampyrer och zombies.
>     svarts vanligaste taktik går ut på att offra/skada sådant man har, till exempel varelser, artefakter och sitt eget liv. svart skyr i regel inga medel för att få ut sina egna i spel.
>
>     ==== röd ====
>     * är frihetens och impulsens färg.
>     * hänger ihop med elementen eld och jord. röd mana kommer främst från berg (mountain)
>     * tema: eld, berg, svartblod (orcher och vättar), och drakar. deras formler är oftast ganska simpla till sin natur, få en blixt att slå ner i motspelaren eller åkalla en vildsint vätte.
>     röds vanligaste taktik är att med hjälp av små snabba monster och effektiva skadebesvärjelser elda upp allt som försöker stoppa dem, innan motståndaren hinner göra någonting åt det.
>
>     ==== grön ====
>     * är livets, instinktens och tillväxlighetens färg.
>     * hör ihop med elementet jord. grön mana kommer ur skogarna (forest)
>     * tema: naturen, jorden, till viss del livet, och varelser. deras formler handlar i väldigt stor mån om att åkalla eller förändra varelser såsom alver och djur. de har även bäst kontakt med världen, och som så har de störst tillgång till mana. de har varelser som vid tappning ger mana, så som  llanowar elves.
>     gröns vanligaste taktik är att väcka liv i stora monster (som till exempel skyshroud behemoth) och krossa allt som kommer i vägen.
>     grön anses inte vara så svårt att spela.
>
>     ==== övriga ====
>     * det finns artifact creatures som till exempel titanium golem. de kan man betala med vilken mana som helst.
>     * det finns flerfärgs (multi-color) kort, där behövs det två eller flera olika sorters mana. exempel:
>     * det finns skämtkort i två specialutgåvor. dessa kort är dock inte tillåtna i tävlingssammanhang. exempel: http://ww2.wizards.com/gatherer/carddetails.aspx?&id=9762 cardboard carapace, clay pigeon.
>
>     == spelregler ==
>     spelets mål är att vinna, vilket vanligen innebär att få motståndarens liv från 20 till 0. varje spelare har en kortlek som vanligen består av 60 kort och högst fyra kopior av varje enskilt kort. undantaget från detta är grundländer, basic lands, som man får ha hur många som helst av. det finns ingen maxgräns för hur många kort spelaren får ha i en lek, så länge du kan blanda leken själv utan hjälpmedel.
>
>     * 1:startfasen består av tre steg:
>     :a. upptappningsfasen: här får spelaren tappa upp länder, varelser och artefakter
>     :b. underhållsteget (upkeep): där aktiveras vissa förmågor som [http://gatherer.wizards.com/pages/card/details.aspx?multiverseid=204987
>     :c. dra kort steget: här drar spelaren ett kort från sin lek.
>
>     * 2: huvudfasen: här kan man spela ut enbart ett land per runda. man kan även spela ut varelser, trollformler och snabbtrollformler (instants).
>
>     * 3: stridsfasen: består av fem steg.
>     :a. början: här kan spelarna aktivera snabbtrollformler och förmågor.
>     :b. ange anfallare: bestäm vilka varelser som ska anfalla och tappa dem. spelare kan även aktivera snabbtrollformler (instants) och förmågor.
>     :c. ange blockerare: här bestämmer motståndaren om han/hon vill blockera. om motståndaren inte har några varelser att blockera, så får motståndaren ta skadan. men motståndaren får utse en varelse att blockera med, varelsen blir inte tappad. spelare kan även aktivera snabbtrollformer (instants) och förmågor.
>     :d. stridsskadesteget: motståndaren bestämmer vilken varelse som ska blockera den/de varelse/r som attackerar. den som har högst attackpoäng förgör de med längre försvarspoäng. om attackpoängen och försvarspoängen är lika, förgörs båda och åker till gravhögen. om det finns en anfallare som motståndaren inte kan blockera, så får motståndaren anfallarens attackpoäng minus i liv. spelare kan även aktivera snabbtrollformler (instants) och förmågor.
>     :e. slut på striden: här kan spelarna aktivera snabbtrollformler och förmågor.
>
>     * 4: huvudfas: här kan man spela ut enbart ett land per runda. man kan även spela ut varelser, trollformler och snabbtrollformler.
>
>     * 5: slutfas: består av två steg.
>     :a: slut på rundan: här kan spelarna aktivera snabbtrollformel och förmåger.
>     :b: uppstädning: om man har mer än sju kort på handen så måste man slänga så att man bara har sju kort på handen. korten slängs i gravhögen. alla kort med until end of turn som spelaren har spelat slutar att gälla. spelaren säger att han/hon är klar och nu är det motståndarens tur.
>
>     === spelkort ===
>     det finns över 20 000 olika kort i mtg, och de har traditionellt varit indelade i tre raritetskategorier: common, uncommon, rare (vanlig, ovanlig och sällsynt). sedan hösten 2008 finns även en fjärde raritetsnivå kallad mythic rare.
>
>     ==== power nine ====
>     power nine (p9) är en gammal benämning på de nio mest kraftfulla korten som producerats. några av de traditionella p9-korten anses dock av vissa inte längre hålla måttet för att få vara med i denna exklusiva skara. p9 innehåller även många av de dyraste korten som skapats.
>
>     black lotus är det dyraste kortet bland p9 korten. genom den enormt snabba och stora tillgång till mana som black lotus ger, har den inte bara blivit dyr utan även restriktad (i stället för fyra exemplar av samma kort får man bara ha ett av varje i sin lek) i vintage. en black lotus kostar mellan 25 000 och 900 000 kronor ( i mycket sällsynta fall det sistnämnda beloppet, beroende på gradering 1-10 ) beroende på utgåva och skick, där de äldsta utgåvorna är dyrare.
>
>     de nio power nine-korten är: black lotus, mox pearl, mox sapphire, mox jet, mox ruby, mox emerald, ancestral recall, time walk och timetwister.
>
>     black lotus är dock inte det dyraste kort som tillverkats. kortet world champion 1996 trycktes det bara ett exemplar av och gavs till tom chanpheng som vann världsmästerskapen i magic 1996.
>
>     == organiserat spelande ==
>     turneringar i magic hålls regelbundet i spelbutiker och vid andra tillställningar. dci, som ägs av wizards of the coast, är den officiella förvaltaren av tävlingsspel i magic och tillhandahåller regler och annat material till de som ordnar turneringar. större tävlingar med flera hundra spelare hålls flera gånger varje år med stora vinster till de som hamnar bland topp-placeringarna. de två stora kategorierna vid tävlingsspel är constructed och limited.
>
>     === constructed ===
>     i constructed-formatet så tar alla tävlande med sig en lek man byggt, och möter andra spelare.
>     korten man använt i sin lek måste vara tillåtna i det format man spelar. det format med.
>     mest turneringar i kallas standard, och där är alla kort som är ifrån de senaste två expansionerna och det senaste core setet tillåtna. helt enkelt de nyaste korten.
>
>     eftersom det finns så många kort så är det oundvikligt att alla är lika bra, och i.
>     constructed-formatet är det bara de bästa korten som spelas. detta leder till att.
>     kort som är populära i constructed-formatet går upp i pris.
>
>     === limited ===
>     i limited-format bygger man sina lekar på plats av kort man tilldelas i form av så kallade booster packs. ett booster pack innehåller 15 slumpmässiga kort av blandade färger.
>     eftersom man inte kan välja och vraka bland så stort utbud av kort brukar lekarna man bygger vara relativt svaga.
>
>     det finns två olika limited-format, sealed deck och draft.
>     i sealed deck får man sex stycken booster packs (totalt 90 kort) som man helt enkelt bygger en lek på 40 kort av.
>
>     i draft så får man tre booster packs var, men man bygger inte leken på samma sätt som i sealed deck. alla spelare sätter sig kring ett bord och öppnar var sitt booster pack. alla väljer ett kort och skickar vidare de återstående 14 korten åt höger. detta pågår tills korten är slut, och då börjar man med ett nytt booster pack. korten skickas första omgången åt höger, nästa vänster, och sista höger igen. korten man plockat på sig när alla booster packs är slut är de man bygger en lek av.
>
>     === världsmästerskap ===
>     världsmästerskap i agimc: the gathering har anordnats årligen sedan 1994 och var från början en turnering som var öppen för alla att delta i och det tävlades mest om äran. från 1996 blev vm-turneringen istället en del av den professionella touren och därmed en turnering endast för den som först klarat en kvaltävling. prispotten höjdes samtidigt rejält vid professionaliseringen och förstapriset i vm 1996 var 26 000 amerikanska dollar. &lt;ref&gt;https://magic.wizards.com/en/articles/archive/feature/musings-history-magic-pro-tour-2009-06-01&lt;/ref&gt;
>     trots att man var tungen att kvalificera sig genom kvalspel hade vm 1996-2011 flera hundra deltagare. från och med 2012 skars deltagarantalet ned till endast 16 personer och blev ytterligare reserverat för spelets absolut bästa spelare. 2012-2020 har vm spelats med mellan 16 och 24 spelare utvalda bland de absolut mest framgångsrika spelarna på proffstouren.
>     spelaren shahar shenhar från israel är hittills den enda som vunnit individuella vm mer än en gång, vilket han gjorde 2013 och 2014. &lt;ref&gt;https://magic.wizards.com/en/articles/archive/feature/shahar-shenhars-origin-story-2015-08-03&lt;/ref&gt;
>
>     === nationella mästerskapen ===
>     nationella mästerskap i magic: the gathering anordnas årligen, vanligen kring sommar. de har anordnats sedan 1995 och hålls nu i 67 olika länder.
>
>     ,,Magic:)), (101860558,Page(Thomas Cromwell, 1:e earl av Essex,
>
>     thomas cromwell, från 1540 earl av essex (lord essex), född omkring 1485 i putney, död 28 juli 1540 på towern, london (avrättad), var en engelsk statsman, en av de viktigaste politiska personligheterna under henrik viii:s styre.
>
>     == biografi ==
>     cromwell föddes omkring 1485 i putney. han var son till smeden och klädhandlaren walter cromwell. i början av 1500-talet reste den unge thomas cromwell runt i frankrike, nederländerna och italien och tjänstgjorde bland annat som legosoldat i franska armén. efter att ha inlett en karriär som jurist i england och blivit parlamentsledamot 1523 trädde thomas cromwell i tjänst hos kungens dåvarande närmaste man kardinal thomas wolsey. år 1533 hade cromwell nått positionen som secretary of state.
>
>     i samband med wolseys fall, till följd av att denne misslyckades med att få katolska kyrkan att bevilja henrik viii annullering av sitt äktenskap med katarina av aragonien, fick cromwell under 1530-talet en mer framträdande roll då han var den mest framträdande av dem som föreslog för henrik viii att kungen själv skulle bryta med katolska kyrkan genom att göra sig till överhuvud för den engelska kyrkan och drev igenom suprematiakten i parlamentet 1534.
>
>     år 1535 utnämnde henrik viii cromwell till sin ställföreträdare i kyrkliga frågor. detta gav honom makt som högste domare i kyrkliga ärenden och ämbetet innebar en enda förenande institution över den engelska kyrkans två provinser. som henriks generalvikarie övervakade han klosterupplösningen, som började med hans visitation av kloster och klosterkyrkor, tillkännagiven 1535 och påbörjad vintern 1536.
>
>     klosterupplösningen har ofta beskrivits som ett cyniskt initiativ för att roffa åt staten pengar, men cromwell och hans anhängare hade genuina teologiska invändningar mot klosterlivet som idé, särskilt om medlingsböner för de döda. medan cromwell var vid makten var henrik viii:s regering mycket öppnare för religiösa reformer än den senare kom att vara.
>
>     cromwell var också arkitekten bakom laws in wales acts, som innebar att englands lag skulle gälla i wales och de två skulle räknas som en stat, som antogs för första gången 1535.
>
>     cromwell, som först varit henrik viii:s andra hustru anne boleyns bundsförvant i kampen för att få kungen att bryta med påvedömet för att kunna annullera sitt första äktenskap för att gifta sig med henne, stödde dock henrik när denne 1536 lät avrätta henne och sedan gifte sig med jane seymour. som belöning för sina tjänster utnämnde henrik viii cromwell till baron av wimbledon och lord privy seal 1536. på den sistnämnda posten efterträdde han anne boleyns far thomas boleyn som fallit i onåd i samband med dotterns fall.
>
>     strax efter kungens omgifte utbröt ett stort uppror i norra england, kallat nådens pilgrimsfärd (pilgrimage of grace) i syfte att stoppa klosterupplösningen. inledningsvis var det ett folkligt uppror men det kom även att stödjas av lågadel och en del högadel. för att lugna de rebelliska katolikerna gick henrik viii med på att låta parlamentet ta upp deras frågor. han gick även med på att ge amnesti till alla inblandade. han höll inget av löftena, vilket utlöste ett nytt uppror 1537. den här gången slog kungen till snabbt och det hela slutade med att upprorsledarna dömdes för högförräderi och avrättades.
>
>     år 1538 sanktionerade henrik förstörelsen av helgedomar över romersk-katolska helgon. 1539 upplöstes englands kvarvarande kloster och deras egendomar beslagtogs av kronan. året därpå 1540 utnämndes cromwell till greve (earl) av essex.
>
>     jane seymour hade dött i barnsäng 1537 i samband med den blivande edvard vi:s födelse. cromwell uppmanade kungen att gifta om sig då det behövdes ytterligare arvingar ifall den sjuklige lille edvard skulle dö. valet föll 1540 på den tyska prinsessan anna av kleve, då cromwell menade att detta innebar en fördelaktig allians. när kungen vantrivdes i sitt nya äktenskap föll cromwell snabbt i onåd. detta var det egentliga motivet till att cromwell arresterades för högförräderi den 10 juni 1540 och senare dömdes till döden.
>
>     cromwell avrättades enskilt i towern den 28 juli 1540. det sägs att henrik viii avsiktligt valde en oerfaren bödel. cromwell sade att han dog i den traditionella tron (katolska kyrkan) innan han dog. den unge bödeln försökte hugga av cromwells huvud tre gånger innan han lyckades. efter avrättningen kokades huvudet och sattes på en spik på london bridge, med ansiktet bort från city of london.&lt;ref&gt;&lt;/ref&gt;
>
>     henrik kom senare att ångra att han hade låtit avrätta cromwell och i ett uttalande 1541 kallade kungen honom den trognaste tjänare han någonsin haft.
>
>     lordprotektorn av england oliver cromwell (1599–1658), var ättling till thomas cromwells syster catherine cromwell.
>
>     ,,Thomas Cromwell, 1:)), (103617262,Page(Karl XII:s fältmarsch,karl xii:s fältmarsch eller marsch bussar! gån på uti herrans namn är en marsch som den svenske kungen karl xii lät sina trupper sjunga inför ett fältslag. dock först sedan psalmen vår gud är oss en väldig borg hade framförts.
>
>     texten anses vara författad av magnus stenbock och marschen kallas därför även stenbockens marsch. den finns, efter en av e. j. arrhén von kapfelman utgiven bearbetning i stämmor, införd i a. j. arvidssons »svenska fornsånger» (ii, n:o 162).
>
>     stenbocksmarschen är sedan 1939 dalregementets paradmarsch.
>
>     ,,Karl XII:)), (104018940,Page(Horace Walpole, 4:e earl av Orford,horace walpole.
>     horatio walpole, 4:e earl av orford, känd som horace walpole, född 24 september 1717 i london, död 2 mars 1797 vid berkeley square i mayfair, london, var en brittisk författare, konsthistoriker, samlare och medlem av underhuset (whig). han var yngste son till storbritanniens förste premiärminister, robert walpole. horace walpole är mest känd som förgrundsfigur inom romantiken och författare till den första gotiska skräckromanen.
>
>     ==biografi==
>     walpole studerade vid eton och king's college i cambridge.1739-1741 reste han i frankrike och italien. han hade ingen framträdande roll i parlamentet men deltog regelbundet i debatter fram till 1768.
>
>     hans korta roman borgen i otranto (1764) räknas som den första gotiska skräckromanen. den publicerades först anonymt.
>
>     han är även känd som en av det engelska språkområdets flitigaste brevskrivare. han utvecklade medvetet brevskrivandets konst i mme de sévignés anda och hans privata korrespondens på runt 4000 brev är en exposé över tidens seder och smak.
>
>     han myntade ordet serendipitet. det är ett begrepp som används rikligt inom bland annat vetenskapen och innebär att man av en slump gör en tursam upptäckt som man tar tillvara. entreprenörskap och genialitet anses ofta höra samman med serendipitet.
>
>     på grund av sitt intresse för gotisk arkitektur lät han föra sin egen nygotiska byggnad och bostad strawberry hill. huset var öppet för besökare och blev berömt redan under walpoles livstid. strawberry hill inspirerade många efterföljare och återupplivade gotiken i engelsk arkitektur.&lt;ref&gt;&lt;/ref&gt;
>
>     horace walpole förblev ogift och vid hans död föll earltiteln ur bruk.
>
>     både horace walpole och fadern robert ligger begravda i st martin's church i houghton i norfolk.&lt;ref&gt;houghton hall. literary norfolk. läst 8 december 2013.&lt;/ref&gt;
>
>     ,,Horace Walpole, 4:)))

In [ ]:
articles.count()

  

>     res5: Long = 5593

In [ ]:
corpus.count()

  

>     res6: Long = 3552747

  

Find problem with filtering
---------------------------

In [ ]:
// Some random test cases
println(filterSource("""test <title>titles</title> mer <text bytes="7485" xml:space="preserve">info
</text>"""))
println(filterSource("""test <title>space:nospace</title> mer <text bytes="7485" xml:space="preserve">info
</text>"""))
println(filterSource("test"))
println(filterSource("test [[| asf|]] [[nada|]] [[asf|show]] "))
println(filterSource("fail [[] {"))
println(filterSource("fail [[|]] [[]] {}"))
println(filterSource("fail [[dollar|$]]"))


  

>     Page(titles,info
>     ,,null)
>     Page(space:nospace,info
>     ,,space:)
>     Page(,**EXCEPTION**
>     <title[^>]*>((.*:)?.*)</title>
>     test,java.lang.IllegalStateException
>     	at scala.util.matching.Regex$MatchIterator.ensure(Regex.scala:850)
>     	at scala.util.matching.Regex$MatchIterator.start(Regex.scala:860)
>     	at scala.util.matching.Regex$MatchData.group(Regex.scala:655)
>     	at scala.util.matching.Regex$MatchData.group$(Regex.scala:654)
>     	at scala.util.matching.Regex$MatchIterator.group(Regex.scala:810)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91439.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.filterSource(command-738986526041627:35)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:5)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:77)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:79)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:81)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:83)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:85)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw.<init>(command-738986526041635:87)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw.<init>(command-738986526041635:89)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw.<init>(command-738986526041635:91)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw.<init>(command-738986526041635:93)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read.<init>(command-738986526041635:95)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$.<init>(command-738986526041635:99)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$.<clinit>(command-738986526041635)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$eval$.$print$lzycompute(<notebook>:7)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$eval$.$print(<notebook>:6)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$eval.$print(<notebook>)
>     	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
>     	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
>     	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
>     	at java.lang.reflect.Method.invoke(Method.java:498)
>     	at scala.tools.nsc.interpreter.IMain$ReadEvalPrint.call(IMain.scala:745)
>     	at scala.tools.nsc.interpreter.IMain$Request.loadAndRun(IMain.scala:1021)
>     	at scala.tools.nsc.interpreter.IMain.$anonfun$interpret$1(IMain.scala:574)
>     	at scala.reflect.internal.util.ScalaClassLoader.asContext(ScalaClassLoader.scala:41)
>     	at scala.reflect.internal.util.ScalaClassLoader.asContext$(ScalaClassLoader.scala:37)
>     	at scala.reflect.internal.util.AbstractFileClassLoader.asContext(AbstractFileClassLoader.scala:41)
>     	at scala.tools.nsc.interpreter.IMain.loadAndRunReq$1(IMain.scala:573)
>     	at scala.tools.nsc.interpreter.IMain.interpret(IMain.scala:600)
>     	at scala.tools.nsc.interpreter.IMain.interpret(IMain.scala:570)
>     	at com.databricks.backend.daemon.driver.DriverILoop.execute(DriverILoop.scala:219)
>     	at com.databricks.backend.daemon.driver.ScalaDriverLocal.$anonfun$repl$1(ScalaDriverLocal.scala:204)
>     	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
>     	at com.databricks.backend.daemon.driver.DriverLocal$TrapExitInternal$.trapExit(DriverLocal.scala:773)
>     	at com.databricks.backend.daemon.driver.DriverLocal$TrapExit$.apply(DriverLocal.scala:726)
>     	at com.databricks.backend.daemon.driver.ScalaDriverLocal.repl(ScalaDriverLocal.scala:204)
>     	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$10(DriverLocal.scala:431)
>     	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:239)
>     	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
>     	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:234)
>     	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:231)
>     	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:48)
>     	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:276)
>     	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:269)
>     	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:48)
>     	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:408)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:653)
>     	at scala.util.Try$.apply(Try.scala:213)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:645)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.getCommandOutputAndError(DriverWrapper.scala:486)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommand(DriverWrapper.scala:598)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.runInnerLoop(DriverWrapper.scala:391)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.runInner(DriverWrapper.scala:337)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.run(DriverWrapper.scala:219)
>     	at java.lang.Thread.run(Thread.java:748)
>
>     ,)
>     Page(,**EXCEPTION**
>     <title[^>]*>((.*:)?.*)</title>
>     test [[| asf|]] [[nada|]] [[asf|show]] ,java.lang.IllegalStateException
>     	at scala.util.matching.Regex$MatchIterator.ensure(Regex.scala:850)
>     	at scala.util.matching.Regex$MatchIterator.start(Regex.scala:860)
>     	at scala.util.matching.Regex$MatchData.group(Regex.scala:655)
>     	at scala.util.matching.Regex$MatchData.group$(Regex.scala:654)
>     	at scala.util.matching.Regex$MatchIterator.group(Regex.scala:810)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91439.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.filterSource(command-738986526041627:35)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:6)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:77)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:79)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:81)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:83)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:85)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw.<init>(command-738986526041635:87)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw.<init>(command-738986526041635:89)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw.<init>(command-738986526041635:91)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw.<init>(command-738986526041635:93)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read.<init>(command-738986526041635:95)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$.<init>(command-738986526041635:99)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$.<clinit>(command-738986526041635)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$eval$.$print$lzycompute(<notebook>:7)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$eval$.$print(<notebook>:6)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$eval.$print(<notebook>)
>     	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
>     	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
>     	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
>     	at java.lang.reflect.Method.invoke(Method.java:498)
>     	at scala.tools.nsc.interpreter.IMain$ReadEvalPrint.call(IMain.scala:745)
>     	at scala.tools.nsc.interpreter.IMain$Request.loadAndRun(IMain.scala:1021)
>     	at scala.tools.nsc.interpreter.IMain.$anonfun$interpret$1(IMain.scala:574)
>     	at scala.reflect.internal.util.ScalaClassLoader.asContext(ScalaClassLoader.scala:41)
>     	at scala.reflect.internal.util.ScalaClassLoader.asContext$(ScalaClassLoader.scala:37)
>     	at scala.reflect.internal.util.AbstractFileClassLoader.asContext(AbstractFileClassLoader.scala:41)
>     	at scala.tools.nsc.interpreter.IMain.loadAndRunReq$1(IMain.scala:573)
>     	at scala.tools.nsc.interpreter.IMain.interpret(IMain.scala:600)
>     	at scala.tools.nsc.interpreter.IMain.interpret(IMain.scala:570)
>     	at com.databricks.backend.daemon.driver.DriverILoop.execute(DriverILoop.scala:219)
>     	at com.databricks.backend.daemon.driver.ScalaDriverLocal.$anonfun$repl$1(ScalaDriverLocal.scala:204)
>     	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
>     	at com.databricks.backend.daemon.driver.DriverLocal$TrapExitInternal$.trapExit(DriverLocal.scala:773)
>     	at com.databricks.backend.daemon.driver.DriverLocal$TrapExit$.apply(DriverLocal.scala:726)
>     	at com.databricks.backend.daemon.driver.ScalaDriverLocal.repl(ScalaDriverLocal.scala:204)
>     	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$10(DriverLocal.scala:431)
>     	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:239)
>     	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
>     	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:234)
>     	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:231)
>     	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:48)
>     	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:276)
>     	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:269)
>     	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:48)
>     	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:408)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:653)
>     	at scala.util.Try$.apply(Try.scala:213)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:645)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.getCommandOutputAndError(DriverWrapper.scala:486)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommand(DriverWrapper.scala:598)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.runInnerLoop(DriverWrapper.scala:391)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.runInner(DriverWrapper.scala:337)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.run(DriverWrapper.scala:219)
>     	at java.lang.Thread.run(Thread.java:748)
>
>     ,)
>     Page(,**EXCEPTION**
>     <title[^>]*>((.*:)?.*)</title>
>     fail [[] {,java.lang.IllegalStateException
>     	at scala.util.matching.Regex$MatchIterator.ensure(Regex.scala:850)
>     	at scala.util.matching.Regex$MatchIterator.start(Regex.scala:860)
>     	at scala.util.matching.Regex$MatchData.group(Regex.scala:655)
>     	at scala.util.matching.Regex$MatchData.group$(Regex.scala:654)
>     	at scala.util.matching.Regex$MatchIterator.group(Regex.scala:810)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91439.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.filterSource(command-738986526041627:35)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:7)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:77)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:79)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:81)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:83)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:85)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw.<init>(command-738986526041635:87)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw.<init>(command-738986526041635:89)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw.<init>(command-738986526041635:91)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw.<init>(command-738986526041635:93)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read.<init>(command-738986526041635:95)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$.<init>(command-738986526041635:99)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$.<clinit>(command-738986526041635)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$eval$.$print$lzycompute(<notebook>:7)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$eval$.$print(<notebook>:6)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$eval.$print(<notebook>)
>     	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
>     	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
>     	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
>     	at java.lang.reflect.Method.invoke(Method.java:498)
>     	at scala.tools.nsc.interpreter.IMain$ReadEvalPrint.call(IMain.scala:745)
>     	at scala.tools.nsc.interpreter.IMain$Request.loadAndRun(IMain.scala:1021)
>     	at scala.tools.nsc.interpreter.IMain.$anonfun$interpret$1(IMain.scala:574)
>     	at scala.reflect.internal.util.ScalaClassLoader.asContext(ScalaClassLoader.scala:41)
>     	at scala.reflect.internal.util.ScalaClassLoader.asContext$(ScalaClassLoader.scala:37)
>     	at scala.reflect.internal.util.AbstractFileClassLoader.asContext(AbstractFileClassLoader.scala:41)
>     	at scala.tools.nsc.interpreter.IMain.loadAndRunReq$1(IMain.scala:573)
>     	at scala.tools.nsc.interpreter.IMain.interpret(IMain.scala:600)
>     	at scala.tools.nsc.interpreter.IMain.interpret(IMain.scala:570)
>     	at com.databricks.backend.daemon.driver.DriverILoop.execute(DriverILoop.scala:219)
>     	at com.databricks.backend.daemon.driver.ScalaDriverLocal.$anonfun$repl$1(ScalaDriverLocal.scala:204)
>     	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
>     	at com.databricks.backend.daemon.driver.DriverLocal$TrapExitInternal$.trapExit(DriverLocal.scala:773)
>     	at com.databricks.backend.daemon.driver.DriverLocal$TrapExit$.apply(DriverLocal.scala:726)
>     	at com.databricks.backend.daemon.driver.ScalaDriverLocal.repl(ScalaDriverLocal.scala:204)
>     	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$10(DriverLocal.scala:431)
>     	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:239)
>     	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
>     	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:234)
>     	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:231)
>     	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:48)
>     	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:276)
>     	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:269)
>     	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:48)
>     	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:408)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:653)
>     	at scala.util.Try$.apply(Try.scala:213)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:645)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.getCommandOutputAndError(DriverWrapper.scala:486)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommand(DriverWrapper.scala:598)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.runInnerLoop(DriverWrapper.scala:391)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.runInner(DriverWrapper.scala:337)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.run(DriverWrapper.scala:219)
>     	at java.lang.Thread.run(Thread.java:748)
>
>     ,)
>     Page(,**EXCEPTION**
>     <title[^>]*>((.*:)?.*)</title>
>     fail [[|]] [[]] {},java.lang.IllegalStateException
>     	at scala.util.matching.Regex$MatchIterator.ensure(Regex.scala:850)
>     	at scala.util.matching.Regex$MatchIterator.start(Regex.scala:860)
>     	at scala.util.matching.Regex$MatchData.group(Regex.scala:655)
>     	at scala.util.matching.Regex$MatchData.group$(Regex.scala:654)
>     	at scala.util.matching.Regex$MatchIterator.group(Regex.scala:810)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91439.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.filterSource(command-738986526041627:35)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:8)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:77)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:79)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:81)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:83)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:85)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw.<init>(command-738986526041635:87)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw.<init>(command-738986526041635:89)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw.<init>(command-738986526041635:91)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw.<init>(command-738986526041635:93)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read.<init>(command-738986526041635:95)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$.<init>(command-738986526041635:99)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$.<clinit>(command-738986526041635)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$eval$.$print$lzycompute(<notebook>:7)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$eval$.$print(<notebook>:6)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$eval.$print(<notebook>)
>     	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
>     	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
>     	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
>     	at java.lang.reflect.Method.invoke(Method.java:498)
>     	at scala.tools.nsc.interpreter.IMain$ReadEvalPrint.call(IMain.scala:745)
>     	at scala.tools.nsc.interpreter.IMain$Request.loadAndRun(IMain.scala:1021)
>     	at scala.tools.nsc.interpreter.IMain.$anonfun$interpret$1(IMain.scala:574)
>     	at scala.reflect.internal.util.ScalaClassLoader.asContext(ScalaClassLoader.scala:41)
>     	at scala.reflect.internal.util.ScalaClassLoader.asContext$(ScalaClassLoader.scala:37)
>     	at scala.reflect.internal.util.AbstractFileClassLoader.asContext(AbstractFileClassLoader.scala:41)
>     	at scala.tools.nsc.interpreter.IMain.loadAndRunReq$1(IMain.scala:573)
>     	at scala.tools.nsc.interpreter.IMain.interpret(IMain.scala:600)
>     	at scala.tools.nsc.interpreter.IMain.interpret(IMain.scala:570)
>     	at com.databricks.backend.daemon.driver.DriverILoop.execute(DriverILoop.scala:219)
>     	at com.databricks.backend.daemon.driver.ScalaDriverLocal.$anonfun$repl$1(ScalaDriverLocal.scala:204)
>     	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
>     	at com.databricks.backend.daemon.driver.DriverLocal$TrapExitInternal$.trapExit(DriverLocal.scala:773)
>     	at com.databricks.backend.daemon.driver.DriverLocal$TrapExit$.apply(DriverLocal.scala:726)
>     	at com.databricks.backend.daemon.driver.ScalaDriverLocal.repl(ScalaDriverLocal.scala:204)
>     	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$10(DriverLocal.scala:431)
>     	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:239)
>     	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
>     	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:234)
>     	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:231)
>     	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:48)
>     	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:276)
>     	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:269)
>     	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:48)
>     	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:408)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:653)
>     	at scala.util.Try$.apply(Try.scala:213)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:645)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.getCommandOutputAndError(DriverWrapper.scala:486)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommand(DriverWrapper.scala:598)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.runInnerLoop(DriverWrapper.scala:391)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.runInner(DriverWrapper.scala:337)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.run(DriverWrapper.scala:219)
>     	at java.lang.Thread.run(Thread.java:748)
>
>     ,)
>     Page(,**EXCEPTION**
>     <title[^>]*>((.*:)?.*)</title>
>     fail [[dollar|$]],java.lang.IllegalStateException
>     	at scala.util.matching.Regex$MatchIterator.ensure(Regex.scala:850)
>     	at scala.util.matching.Regex$MatchIterator.start(Regex.scala:860)
>     	at scala.util.matching.Regex$MatchData.group(Regex.scala:655)
>     	at scala.util.matching.Regex$MatchData.group$(Regex.scala:654)
>     	at scala.util.matching.Regex$MatchIterator.group(Regex.scala:810)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91439.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.filterSource(command-738986526041627:35)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:9)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:77)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:79)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:81)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:83)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw$$iw.<init>(command-738986526041635:85)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw$$iw.<init>(command-738986526041635:87)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw$$iw.<init>(command-738986526041635:89)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw$$iw.<init>(command-738986526041635:91)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$$iw.<init>(command-738986526041635:93)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read.<init>(command-738986526041635:95)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$.<init>(command-738986526041635:99)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$read$.<clinit>(command-738986526041635)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$eval$.$print$lzycompute(<notebook>:7)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$eval$.$print(<notebook>:6)
>     	at line80ed0ade3d4b4100b5f2fb3e1450b91451.$eval.$print(<notebook>)
>     	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
>     	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
>     	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
>     	at java.lang.reflect.Method.invoke(Method.java:498)
>     	at scala.tools.nsc.interpreter.IMain$ReadEvalPrint.call(IMain.scala:745)
>     	at scala.tools.nsc.interpreter.IMain$Request.loadAndRun(IMain.scala:1021)
>     	at scala.tools.nsc.interpreter.IMain.$anonfun$interpret$1(IMain.scala:574)
>     	at scala.reflect.internal.util.ScalaClassLoader.asContext(ScalaClassLoader.scala:41)
>     	at scala.reflect.internal.util.ScalaClassLoader.asContext$(ScalaClassLoader.scala:37)
>     	at scala.reflect.internal.util.AbstractFileClassLoader.asContext(AbstractFileClassLoader.scala:41)
>     	at scala.tools.nsc.interpreter.IMain.loadAndRunReq$1(IMain.scala:573)
>     	at scala.tools.nsc.interpreter.IMain.interpret(IMain.scala:600)
>     	at scala.tools.nsc.interpreter.IMain.interpret(IMain.scala:570)
>     	at com.databricks.backend.daemon.driver.DriverILoop.execute(DriverILoop.scala:219)
>     	at com.databricks.backend.daemon.driver.ScalaDriverLocal.$anonfun$repl$1(ScalaDriverLocal.scala:204)
>     	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
>     	at com.databricks.backend.daemon.driver.DriverLocal$TrapExitInternal$.trapExit(DriverLocal.scala:773)
>     	at com.databricks.backend.daemon.driver.DriverLocal$TrapExit$.apply(DriverLocal.scala:726)
>     	at com.databricks.backend.daemon.driver.ScalaDriverLocal.repl(ScalaDriverLocal.scala:204)
>     	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$10(DriverLocal.scala:431)
>     	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:239)
>     	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
>     	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:234)
>     	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:231)
>     	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:48)
>     	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:276)
>     	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:269)
>     	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:48)
>     	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:408)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.$anonfun$tryExecutingCommand$1(DriverWrapper.scala:653)
>     	at scala.util.Try$.apply(Try.scala:213)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:645)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.getCommandOutputAndError(DriverWrapper.scala:486)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommand(DriverWrapper.scala:598)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.runInnerLoop(DriverWrapper.scala:391)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.runInner(DriverWrapper.scala:337)
>     	at com.databricks.backend.daemon.driver.DriverWrapper.run(DriverWrapper.scala:219)
>     	at java.lang.Thread.run(Thread.java:748)
>
>     ,)

In [ ]:
val failed = corpus.filter({ case (id, source) => source.status != ""})
failed.take(10)

  

>     failed: org.apache.spark.rdd.RDD[(Long, Page)] = MapPartitionsRDD[577] at filter at command-738986526041636:1
>     res8: Array[(Long, Page)] = Array()

In [ ]:
failed.count()

  

>     res9: Long = 0

In [ ]:
// Are the empty articles mistakes or reasonable? Looks reasonable.
val failed_empty = corpus.filter({ case (id, page) => page.text.length() < 20})
failed_empty.take(10)

  

>     failed_empty: org.apache.spark.rdd.RDD[(Long, Page)] = MapPartitionsRDD[620] at filter at command-738986526041638:1
>     res10: Array[(Long, Page)] =
>     Array((239722707,Page(Lista över nationaldagar,
>
>
>
>     ,,null)), (371889681,Page(Lista över USA:s delstater och territorier efter yta,
>
>     ,,Lista över USA:)), (583039664,Page(Lista över fjäll på Färöarna,
>
>     färöarnas berg,,null)), (679739723,Page(Lista över öar efter folkmängd,
>
>
>     ,,null)), (739536420,Page(Lista över Wimbledonmästare i damsingel,
>     ==lista==
>
>
>     ,,null)), (752944879,Page(Lista över nationalsånger,
>
>
>     ,,null)), (783397284,Page(Lista över köpingar i Sverige,
>
>
>     ,,null)), (969318751,Page(Lista över Tysklands delstater efter yta,
>
>     delstater,,null)), (1188511201,Page(Lista över Greklands utrikesministrar,
>
>     ,,null)), (1198242590,Page(Lista över Schleswig-Holsteins ministerpresidenter,
>
>       ,,null)))

In [ ]:
val example = dump.filter({ case (id, source) => id == 2577428})
val fail_source = example.take(1)(0)._2

In [ ]:
val fail_state = filterSource(fail_source)

  

>     fail_state: Page = Page(Adeln,#omdirigering adel,,null)

In [ ]:
import java.util.regex.Matcher;

val regexp = raw"\[\[[^\]\[]*\|([^\[\]]*)\]\]".r 
var current = fail_source
var count = 0
current = regexp.replaceAllIn(current, m => {println(m.group(0) + " ==> " + m.group(1)); Matcher.quoteReplacement(m.group(1))}) // )


  

>     import java.util.regex.Matcher
>     regexp: scala.util.matching.Regex = \[\[[^\]\[]*\|([^\[\]]*)\]\]
>     current: String =
>     <page>
>         <title>Adeln</title>
>         <ns>0</ns>
>         <id>87</id>
>         <redirect title="Adel" />
>         <revision>
>           <id>8687318</id>
>           <parentid>662406</parentid>
>           <timestamp>2009-03-04T09:01:19Z</timestamp>
>           <contributor>
>             <username>LA2-bot</username>
>             <id>47085</id>
>           </contributor>
>           <minor />
>           <comment>bildputs</comment>
>           <model>wikitext</model>
>           <format>text/x-wiki</format>
>           <text bytes="22" xml:space="preserve">#OMDIRIGERING [[adel]]</text>
>           <sha1>9il0pd8u635nn6kwwoc7wcayoomwn44</sha1>
>         </revision>
>       </page>
>     count: Int = 0
>     current: String = <page>
>         <title>Adeln</title>
>         <ns>0</ns>
>         <id>87</id>
>         <redirect title="Adel" />
>         <revision>
>           <id>8687318</id>
>           <parentid>662406</parentid>
>           <timestamp>2009-03-04T09:01:19Z</timestamp>
>           <contributor>
>             <username>LA2-bot</username>
>             <id>47085</id>
>           </contributor>
>           <minor />
>           <comment>bildputs</comment>
>           <model>wikitext</model>
>           <format>text/x-wiki</format>
>           <text bytes="22" xml:space="preserve">#OMDIRIGERING [[adel]]</text>
>           <sha1>9il0pd8u635nn6kwwoc7wcayoomwn44</sha1>
>         </revision>
>       </page>

  

Convert to DataFrame

In [ ]:
// Let's discard the smaller articles 
val recorpus = corpus.filter({ case (id, page) => page.text.length() >= 1000})

  

>     recorpus: org.apache.spark.rdd.RDD[(Long, Page)] = MapPartitionsRDD[2668] at filter at command-2158711750343073:2

In [ ]:
// Convert RDD to DF with ID for every document 
val corpus_df = recorpus.map({case (wpid,item) => (item.title,item.text,wpid)}).toDF("title","corpus", "id")

  

>     corpus_df: org.apache.spark.sql.DataFrame = [title: string, corpus: string ... 1 more field]

  

Tokenization
============

In [ ]:
import org.apache.spark.ml.feature.RegexTokenizer

// Set params for RegexTokenizer
val tokenizer = new RegexTokenizer()
//.setPattern("[\\W_]+") // break at non-word character(s). \\W is [^A-Za-z0-9_]
.setPattern("[^a-zåäö]+") // break at non-word character(s). \\W is [^A-Za-z0-9_]
.setMinTokenLength(4) // Filter away tokens with length < 4
.setInputCol("corpus") // name of the input column
.setOutputCol("tokens") // name of the output column

// Tokenize document
val tokenized_df = tokenizer.transform(corpus_df)

  

>     import org.apache.spark.ml.feature.RegexTokenizer
>     tokenizer: org.apache.spark.ml.feature.RegexTokenizer = RegexTokenizer: uid=regexTok_76805a7aeefa, minTokenLength=4, gaps=true, pattern=[^a-zåäö]+, toLowercase=true
>     tokenized_df: org.apache.spark.sql.DataFrame = [title: string, corpus: string ... 2 more fields]

In [ ]:
display(tokenized_df.sample(0.001).select("tokens"))

  

Remove Stopwords
================

In [ ]:
val stopwords = sc.textFile(stopword_path).collect()

  

>     stopwords: Array[String] = Array(word, all, alla, allas, allt, alltså, andra, andras, annan, annat, artonde, artonn, att, av, bakom, bara, bland, blev, bli, blir, blivit, båda, bådas, både, dag, dagar, dagarna, dagen, de, del, delen, dem, den, denna, deras, dess, dessa, det, detta, dig, din, dina, dit, ditt, dock, dom, du, där, därför, då, efter, eftersom, ej, elfte, eller, elva, en, er, era, ert, ett, fall, fanns, fast, fem, femte, femtio, femtionde, femton, femtonde, fick, fin, finnas, finns, fjorton, fjortonde, fjärde, fler, flera, flesta, fram, framför, från, fyra, fyrtio, fyrtionde, få, får, fått, för, före, förra, första, ge, genom, ger, gick, gjorde, gjort, gälla, gäller, gällt, gärna, gå, gång, går, gått, gör, göra, ha, hade, haft, han, hans, har, hela, heller, hellre, helst, helt, henne, hennes, heter, hit, hjälp, hon, honom, hundra, hundraen, hundraett, hur, här, i, ibland, icke, igen, in, inför, inga, ingen, inget, innan, inne, inom, inte, inuti, ja, jag, ju, jämfört, kan, kom, komma, kommer, kommit, kunde, kunna, kunnat, kvar, kör, legat, ligga, ligger, lägga, man, med, mellan, men, menar, mer, mera, mig, min, mina, mitt, mittemot, mot, ned, nederst, nedersta, nedre, nej, ner, ni, nio, nionde, nittio, nittionde, nitton, nittonde, nog, noll, nr, nu, nummer, nya, när, nästa, någon, någonting, något, några, nån, nåt, och, också, om, oss, på, rakt, redan, sade, sagt, samt, sedan, sen, ser, sex, sextio, sextionde, sexton, sextonde, sig, sin, sina, sist, sista, siste, sitt, sitta, sju, sjunde, sjuttio, sjuttionde, sjutton, sjuttonde, själv, sjätte, ska, skall, skriver, skulle, slutligen, snart, som, står, säga, säger, sätt, så, sådan, sådana, sådant, ta, tack, tar, till, tills, tio, tionde, tjugo, tjugoen, tjugoett, tjugonde, tjugotre, tjugotvå, tjungo, tog, tolfte, tolv, tre, tredje, trettio, trettionde, tretton, trettonde, tro, tror, två, tvåhundra, under, upp, ur, ut, utan, utanför, ute, vad, var, vara, varför, varifrån, varit, varje, varken, vars, varsågod, vart, vem, vems, verkligen, vet, vi, vid, vidare, vilka, vilkas, vilken, vilket, vill, visst, väl, värre, vår, våra, vårt, än, ändå, ännu, är, även, året, åt, åtminstone, åtta, åttio, åttionde, åttonde, över, övre)

  

add custom stopwords

In [ ]:
val add_stopwords = Array("label", "note", "area", "unit", "type", "mark", "long", "right", "kartposition", "quot", "text", "title", "page", "timestamp", "revision", "name", "username", "sha1", "format", "coord", "left", "center", "align", "region", "nasa", "source", "mouth", "species", "highest", "style", "kategori", "http", "wikipedia", "referenser", "källor", "noter")

  

>     add_stopwords: Array[String] = Array(label, note, area, unit, type, mark, long, right, kartposition, quot, text, title, page, timestamp, revision, name, username, sha1, format, coord, left, center, align, region, nasa, source, mouth, species, highest, style, kategori, http, wikipedia, referenser, källor, noter)

In [ ]:
// Combine newly identified stopwords to our exising list of stopwords
val new_stopwords = stopwords.union(add_stopwords)

  

>     new_stopwords: Array[String] = Array(word, all, alla, allas, allt, alltså, andra, andras, annan, annat, artonde, artonn, att, av, bakom, bara, bland, blev, bli, blir, blivit, båda, bådas, både, dag, dagar, dagarna, dagen, de, del, delen, dem, den, denna, deras, dess, dessa, det, detta, dig, din, dina, dit, ditt, dock, dom, du, där, därför, då, efter, eftersom, ej, elfte, eller, elva, en, er, era, ert, ett, fall, fanns, fast, fem, femte, femtio, femtionde, femton, femtonde, fick, fin, finnas, finns, fjorton, fjortonde, fjärde, fler, flera, flesta, fram, framför, från, fyra, fyrtio, fyrtionde, få, får, fått, för, före, förra, första, ge, genom, ger, gick, gjorde, gjort, gälla, gäller, gällt, gärna, gå, gång, går, gått, gör, göra, ha, hade, haft, han, hans, har, hela, heller, hellre, helst, helt, henne, hennes, heter, hit, hjälp, hon, honom, hundra, hundraen, hundraett, hur, här, i, ibland, icke, igen, in, inför, inga, ingen, inget, innan, inne, inom, inte, inuti, ja, jag, ju, jämfört, kan, kom, komma, kommer, kommit, kunde, kunna, kunnat, kvar, kör, legat, ligga, ligger, lägga, man, med, mellan, men, menar, mer, mera, mig, min, mina, mitt, mittemot, mot, ned, nederst, nedersta, nedre, nej, ner, ni, nio, nionde, nittio, nittionde, nitton, nittonde, nog, noll, nr, nu, nummer, nya, när, nästa, någon, någonting, något, några, nån, nåt, och, också, om, oss, på, rakt, redan, sade, sagt, samt, sedan, sen, ser, sex, sextio, sextionde, sexton, sextonde, sig, sin, sina, sist, sista, siste, sitt, sitta, sju, sjunde, sjuttio, sjuttionde, sjutton, sjuttonde, själv, sjätte, ska, skall, skriver, skulle, slutligen, snart, som, står, säga, säger, sätt, så, sådan, sådana, sådant, ta, tack, tar, till, tills, tio, tionde, tjugo, tjugoen, tjugoett, tjugonde, tjugotre, tjugotvå, tjungo, tog, tolfte, tolv, tre, tredje, trettio, trettionde, tretton, trettonde, tro, tror, två, tvåhundra, under, upp, ur, ut, utan, utanför, ute, vad, var, vara, varför, varifrån, varit, varje, varken, vars, varsågod, vart, vem, vems, verkligen, vet, vi, vid, vidare, vilka, vilkas, vilken, vilket, vill, visst, väl, värre, vår, våra, vårt, än, ändå, ännu, är, även, året, åt, åtminstone, åtta, åttio, åttionde, åttonde, över, övre, label, note, area, unit, type, mark, long, right, kartposition, quot, text, title, page, timestamp, revision, name, username, sha1, format, coord, left, center, align, region, nasa, source, mouth, species, highest, style, kategori, http, wikipedia, referenser, källor, noter)

  

Remove stopwords

In [ ]:
import org.apache.spark.ml.feature.StopWordsRemover

// Set params for StopWordsRemover
val remover = new StopWordsRemover()
.setStopWords(new_stopwords) // This parameter is optional
.setInputCol("tokens")
.setOutputCol("filtered")

// Create new DF with Stopwords removed
val filtered_df = remover.transform(tokenized_df)

  

>     import org.apache.spark.ml.feature.StopWordsRemover
>     remover: org.apache.spark.ml.feature.StopWordsRemover = StopWordsRemover: uid=stopWords_5e439fe0fa41, numStopWords=366, locale=en, caseSensitive=false
>     filtered_df: org.apache.spark.sql.DataFrame = [title: string, corpus: string ... 3 more fields]

  

Vector of token counts
======================

In [ ]:
import org.apache.spark.ml.feature.CountVectorizer

// Set params for CountVectorizer
val vectorizer = new CountVectorizer()
.setInputCol("filtered")
.setOutputCol("features")
.setVocabSize(10000) 
.setMinDF(5) // the minimum number of different documents a term must appear in to be included in the vocabulary.
.fit(filtered_df)

  

>     import org.apache.spark.ml.feature.CountVectorizer
>     vectorizer: org.apache.spark.ml.feature.CountVectorizerModel = CountVectorizerModel: uid=cntVec_ab86d7993954, vocabularySize=10000

In [ ]:
// Create vector of token counts
val countVectors = vectorizer.transform(filtered_df).select("id", "features")

  

>     countVectors: org.apache.spark.sql.DataFrame = [id: bigint, features: vector]

In [ ]:
// see the first countVectors
countVectors.take(2)

  

>     res56: Array[org.apache.spark.sql.Row] = Array([10671,(10000,[32,46,50,56,58,67,85,86,88,94,103,104,106,107,111,119,122,123,127,129,148,153,158,159,162,164,173,177,186,187,190,198,200,228,233,247,254,272,276,279,280,299,314,375,377,411,465,476,519,532,565,591,593,594,632,648,714,734,755,801,837,855,883,886,888,917,949,1019,1055,1077,1097,1098,1124,1221,1240,1248,1295,1309,1399,1464,1484,1521,1555,1561,1744,1799,1821,1840,1848,1866,2033,2092,2130,2136,2197,2442,2451,2528,2598,2809,2937,3044,3328,3487,3571,3580,3626,3669,3690,3867,3871,3967,4277,4292,4608,4684,4795,4837,4848,5199,5327,5388,5431,5467,5483,5901,5948,5963,6158,6250,6258,6377,6401,6417,6452,6823,7458,8363,8755,8806,8807,9208,9645,9943],[1.0,1.0,7.0,2.0,1.0,4.0,1.0,2.0,1.0,1.0,3.0,3.0,2.0,1.0,3.0,2.0,1.0,2.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,2.0,4.0,1.0,4.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0])], [52187,(10000,[0,1,3,6,9,14,17,22,28,29,32,37,41,42,45,50,56,57,58,60,61,63,67,73,75,77,85,87,88,89,91,94,102,103,104,106,107,111,113,116,119,123,124,126,127,129,134,136,138,139,142,147,148,151,155,157,158,159,162,164,165,169,171,175,179,180,181,186,189,190,198,199,200,202,205,208,211,212,217,218,219,225,231,233,234,248,249,256,263,265,267,274,276,278,280,286,287,290,294,295,296,297,299,303,304,314,319,326,328,329,334,338,340,344,346,351,354,375,378,383,396,397,401,403,406,407,408,410,416,421,423,427,429,433,435,439,441,454,455,456,459,475,476,482,489,495,497,508,519,550,565,578,591,593,595,597,598,601,611,612,617,632,639,642,644,650,659,661,688,689,691,693,695,701,706,713,714,722,724,727,731,732,735,746,757,762,764,766,768,769,771,775,777,781,784,790,803,808,813,817,822,823,830,837,840,842,846,858,869,871,874,875,880,890,906,911,923,925,953,958,976,979,982,987,995,1025,1028,1036,1050,1055,1058,1061,1062,1067,1072,1077,1083,1088,1092,1098,1103,1106,1110,1124,1128,1142,1143,1149,1153,1169,1172,1173,1178,1182,1191,1221,1232,1233,1234,1243,1248,1252,1253,1269,1284,1289,1295,1301,1307,1309,1329,1335,1348,1353,1356,1376,1387,1399,1415,1419,1424,1446,1467,1494,1496,1514,1515,1517,1527,1544,1549,1562,1563,1584,1587,1608,1620,1641,1646,1651,1741,1751,1755,1758,1760,1773,1774,1824,1829,1842,1847,1849,1871,1887,1912,1929,1949,1976,1979,1980,1986,1991,1994,2000,2002,2010,2022,2033,2068,2095,2111,2127,2129,2131,2136,2170,2187,2197,2201,2225,2236,2259,2276,2307,2324,2336,2384,2398,2415,2416,2424,2438,2472,2488,2499,2509,2542,2599,2601,2645,2658,2663,2715,2726,2729,2747,2753,2767,2784,2809,2820,2841,2895,2912,2918,2937,3000,3012,3027,3043,3044,3045,3091,3126,3155,3162,3189,3300,3345,3350,3367,3380,3452,3490,3493,3500,3501,3509,3520,3556,3569,3582,3626,3635,3660,3694,3715,3743,3750,3792,3799,3800,3865,3880,3892,3902,3915,3932,3936,3956,3977,3990,3992,4013,4049,4054,4082,4119,4138,4147,4183,4193,4205,4245,4313,4329,4344,4359,4368,4392,4430,4459,4471,4476,4511,4535,4550,4577,4580,4600,4614,4621,4624,4650,4711,4712,4777,4832,4895,4926,4930,4976,5001,5112,5143,5173,5192,5213,5247,5272,5295,5342,5373,5395,5406,5411,5470,5484,5489,5576,5581,5642,5659,5665,5713,5735,5738,5911,5954,6060,6074,6152,6218,6331,6332,6387,6444,6495,6527,6536,6540,6607,6613,6731,6799,6841,6847,7032,7039,7066,7070,7145,7215,7220,7223,7327,7403,7461,7565,7594,7631,7658,7684,7695,7696,7796,7814,7822,7831,7961,8027,8089,8110,8294,8316,8372,8405,8421,8457,8582,8584,8647,8699,8728,8733,8785,8816,8821,8912,8914,8921,8969,9003,9007,9069,9080,9086,9109,9120,9277,9318,9358,9365,9441,9448,9453,9469,9528,9529,9584,9697,9712,9728,9753,9831,9891,9941,9975,9982],[1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,8.0,6.0,1.0,5.0,1.0,1.0,2.0,3.0,1.0,3.0,1.0,2.0,2.0,6.0,1.0,11.0,2.0,1.0,2.0,7.0,5.0,6.0,3.0,1.0,1.0,5.0,2.0,4.0,2.0,3.0,1.0,2.0,12.0,1.0,2.0,1.0,1.0,6.0,2.0,1.0,1.0,1.0,2.0,3.0,2.0,2.0,4.0,1.0,1.0,4.0,3.0,1.0,5.0,2.0,4.0,4.0,1.0,3.0,1.0,1.0,2.0,11.0,1.0,1.0,1.0,1.0,1.0,5.0,2.0,1.0,1.0,4.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,4.0,4.0,3.0,9.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,10.0,1.0,3.0,1.0,1.0,1.0,4.0,1.0,1.0,3.0,3.0,6.0,6.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,10.0,1.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,4.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,3.0,1.0,1.0,1.0,3.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,7.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,3.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,1.0,5.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,3.0,3.0,1.0,1.0,1.0,1.0,44.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,3.0,1.0,1.0,1.0,3.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,6.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,4.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,6.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,3.0,1.0,1.0,2.0,3.0,1.0,1.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,9.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,4.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,7.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,16.0,3.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,6.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,3.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,3.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,4.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,10.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,6.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,6.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0])])

In [ ]:
// Convert DF to RDD
import org.apache.spark.ml.linalg.Vector

val lda_countVector = countVectors.map { case Row(id: Long, countVector: Vector) => (id, countVector) }

  

>     import org.apache.spark.ml.linalg.Vector
>     lda_countVector: org.apache.spark.sql.Dataset[(Long, org.apache.spark.ml.linalg.Vector)] = [_1: bigint, _2: vector]

In [ ]:
// format: Array(id, (VocabSize, Array(indexedTokens), Array(Token Frequency)))
lda_countVector.take(1)

  

>     res58: Array[(Long, org.apache.spark.ml.linalg.Vector)] = Array((10671,(10000,[32,46,50,56,58,67,85,86,88,94,103,104,106,107,111,119,122,123,127,129,148,153,158,159,162,164,173,177,186,187,190,198,200,228,233,247,254,272,276,279,280,299,314,375,377,411,465,476,519,532,565,591,593,594,632,648,714,734,755,801,837,855,883,886,888,917,949,1019,1055,1077,1097,1098,1124,1221,1240,1248,1295,1309,1399,1464,1484,1521,1555,1561,1744,1799,1821,1840,1848,1866,2033,2092,2130,2136,2197,2442,2451,2528,2598,2809,2937,3044,3328,3487,3571,3580,3626,3669,3690,3867,3871,3967,4277,4292,4608,4684,4795,4837,4848,5199,5327,5388,5431,5467,5483,5901,5948,5963,6158,6250,6258,6377,6401,6417,6452,6823,7458,8363,8755,8806,8807,9208,9645,9943],[1.0,1.0,7.0,2.0,1.0,4.0,1.0,2.0,1.0,1.0,3.0,3.0,2.0,1.0,3.0,2.0,1.0,2.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,2.0,4.0,1.0,4.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0])))

  

LDA
===

In [ ]:
val numTopics = 50 // how to set this?

  

>     numTopics: Int = 50

In [ ]:
import org.apache.spark.mllib.clustering.{LDA, OnlineLDAOptimizer}

// Set LDA params
val lda = new LDA()
.setOptimizer(new OnlineLDAOptimizer().setMiniBatchFraction(0.8))
.setK(numTopics)
.setMaxIterations(20) // this probably has to be increased (from 3)
.setDocConcentration(-1) // use default values
.setTopicConcentration(-1) // use default values

  

>     import org.apache.spark.mllib.clustering.{LDA, OnlineLDAOptimizer}
>     lda: org.apache.spark.mllib.clustering.LDA = org.apache.spark.mllib.clustering.LDA@6e5c9b06

In [ ]:
// convert ML vectors into MLlib vectors
val lda_countVector_mllib = lda_countVector.map { case (id, vector) => (id, org.apache.spark.mllib.linalg.Vectors.fromML(vector)) }.rdd

// Run LDA
val ldaModel = lda.run(lda_countVector_mllib)

  

>     lda_countVector_mllib: org.apache.spark.rdd.RDD[(Long, org.apache.spark.mllib.linalg.Vector)] = MapPartitionsRDD[2537] at rdd at command-738986526041666:2
>     ldaModel: org.apache.spark.mllib.clustering.LDAModel = org.apache.spark.mllib.clustering.LocalLDAModel@340cd394

  

Review topics
=============

In [ ]:
val topicIndices = ldaModel.describeTopics(maxTermsPerTopic = 5)

  

>     topicIndices: Array[(Array[Int], Array[Double])] = Array((Array(910, 901, 510, 1197, 1761),Array(0.025059308870353184, 0.023400523968226696, 0.021195980743513396, 0.01626841008683287, 0.011214220970687016)), (Array(86, 146, 84, 196, 79),Array(0.03489516880955898, 0.025767746354246373, 0.016433187014153725, 0.014906573557897212, 0.010856865738603862)), (Array(651, 1700, 856, 859, 482),Array(0.10405263400187738, 0.03358165905918105, 0.03229756279665303, 0.032173363022884996, 0.02460256231682459)), (Array(50, 78, 69, 9, 85),Array(0.0057425403481240886, 0.00496435167053226, 0.004085380412935543, 0.0037827665048466796, 0.003671126083304182)), (Array(301, 300, 330, 336, 283),Array(0.010721479677025468, 0.010469511952728851, 0.008977552368263735, 0.008915902027876533, 0.007890189722208438)), (Array(128, 91, 136, 189, 127),Array(0.016560445257658117, 0.011293495672173312, 0.010506676818781705, 0.008449219540389546, 0.007499141959420526)), (Array(245, 2, 7, 3, 2436),Array(0.0542557984447277, 0.040408714303919666, 0.031171475853542692, 0.030511596184153392, 0.023540508986777616)), (Array(31, 51, 117, 92, 42),Array(0.15211150313293795, 0.04989770661531165, 0.049563866073212975, 0.04549484447553375, 0.03270669790929784)), (Array(31, 51, 345, 269, 219),Array(0.09255690838138343, 0.04920060501893783, 0.045482792389492585, 0.04374604159360783, 0.03574394980068257)), (Array(323, 1, 0, 2, 3),Array(0.07927038318689555, 0.028093246774054958, 0.02280181919266087, 0.021026495614210137, 0.018748571452188615)), (Array(185, 310, 503, 677, 264),Array(0.05751325153735175, 0.03443085235167512, 0.019601654340390597, 0.016781617645777836, 0.01658381820384268)), (Array(197, 188, 50, 143, 926),Array(0.026932065720601476, 0.022371972152705526, 0.0149631243059905, 0.008417565189216571, 0.006873690561937945)), (Array(0, 1, 2, 3, 4),Array(0.04504264786028259, 0.037326055844557414, 0.03257595419588929, 0.029718365953605424, 0.029331310988027017)), (Array(69, 84, 78, 70, 195),Array(0.010213734058180836, 0.0071866451286654335, 0.006683540279448935, 0.005883089788623208, 0.0054395801833071545)), (Array(213, 204, 403, 173, 183),Array(0.01740744406565029, 0.013162308863941057, 0.012092680452178768, 0.011448032616412316, 0.009096444292483394)), (Array(307, 70, 85, 291, 50),Array(0.00854504531365484, 0.006693592306258885, 0.005373850640154544, 0.005045685049475926, 0.004713423047922566)), (Array(528, 85, 107, 67, 91),Array(0.005652390811371258, 0.004741662061397474, 0.004514445765318061, 0.004203273591058431, 0.004084683065638065)), (Array(277, 484, 0, 2216, 1520),Array(0.06084841209854684, 0.051127666322883146, 0.01853132239855695, 0.01773031736821437, 0.016088442450282316)), (Array(1528, 3445, 3928, 2437, 1521),Array(0.04045624440182511, 0.034005650512617336, 0.02535678915947196, 0.023028259444303992, 0.01929037364979745)), (Array(522, 51, 483, 936, 31),Array(0.09760147281027223, 0.09595713187524844, 0.049993025274011706, 0.03046714380890495, 0.020460914651187472)), (Array(149, 0, 2, 3, 4),Array(0.04450632653283522, 0.03192872030447543, 0.025280264220534414, 0.024144452977865827, 0.02158435649122772)), (Array(50, 147, 111, 104, 143),Array(0.018287793328550822, 0.011660261152121208, 0.0065906421664898955, 0.00635955956012892, 0.006036999940163723)), (Array(2157, 4331, 8749, 6652, 5430),Array(0.04725750825084046, 0.023433361090418693, 0.01323259071372079, 0.013092630222814863, 0.011284949984719083)), (Array(730, 904, 0, 363, 2),Array(0.0740933767994304, 0.02902821451152532, 0.01688992637013919, 0.015069746721666782, 0.013077685991533665)), (Array(1006, 1427, 298, 1655, 50),Array(0.011875757162670324, 0.010646493174694176, 0.009891987765354214, 0.008193162135398413, 0.005456629205741401)), (Array(0, 2, 1, 4, 5),Array(0.04598539439861342, 0.03443065132806134, 0.03258287132483066, 0.030395182408479566, 0.030354929698208518)), (Array(352, 204, 225, 557, 125),Array(0.023879395302562268, 0.02002733234360079, 0.018538815765005035, 0.016183152904712696, 0.014904949179581357)), (Array(44, 102, 46, 7, 1),Array(0.19825300837251528, 0.03267677751256385, 0.031372696745660296, 0.019441278449670286, 0.0192495821975038)), (Array(79, 70, 144, 206, 242),Array(0.029605227787081547, 0.02862249487074852, 0.027991174520283903, 0.022262250791660873, 0.015430182241441267)), (Array(645, 2, 429, 142, 1041),Array(0.023755348731854732, 0.020176054236135908, 0.018228241395018146, 0.014998717568131764, 0.012198259818301941)), (Array(261, 70, 343, 367, 244),Array(0.01805314877885552, 0.013481544995189861, 0.012079215089885553, 0.011259797873311078, 0.010672282286260678)), (Array(271, 302, 931, 83, 1332),Array(0.10447372402735176, 0.09695325932564189, 0.03459700308233657, 0.028370705755756644, 0.02493111812949939)), (Array(487, 1281, 1259, 2066, 1783),Array(0.009569167419331554, 0.0071050551840972, 0.005528217027909177, 0.00525733000491023, 0.004659204820748785)), (Array(1652, 2237, 2591, 1091, 1551),Array(0.045817120411665344, 0.03254208587767372, 0.029632272074815145, 0.02827867170768587, 0.01639467086457866)), (Array(11, 47, 68, 54, 772),Array(0.012362892953283203, 0.012334199975737302, 0.011563682869594232, 0.01072995324382497, 0.008327916437339029)), (Array(1594, 169, 6, 1252, 3),Array(0.015461014895925804, 0.01438616727808855, 0.013585745962777604, 0.012073701553225354, 0.009914954538117455)), (Array(312, 203, 750, 1550, 69),Array(0.04906384267437765, 0.01844981022607237, 0.013242853842446398, 0.01164679731230208, 0.010784594256947616)), (Array(31, 95, 51, 121, 71),Array(0.11913277442194449, 0.1068740519361396, 0.10073116068293635, 0.08612996755830332, 0.02096991701410841)), (Array(1518, 2624, 3194, 5522, 0),Array(0.04862130674694627, 0.012955210935212914, 0.012097185039298429, 0.011658647411780567, 0.011398066152273945)), (Array(44, 71, 31, 238, 95),Array(0.11858698163965047, 0.06075058933507229, 0.04188735863269003, 0.024933961017808128, 0.023368281723564005)), (Array(1535, 520, 437, 1012, 2550),Array(0.018511806206536377, 0.018491874573148, 0.010229130946065097, 0.009923475795512782, 0.009427019233066956)), (Array(79, 559, 185, 324, 439),Array(0.00822465048589354, 0.007279394538684116, 0.007140796703834101, 0.0067471563229271215, 0.006729316593919472)), (Array(1, 0, 2, 4, 3),Array(0.044057251546186484, 0.033629229480123694, 0.03130301649642094, 0.03091824560881027, 0.03067893992134706)), (Array(166, 0, 1, 145, 150),Array(0.037308579989265865, 0.03702054761922084, 0.0325624055866806, 0.026139559696209277, 0.02438514776341585)), (Array(70, 68, 48, 80, 62),Array(0.009875959165704832, 0.009217054727638114, 0.009130097490158934, 0.008858377271573493, 0.008727261943084602)), (Array(191, 161, 327, 335, 152),Array(0.020736851111366656, 0.01633330545811541, 0.015635380079718257, 0.01289782084286368, 0.01279265489050279)), (Array(82, 105, 98, 130, 172),Array(0.0786507643327494, 0.06423691443896841, 0.05705289322090071, 0.04409509137084651, 0.031785742083129814)), (Array(39, 42, 83, 221, 92),Array(0.24746595175338434, 0.07094748980724597, 0.03736557629148667, 0.02492131478963847, 0.017170592223449976)), (Array(216, 2423, 2426, 4993, 8020),Array(0.10011429904845326, 0.05631896060381116, 0.029576990277144867, 0.026338775967562476, 0.011919538108387324)), (Array(1665, 841, 1078, 1703, 259),Array(0.02090893858307959, 0.015297563921716708, 0.015181805530522997, 0.013835395518137523, 0.013160601333366323)))

In [ ]:
val vocabList = vectorizer.vocabulary

  

>     vocabList: Array[String] = Array(runt, trakten, meter, havet, månaden, nederbörd, invånare, kvadratkilometer, terrängen, landet, närheten, januari, huvudstaden, medeltemperaturen, varmaste, kallaste, årsmedeltemperaturen, högsta, kommentarer, nasalandcover, nasapop, millimeter, genomsnitt, genomsnittlig, torraste, årsnederbörd, regnigaste, nasarain, punkten, juli, koppen, geonames, större, omgivningarna, kuperad, augusti, råder, huvudsak, närmaste, cerro, samhälle, provinsen, berg, toppen, lake, ganska, kilometer, februari, juni, öster, talet, vattendrag, nordväst, nordost, december, platt, söder, väster, norr, glesbefolkat, växer, sydväst, september, huvudsakligen, tätbefolkat, sydost, november, mycket, mars, svenska, född, kanada, ingår, består, nära, mindre, april, centrala, sverige, svensk, oktober, samma, from, kulle, stockholm, senare, kommun, antalet, största, omgivande, basen, olika, mexiko, bredden, norra, county, sydöstra, bergig, small, klimatzonen, jordbruksmark, kommunen, riktning, tidigare, stora, shift, södra, många, regionen, obefolkad, blandskog, området, följande, tillsammans, ottawa, växtlighet, lite, robotskapade, hemiboreala, östra, delstaten, creek, namn, västra, enligt, läst, namnet, ofta, math, började, value, sveriges, öppet, city, stor, countyt, exempel, stäppklimat, nord, början, busklandskap, kallt, plats, gallery, amerikansk, iran, socken, staden, mest, gunung, teheran, historia, vann, personer, gräsmarker, persiska, mexico, därefter, sträcker, grund, arean, spelade, endast, platser, medan, frankrike, cheshmeh, lövskog, carl, cirka, västlig, sydlig, linegrey, förekommer, norrut, övrigt, cobar, åren, john, först, gamla, söderut, klimatet, familjen, ökenklimat, filmen, idag, naturlig, kyrka, används, vissa, säsongen, mosaik, nordvästra, johan, barn, församling, kyrkan, antal, österut, delar, västerut, tropiskt, författare, iucn, bort, skådespelare, medelhavsklimat, slutet, göteborg, samhällen, omkring, nordöstra, arten, städsegrön, stockholms, saint, tiden, tyskland, indonesien, ofruktbar, loma, karl, tyska, savannklimat, world, distriktet, skrev, kallas, känd, band, främst, erik, stort, bästa, gången, finland, departementet, ontario, width, gray, sluttar, politiker, periodiskt, uppsala, sjön, istället, samtidigt, amerikanska, nuvarande, serien, york, ordningen, timeline, malmö, london, engelska, plotdata, gift, gånger, biografi, universitet, egen, franska, regi, tillbaka, file, trots, version, ryssland, island, bosnien, byggdes, peter, äldre, black, kring, santa, form, samband, kung, hett, deltog, sång, musik, maria, dessutom, därmed, slut, chile, europa, paris, jean, ytterligare, storbritannien, kallade, lång, land, sida, kända, album, albumet, hercegovina, sidan, miljoner, enda, medlemmar, verk, gruppen, jabal, film, period, översättning, england, övriga, jakarta, dotter, måste, gustaf, sydvästra, ledde, inlandsklimat, ordförande, mountain, anders, längre, norge, matcher, spanien, fortfarande, låten, savannskog, stad, flyttade, världen, spelare, bandet, källa, särskilt, lars, grupp, david, numera, professor, aldrig, brasilien, kriget, anna, varierad, bildades, visar, länder, birds, australien, världskriget, läggs, karriär, district, fredrik, provincia, tall, tysk, totalt, marocko, kort, pass, syfta, ledamot, park, tempererat, indien, urval, ryska, nytt, poäng, tillhör, notera, danmark, exempelvis, kartan, göteborgs, ändras, mount, internationella, arbetade, användes, puerto, stod, rabat, museum, släpptes, kallad, platserna, innehåller, university, robert, varav, människor, start, martin, color, nästan, förbli, arter, gitarr, såsom, dagens, höjd, världens, bergstopp, ungefär, liksom, rubrik, österrike, studerade, boreala, italien, korrigeras, mallen, rättvisande, ursprungligen, brittiska, paul, egna, slutade, säsong, kvinnor, mitten, klubben, guld, fortsatte, thomas, fuktigt, bild, boken, subtropiskt, lämnade, division, ville, andersson, djebel, språk, gifte, roll, kungliga, year, lika, förgreningar, huset, laget, medlem, population, spela, oftast, serie, liten, politiska, nils, arbete, högre, gustav, långa, förutom, afghanistan, procent, verksamhet, kina, william, james, regeringen, bergspass, spelades, kantonen, berlin, studier, själva, områden, punta, president, lyckades, british, brittisk, argentina, colombia, alto, bror, befolkningsutveckling, företaget, michael, vatten, love, lever, historik, lund, gora, europeiska, regnskogsklimat, republiken, chef, airport, sjöarna, spelar, varmt, landskommun, varandra, utbredning, innebär, släktet, respektive, rätt, grand, uppfördes, officiella, anses, sven, årets, använda, förlorade, skolan, område, league, henrik, quebrada, direkt, wien, columbia, santiago, japan, spelet, grundades, vita, slaget, dels, late, hill, titeln, george, little, referens, hand, kabul, böcker, international, familj, fransk, johansson, fontsize, startade, increment, olof, modern, radio, lilla, spng, https, gridcolor, stöd, clements, slott, avsnitt, trummor, användas, bottom, magnus, udde, minst, liknande, height, ghar, colors, vertical, använder, orientation, structure, ålder, föddes, svenskt, enwiki, längs, nilsson, imagesize, timeaxis, plotarea, scalemajor, dateformat, helsingfors, alignbars, yyyy, tillhörde, textcolor, bilder, talets, monsunklimat, långt, danska, uppdrag, volution, äldsta, svart, avled, arrondissementet, kungen, visade, tidigt, tidningen, sommaren, grande, insee, chiffres, togs, hösten, alltid, vanligen, madrid, problem, börjar, samtliga, länge, bestod, kronor, system, sankt, richard, charles, spel, låtar, gavs, uppe, flertal, pasir, troligen, south, moderna, åter, musiker, jawa, bibliografi, förlag, byggnaden, delas, axel, enbart, utgör, lagen, gammal, olympiska, ingick, företag, centrum, utveckling, omfattande, laguna, eget, samhället, växte, rock, person, sett, nådde, betydelse, partiet, kallades, north, grekland, nere, gunnar, gjordes, villa, strax, life, viss, live, resultat, stadens, kalifornien, utseende, medverkade, filmer, ordning, jebel, bukit, ledare, betyder, status, marie, handling, silver, sångare, ursprungliga, geografi, vägen, såväl, snabbt, röda, släkte, konst, gård, uppgick, with, ihop, återvände, vapen, administrativ, brdo, bättre, washington, skedde, adolf, underarter, valet, larsson, valde, sarajevo, utbildning, ursprung, national, födda, records, historiska, månader, norska, platsen, ordet, listan, algeriet, general, örebro, höga, verksam, finalen, koudiat, spår, program, kustklimat, ägde, arabiska, august, schweiz, polen, bygga, museet, final, lärare, beroende, handlar, tekniska, byggnader, dåvarande, allra, utmärkelser, roman, delhi, saguenay, match, starkt, delvis, daniel, livet, länet, lappland, press, finska, djur, sten, gränsen, fotboll, följd, sker, samarbete, ekonomiska, föreningen, räknas, programmet, arkitekt, sonora, stället, phou, bakgrund, yngre, stil, härad, använde, främsta, bengt, bildade, cruz, juan, just, konstnär, wilhelm, svarta, emellertid, våren, unga, west, alger, colorado, singel, alexander, river, tjänst, dansk, litteratur, innebar, louis, reservoar, smith, music, landets, pris, mont, skapa, brukar, skillnad, viktig, nordiska, målare, lokala, emot, däremot, lista, mängd, bodde, kultur, hustru, drottning, lades, bruk, spanska, entiteten, känt, priset, allmänna, förste, christian, ledning, chihuahua, befolkningen, krig, belägen, material, star, tidiga, följde, francisco, vanliga, grupper, piano, georg, medeltida, därpå, information, möjligt, tävlingen, arroyo, köpenhamn, aires, forskning, riket, buenos, johannes, italienska, vanligt, sydkorea, ungern, oscar, kroatien, peru, hölls, leder, henry, relativt, nyheter, albert, inklusive, egypten, beslut, ansåg, flyttades, spelat, grekiska, kompositör, jorden, höll, petit, försökte, slog, nova, militär, försök, nederländerna, guvernementet, fotbollsspelare, premiär, teater, valdes, samman, wood, barrskog, diskografi, verksamheten, ford, bong, förening, koudyet, socknen, städer, eriksson, denne, group, länsväg, omfattar, skola, utsågs, kalmar, plan, morro, senaste, produktion, olsson, norsk, gården, exemplar, sovjetunionen, arbetet, grundade, kyrkans, hall, texas, kvinna, tour, federationen, björn, vanlig, hålla, matchen, venezuela, orgel, utnämndes, american, röst, ersattes, staten, församlingen, skåne, distrikt, nationella, beskrevs, jesus, marken, skrivit, dennes, bytte, öppna, umeå, debuterade, skriven, antingen, folk, invigdes, turn, resa, svårt, kontrakt, massa, byggde, förslag, engelsk, kvinnliga, mode, savann, läns, bygger, ansågs, skriva, aktiv, lägre, eric, statens, såldes, tagit, monte, befolkning, tilldelades, skrevs, färg, levande, perioden, egenskaper, royal, kritik, arbeta, lämna, sullivan, stift, inre, sändes, journalist, barcelona, bulu)

In [ ]:
val topics = topicIndices.map { case (terms, termWeights) =>
  terms.map(vocabList(_)).zip(termWeights)
}

  

>     topics: Array[Array[(String, Double)]] = Array(Array((länsväg,0.025059308870353184), (ford,0.023400523968226696), (grand,0.021195980743513396), (prix,0.01626841008683287), (loppet,0.011214220970687016)), Array((kommun,0.03489516880955898), (socken,0.025767746354246373), (stockholm,0.016433187014153725), (församling,0.014906573557897212), (svensk,0.010856865738603862)), Array((laguna,0.10405263400187738), (lago,0.03358165905918105), (aires,0.03229756279665303), (buenos,0.032173363022884996), (argentina,0.02460256231682459)), Array((talet,0.0057425403481240886), (sverige,0.00496435167053226), (svenska,0.004085380412935543), (landet,0.0037827665048466796), (senare,0.003671126083304182)), Array((albumet,0.010721479677025468), (album,0.010469511952728851), (låten,0.008977552368263735), (bandet,0.008915902027876533), (sång,0.007890189722208438)), Array((math,0.016560445257658117), (olika,0.011293495672173312), (exempel,0.010506676818781705), (används,0.008449219540389546), (ofta,0.007499141959420526)), Array((sjön,0.0542557984447277), (meter,0.040408714303919666), (kvadratkilometer,0.031171475853542692), (havet,0.030511596184153392), (avrinningsområdet,0.023540508986777616)), Array((geonames,0.15211150313293795), (vattendrag,0.04989770661531165), (robotskapade,0.049563866073212975), (mexiko,0.04549484447553375), (berg,0.03270669790929784)), Array((geonames,0.09255690838138343), (vattendrag,0.04920060501893783), (brasilien,0.045482792389492585), (ryssland,0.04374604159360783), (indonesien,0.03574394980068257)), Array((mountain,0.07927038318689555), (trakten,0.028093246774054958), (runt,0.02280181919266087), (meter,0.021026495614210137), (havet,0.018748571452188615)), Array((filmen,0.05751325153735175), (film,0.03443085235167512), (landskommun,0.019601654340390597), (filmer,0.016781617645777836), (regi,0.01658381820384268)), Array((kyrkan,0.026932065720601476), (kyrka,0.022371972152705526), (talet,0.0149631243059905), (gallery,0.008417565189216571), (kyrkans,0.006873690561937945)), Array((runt,0.04504264786028259), (trakten,0.037326055844557414), (meter,0.03257595419588929), (havet,0.029718365953605424), (månaden,0.029331310988027017)), Array((svenska,0.010213734058180836), (stockholm,0.0071866451286654335), (sverige,0.006683540279448935), (född,0.005883089788623208), (barn,0.0054395801833071545)), Array((arten,0.01740744406565029), (iucn,0.013162308863941057), (arter,0.012092680452178768), (förekommer,0.011448032616412316), (familjen,0.009096444292483394)), Array((verk,0.00854504531365484), (född,0.006693592306258885), (senare,0.005373850640154544), (paris,0.005045685049475926), (talet,0.004713423047922566)), Array((spelet,0.005652390811371258), (senare,0.004741662061397474), (många,0.004514445765318061), (mycket,0.004203273591058431), (olika,0.004084683065638065)), Array((santa,0.06084841209854684), (alto,0.051127666322883146), (runt,0.01853132239855695), (koudiet,0.01773031736821437), (mali,0.016088442450282316)), Array((naturreservat,0.04045624440182511), (lagoa,0.034005650512617336), (bond,0.02535678915947196), (beach,0.023028259444303992), (strand,0.01929037364979745)), Array((quebrada,0.09760147281027223), (vattendrag,0.09595713187524844), (colombia,0.049993025274011706), (venezuela,0.03046714380890495), (geonames,0.020460914651187472)), Array((gunung,0.04450632653283522), (runt,0.03192872030447543), (meter,0.025280264220534414), (havet,0.024144452977865827), (månaden,0.02158435649122772)), Array((talet,0.018287793328550822), (staden,0.011660261152121208), (området,0.0065906421664898955), (stora,0.00635955956012892), (gallery,0.006036999940163723)), Array((tubu,0.04725750825084046), (qimmat,0.023433361090418693), (drosophila,0.01323259071372079), (seco,0.013092630222814863), (male,0.011284949984719083)), Array((koudiat,0.0740933767994304), (koudyet,0.02902821451152532), (runt,0.01688992637013919), (marocko,0.015069746721666782), (meter,0.013077685991533665)), Array((fartyg,0.011875757162670324), (fartyget,0.010646493174694176), (sida,0.009891987765354214), (hästar,0.008193162135398413), (talet,0.005456629205741401)), Array((runt,0.04598539439861342), (meter,0.03443065132806134), (trakten,0.03258287132483066), (månaden,0.030395182408479566), (nederbörd,0.030354929698208518)), Array((birds,0.023879395302562268), (iucn,0.02002733234360079), (world,0.018538815765005035), (clements,0.016183152904712696), (läst,0.014904949179581357)), Array((lake,0.19825300837251528), (riktning,0.03267677751256385), (kilometer,0.031372696745660296), (kvadratkilometer,0.019441278449670286), (trakten,0.0192495821975038)), Array((svensk,0.029605227787081547), (född,0.02862249487074852), (amerikansk,0.027991174520283903), (skådespelare,0.022262250791660873), (politiker,0.015430182241441267)), Array((olympiska,0.023755348731854732), (meter,0.020176054236135908), (guld,0.018228241395018146), (plats,0.014998717568131764), (herrar,0.012198259818301941)), Array((universitet,0.01805314877885552), (född,0.013481544995189861), (professor,0.012079215089885553), (ledamot,0.011259797873311078), (uppsala,0.010672282286260678)), Array((bosnien,0.10447372402735176), (hercegovina,0.09695325932564189), (federationen,0.03459700308233657), (kulle,0.028370705755756644), (republika,0.02493111812949939)), Array((företaget,0.009569167419331554), (flygplan,0.0071050551840972), (motor,0.005528217027909177), (moll,0.00525733000491023), (modellen,0.004659204820748785)), Array((taung,0.045817120411665344), (barrage,0.03254208587767372), (buntu,0.029632272074815145), (omgången,0.02827867170768587), (herrarnas,0.01639467086457866)), Array((januari,0.012362892953283203), (februari,0.012334199975737302), (mars,0.011563682869594232), (december,0.01072995324382497), (programmet,0.008327916437339029)), Array((castle,0.015461014895925804), (cirka,0.01438616727808855), (invånare,0.013585745962777604), (öarna,0.012073701553225354), (havet,0.009914954538117455)), Array((översättning,0.04906384267437765), (författare,0.01844981022607237), (roman,0.013242853842446398), (bonnier,0.01164679731230208), (svenska,0.010784594256947616)), Array((geonames,0.11913277442194449), (county,0.1068740519361396), (vattendrag,0.10073116068293635), (creek,0.08612996755830332), (kanada,0.02096991701410841)), Array((morne,0.04862130674694627), (prince,0.012955210935212914), (nationalpark,0.012097185039298429), (haiti,0.011658647411780567), (runt,0.011398066152273945)), Array((lake,0.11858698163965047), (kanada,0.06075058933507229), (geonames,0.04188735863269003), (ontario,0.024933961017808128), (county,0.023368281723564005)), Array((strauss,0.018511806206536377), (league,0.018491874573148), (division,0.010229130946065097), (united,0.009923475795512782), (premier,0.009427019233066956)), Array((svensk,0.00822465048589354), (avsnitt,0.007279394538684116), (filmen,0.007140796703834101), (anders,0.0067471563229271215), (andersson,0.006729316593919472)), Array((trakten,0.044057251546186484), (runt,0.033629229480123694), (meter,0.03130301649642094), (månaden,0.03091824560881027), (havet,0.03067893992134706)), Array((cheshmeh,0.037308579989265865), (runt,0.03702054761922084), (trakten,0.0325624055866806), (iran,0.026139559696209277), (teheran,0.02438514776341585)), Array((född,0.009875959165704832), (mars,0.009217054727638114), (juni,0.009130097490158934), (oktober,0.008858377271573493), (september,0.008727261943084602)), Array((säsongen,0.020736851111366656), (spelade,0.01633330545811541), (matcher,0.015635380079718257), (spelare,0.01289782084286368), (vann,0.01279265489050279)), Array((from,0.0786507643327494), (shift,0.06423691443896841), (small,0.05705289322090071), (value,0.04409509137084651), (linegrey,0.031785742083129814)), Array((cerro,0.24746595175338434), (berg,0.07094748980724597), (kulle,0.03736557629148667), (loma,0.02492131478963847), (mexiko,0.017170592223449976)), Array((saint,0.10011429904845326), (gogae,0.05631896060381116), (dahr,0.029576990277144867), (wadi,0.026338775967562476), (sparbank,0.011919538108387324)), Array((bibcode,0.02090893858307959), (star,0.015297563921716708), (solen,0.015181805530522997), (stars,0.013835395518137523), (gånger,0.013160601333366323)))

In [ ]:
val topicIndices = ldaModel.describeTopics(maxTermsPerTopic = 5)
val vocabList = vectorizer.vocabulary
val topics = topicIndices.map { case (terms, termWeights) =>
  terms.map(vocabList(_)).zip(termWeights)
}
println(s"$numTopics topics:")
topics.zipWithIndex.foreach { case (topic, i) =>
  println(s"TOPIC $i")
  topic.foreach { case (term, weight) => println(s"$term\t$weight") }
  println(s"==========")
}

  

>     50 topics:
>     TOPIC 0
>     länsväg	0.025059308870353184
>     ford	0.023400523968226696
>     grand	0.021195980743513396
>     prix	0.01626841008683287
>     loppet	0.011214220970687016
>     ==========
>     TOPIC 1
>     kommun	0.03489516880955898
>     socken	0.025767746354246373
>     stockholm	0.016433187014153725
>     församling	0.014906573557897212
>     svensk	0.010856865738603862
>     ==========
>     TOPIC 2
>     laguna	0.10405263400187738
>     lago	0.03358165905918105
>     aires	0.03229756279665303
>     buenos	0.032173363022884996
>     argentina	0.02460256231682459
>     ==========
>     TOPIC 3
>     talet	0.0057425403481240886
>     sverige	0.00496435167053226
>     svenska	0.004085380412935543
>     landet	0.0037827665048466796
>     senare	0.003671126083304182
>     ==========
>     TOPIC 4
>     albumet	0.010721479677025468
>     album	0.010469511952728851
>     låten	0.008977552368263735
>     bandet	0.008915902027876533
>     sång	0.007890189722208438
>     ==========
>     TOPIC 5
>     math	0.016560445257658117
>     olika	0.011293495672173312
>     exempel	0.010506676818781705
>     används	0.008449219540389546
>     ofta	0.007499141959420526
>     ==========
>     TOPIC 6
>     sjön	0.0542557984447277
>     meter	0.040408714303919666
>     kvadratkilometer	0.031171475853542692
>     havet	0.030511596184153392
>     avrinningsområdet	0.023540508986777616
>     ==========
>     TOPIC 7
>     geonames	0.15211150313293795
>     vattendrag	0.04989770661531165
>     robotskapade	0.049563866073212975
>     mexiko	0.04549484447553375
>     berg	0.03270669790929784
>     ==========
>     TOPIC 8
>     geonames	0.09255690838138343
>     vattendrag	0.04920060501893783
>     brasilien	0.045482792389492585
>     ryssland	0.04374604159360783
>     indonesien	0.03574394980068257
>     ==========
>     TOPIC 9
>     mountain	0.07927038318689555
>     trakten	0.028093246774054958
>     runt	0.02280181919266087
>     meter	0.021026495614210137
>     havet	0.018748571452188615
>     ==========
>     TOPIC 10
>     filmen	0.05751325153735175
>     film	0.03443085235167512
>     landskommun	0.019601654340390597
>     filmer	0.016781617645777836
>     regi	0.01658381820384268
>     ==========
>     TOPIC 11
>     kyrkan	0.026932065720601476
>     kyrka	0.022371972152705526
>     talet	0.0149631243059905
>     gallery	0.008417565189216571
>     kyrkans	0.006873690561937945
>     ==========
>     TOPIC 12
>     runt	0.04504264786028259
>     trakten	0.037326055844557414
>     meter	0.03257595419588929
>     havet	0.029718365953605424
>     månaden	0.029331310988027017
>     ==========
>     TOPIC 13
>     svenska	0.010213734058180836
>     stockholm	0.0071866451286654335
>     sverige	0.006683540279448935
>     född	0.005883089788623208
>     barn	0.0054395801833071545
>     ==========
>     TOPIC 14
>     arten	0.01740744406565029
>     iucn	0.013162308863941057
>     arter	0.012092680452178768
>     förekommer	0.011448032616412316
>     familjen	0.009096444292483394
>     ==========
>     TOPIC 15
>     verk	0.00854504531365484
>     född	0.006693592306258885
>     senare	0.005373850640154544
>     paris	0.005045685049475926
>     talet	0.004713423047922566
>     ==========
>     TOPIC 16
>     spelet	0.005652390811371258
>     senare	0.004741662061397474
>     många	0.004514445765318061
>     mycket	0.004203273591058431
>     olika	0.004084683065638065
>     ==========
>     TOPIC 17
>     santa	0.06084841209854684
>     alto	0.051127666322883146
>     runt	0.01853132239855695
>     koudiet	0.01773031736821437
>     mali	0.016088442450282316
>     ==========
>     TOPIC 18
>     naturreservat	0.04045624440182511
>     lagoa	0.034005650512617336
>     bond	0.02535678915947196
>     beach	0.023028259444303992
>     strand	0.01929037364979745
>     ==========
>     TOPIC 19
>     quebrada	0.09760147281027223
>     vattendrag	0.09595713187524844
>     colombia	0.049993025274011706
>     venezuela	0.03046714380890495
>     geonames	0.020460914651187472
>     ==========
>     TOPIC 20
>     gunung	0.04450632653283522
>     runt	0.03192872030447543
>     meter	0.025280264220534414
>     havet	0.024144452977865827
>     månaden	0.02158435649122772
>     ==========
>     TOPIC 21
>     talet	0.018287793328550822
>     staden	0.011660261152121208
>     området	0.0065906421664898955
>     stora	0.00635955956012892
>     gallery	0.006036999940163723
>     ==========
>     TOPIC 22
>     tubu	0.04725750825084046
>     qimmat	0.023433361090418693
>     drosophila	0.01323259071372079
>     seco	0.013092630222814863
>     male	0.011284949984719083
>     ==========
>     TOPIC 23
>     koudiat	0.0740933767994304
>     koudyet	0.02902821451152532
>     runt	0.01688992637013919
>     marocko	0.015069746721666782
>     meter	0.013077685991533665
>     ==========
>     TOPIC 24
>     fartyg	0.011875757162670324
>     fartyget	0.010646493174694176
>     sida	0.009891987765354214
>     hästar	0.008193162135398413
>     talet	0.005456629205741401
>     ==========
>     TOPIC 25
>     runt	0.04598539439861342
>     meter	0.03443065132806134
>     trakten	0.03258287132483066
>     månaden	0.030395182408479566
>     nederbörd	0.030354929698208518
>     ==========
>     TOPIC 26
>     birds	0.023879395302562268
>     iucn	0.02002733234360079
>     world	0.018538815765005035
>     clements	0.016183152904712696
>     läst	0.014904949179581357
>     ==========
>     TOPIC 27
>     lake	0.19825300837251528
>     riktning	0.03267677751256385
>     kilometer	0.031372696745660296
>     kvadratkilometer	0.019441278449670286
>     trakten	0.0192495821975038
>     ==========
>     TOPIC 28
>     svensk	0.029605227787081547
>     född	0.02862249487074852
>     amerikansk	0.027991174520283903
>     skådespelare	0.022262250791660873
>     politiker	0.015430182241441267
>     ==========
>     TOPIC 29
>     olympiska	0.023755348731854732
>     meter	0.020176054236135908
>     guld	0.018228241395018146
>     plats	0.014998717568131764
>     herrar	0.012198259818301941
>     ==========
>     TOPIC 30
>     universitet	0.01805314877885552
>     född	0.013481544995189861
>     professor	0.012079215089885553
>     ledamot	0.011259797873311078
>     uppsala	0.010672282286260678
>     ==========
>     TOPIC 31
>     bosnien	0.10447372402735176
>     hercegovina	0.09695325932564189
>     federationen	0.03459700308233657
>     kulle	0.028370705755756644
>     republika	0.02493111812949939
>     ==========
>     TOPIC 32
>     företaget	0.009569167419331554
>     flygplan	0.0071050551840972
>     motor	0.005528217027909177
>     moll	0.00525733000491023
>     modellen	0.004659204820748785
>     ==========
>     TOPIC 33
>     taung	0.045817120411665344
>     barrage	0.03254208587767372
>     buntu	0.029632272074815145
>     omgången	0.02827867170768587
>     herrarnas	0.01639467086457866
>     ==========
>     TOPIC 34
>     januari	0.012362892953283203
>     februari	0.012334199975737302
>     mars	0.011563682869594232
>     december	0.01072995324382497
>     programmet	0.008327916437339029
>     ==========
>     TOPIC 35
>     castle	0.015461014895925804
>     cirka	0.01438616727808855
>     invånare	0.013585745962777604
>     öarna	0.012073701553225354
>     havet	0.009914954538117455
>     ==========
>     TOPIC 36
>     översättning	0.04906384267437765
>     författare	0.01844981022607237
>     roman	0.013242853842446398
>     bonnier	0.01164679731230208
>     svenska	0.010784594256947616
>     ==========
>     TOPIC 37
>     geonames	0.11913277442194449
>     county	0.1068740519361396
>     vattendrag	0.10073116068293635
>     creek	0.08612996755830332
>     kanada	0.02096991701410841
>     ==========
>     TOPIC 38
>     morne	0.04862130674694627
>     prince	0.012955210935212914
>     nationalpark	0.012097185039298429
>     haiti	0.011658647411780567
>     runt	0.011398066152273945
>     ==========
>     TOPIC 39
>     lake	0.11858698163965047
>     kanada	0.06075058933507229
>     geonames	0.04188735863269003
>     ontario	0.024933961017808128
>     county	0.023368281723564005
>     ==========
>     TOPIC 40
>     strauss	0.018511806206536377
>     league	0.018491874573148
>     division	0.010229130946065097
>     united	0.009923475795512782
>     premier	0.009427019233066956
>     ==========
>     TOPIC 41
>     svensk	0.00822465048589354
>     avsnitt	0.007279394538684116
>     filmen	0.007140796703834101
>     anders	0.0067471563229271215
>     andersson	0.006729316593919472
>     ==========
>     TOPIC 42
>     trakten	0.044057251546186484
>     runt	0.033629229480123694
>     meter	0.03130301649642094
>     månaden	0.03091824560881027
>     havet	0.03067893992134706
>     ==========
>     TOPIC 43
>     cheshmeh	0.037308579989265865
>     runt	0.03702054761922084
>     trakten	0.0325624055866806
>     iran	0.026139559696209277
>     teheran	0.02438514776341585
>     ==========
>     TOPIC 44
>     född	0.009875959165704832
>     mars	0.009217054727638114
>     juni	0.009130097490158934
>     oktober	0.008858377271573493
>     september	0.008727261943084602
>     ==========
>     TOPIC 45
>     säsongen	0.020736851111366656
>     spelade	0.01633330545811541
>     matcher	0.015635380079718257
>     spelare	0.01289782084286368
>     vann	0.01279265489050279
>     ==========
>     TOPIC 46
>     from	0.0786507643327494
>     shift	0.06423691443896841
>     small	0.05705289322090071
>     value	0.04409509137084651
>     linegrey	0.031785742083129814
>     ==========
>     TOPIC 47
>     cerro	0.24746595175338434
>     berg	0.07094748980724597
>     kulle	0.03736557629148667
>     loma	0.02492131478963847
>     mexiko	0.017170592223449976
>     ==========
>     TOPIC 48
>     saint	0.10011429904845326
>     gogae	0.05631896060381116
>     dahr	0.029576990277144867
>     wadi	0.026338775967562476
>     sparbank	0.011919538108387324
>     ==========
>     TOPIC 49
>     bibcode	0.02090893858307959
>     star	0.015297563921716708
>     solen	0.015181805530522997
>     stars	0.013835395518137523
>     gånger	0.013160601333366323
>     ==========
>     topicIndices: Array[(Array[Int], Array[Double])] = Array((Array(910, 901, 510, 1197, 1761),Array(0.025059308870353184, 0.023400523968226696, 0.021195980743513396, 0.01626841008683287, 0.011214220970687016)), (Array(86, 146, 84, 196, 79),Array(0.03489516880955898, 0.025767746354246373, 0.016433187014153725, 0.014906573557897212, 0.010856865738603862)), (Array(651, 1700, 856, 859, 482),Array(0.10405263400187738, 0.03358165905918105, 0.03229756279665303, 0.032173363022884996, 0.02460256231682459)), (Array(50, 78, 69, 9, 85),Array(0.0057425403481240886, 0.00496435167053226, 0.004085380412935543, 0.0037827665048466796, 0.003671126083304182)), (Array(301, 300, 330, 336, 283),Array(0.010721479677025468, 0.010469511952728851, 0.008977552368263735, 0.008915902027876533, 0.007890189722208438)), (Array(128, 91, 136, 189, 127),Array(0.016560445257658117, 0.011293495672173312, 0.010506676818781705, 0.008449219540389546, 0.007499141959420526)), (Array(245, 2, 7, 3, 2436),Array(0.0542557984447277, 0.040408714303919666, 0.031171475853542692, 0.030511596184153392, 0.023540508986777616)), (Array(31, 51, 117, 92, 42),Array(0.15211150313293795, 0.04989770661531165, 0.049563866073212975, 0.04549484447553375, 0.03270669790929784)), (Array(31, 51, 345, 269, 219),Array(0.09255690838138343, 0.04920060501893783, 0.045482792389492585, 0.04374604159360783, 0.03574394980068257)), (Array(323, 1, 0, 2, 3),Array(0.07927038318689555, 0.028093246774054958, 0.02280181919266087, 0.021026495614210137, 0.018748571452188615)), (Array(185, 310, 503, 677, 264),Array(0.05751325153735175, 0.03443085235167512, 0.019601654340390597, 0.016781617645777836, 0.01658381820384268)), (Array(197, 188, 50, 143, 926),Array(0.026932065720601476, 0.022371972152705526, 0.0149631243059905, 0.008417565189216571, 0.006873690561937945)), (Array(0, 1, 2, 3, 4),Array(0.04504264786028259, 0.037326055844557414, 0.03257595419588929, 0.029718365953605424, 0.029331310988027017)), (Array(69, 84, 78, 70, 195),Array(0.010213734058180836, 0.0071866451286654335, 0.006683540279448935, 0.005883089788623208, 0.0054395801833071545)), (Array(213, 204, 403, 173, 183),Array(0.01740744406565029, 0.013162308863941057, 0.012092680452178768, 0.011448032616412316, 0.009096444292483394)), (Array(307, 70, 85, 291, 50),Array(0.00854504531365484, 0.006693592306258885, 0.005373850640154544, 0.005045685049475926, 0.004713423047922566)), (Array(528, 85, 107, 67, 91),Array(0.005652390811371258, 0.004741662061397474, 0.004514445765318061, 0.004203273591058431, 0.004084683065638065)), (Array(277, 484, 0, 2216, 1520),Array(0.06084841209854684, 0.051127666322883146, 0.01853132239855695, 0.01773031736821437, 0.016088442450282316)), (Array(1528, 3445, 3928, 2437, 1521),Array(0.04045624440182511, 0.034005650512617336, 0.02535678915947196, 0.023028259444303992, 0.01929037364979745)), (Array(522, 51, 483, 936, 31),Array(0.09760147281027223, 0.09595713187524844, 0.049993025274011706, 0.03046714380890495, 0.020460914651187472)), (Array(149, 0, 2, 3, 4),Array(0.04450632653283522, 0.03192872030447543, 0.025280264220534414, 0.024144452977865827, 0.02158435649122772)), (Array(50, 147, 111, 104, 143),Array(0.018287793328550822, 0.011660261152121208, 0.0065906421664898955, 0.00635955956012892, 0.006036999940163723)), (Array(2157, 4331, 8749, 6652, 5430),Array(0.04725750825084046, 0.023433361090418693, 0.01323259071372079, 0.013092630222814863, 0.011284949984719083)), (Array(730, 904, 0, 363, 2),Array(0.0740933767994304, 0.02902821451152532, 0.01688992637013919, 0.015069746721666782, 0.013077685991533665)), (Array(1006, 1427, 298, 1655, 50),Array(0.011875757162670324, 0.010646493174694176, 0.009891987765354214, 0.008193162135398413, 0.005456629205741401)), (Array(0, 2, 1, 4, 5),Array(0.04598539439861342, 0.03443065132806134, 0.03258287132483066, 0.030395182408479566, 0.030354929698208518)), (Array(352, 204, 225, 557, 125),Array(0.023879395302562268, 0.02002733234360079, 0.018538815765005035, 0.016183152904712696, 0.014904949179581357)), (Array(44, 102, 46, 7, 1),Array(0.19825300837251528, 0.03267677751256385, 0.031372696745660296, 0.019441278449670286, 0.0192495821975038)), (Array(79, 70, 144, 206, 242),Array(0.029605227787081547, 0.02862249487074852, 0.027991174520283903, 0.022262250791660873, 0.015430182241441267)), (Array(645, 2, 429, 142, 1041),Array(0.023755348731854732, 0.020176054236135908, 0.018228241395018146, 0.014998717568131764, 0.012198259818301941)), (Array(261, 70, 343, 367, 244),Array(0.01805314877885552, 0.013481544995189861, 0.012079215089885553, 0.011259797873311078, 0.010672282286260678)), (Array(271, 302, 931, 83, 1332),Array(0.10447372402735176, 0.09695325932564189, 0.03459700308233657, 0.028370705755756644, 0.02493111812949939)), (Array(487, 1281, 1259, 2066, 1783),Array(0.009569167419331554, 0.0071050551840972, 0.005528217027909177, 0.00525733000491023, 0.004659204820748785)), (Array(1652, 2237, 2591, 1091, 1551),Array(0.045817120411665344, 0.03254208587767372, 0.029632272074815145, 0.02827867170768587, 0.01639467086457866)), (Array(11, 47, 68, 54, 772),Array(0.012362892953283203, 0.012334199975737302, 0.011563682869594232, 0.01072995324382497, 0.008327916437339029)), (Array(1594, 169, 6, 1252, 3),Array(0.015461014895925804, 0.01438616727808855, 0.013585745962777604, 0.012073701553225354, 0.009914954538117455)), (Array(312, 203, 750, 1550, 69),Array(0.04906384267437765, 0.01844981022607237, 0.013242853842446398, 0.01164679731230208, 0.010784594256947616)), (Array(31, 95, 51, 121, 71),Array(0.11913277442194449, 0.1068740519361396, 0.10073116068293635, 0.08612996755830332, 0.02096991701410841)), (Array(1518, 2624, 3194, 5522, 0),Array(0.04862130674694627, 0.012955210935212914, 0.012097185039298429, 0.011658647411780567, 0.011398066152273945)), (Array(44, 71, 31, 238, 95),Array(0.11858698163965047, 0.06075058933507229, 0.04188735863269003, 0.024933961017808128, 0.023368281723564005)), (Array(1535, 520, 437, 1012, 2550),Array(0.018511806206536377, 0.018491874573148, 0.010229130946065097, 0.009923475795512782, 0.009427019233066956)), (Array(79, 559, 185, 324, 439),Array(0.00822465048589354, 0.007279394538684116, 0.007140796703834101, 0.0067471563229271215, 0.006729316593919472)), (Array(1, 0, 2, 4, 3),Array(0.044057251546186484, 0.033629229480123694, 0.03130301649642094, 0.03091824560881027, 0.03067893992134706)), (Array(166, 0, 1, 145, 150),Array(0.037308579989265865, 0.03702054761922084, 0.0325624055866806, 0.026139559696209277, 0.02438514776341585)), (Array(70, 68, 48, 80, 62),Array(0.009875959165704832, 0.009217054727638114, 0.009130097490158934, 0.008858377271573493, 0.008727261943084602)), (Array(191, 161, 327, 335, 152),Array(0.020736851111366656, 0.01633330545811541, 0.015635380079718257, 0.01289782084286368, 0.01279265489050279)), (Array(82, 105, 98, 130, 172),Array(0.0786507643327494, 0.06423691443896841, 0.05705289322090071, 0.04409509137084651, 0.031785742083129814)), (Array(39, 42, 83, 221, 92),Array(0.24746595175338434, 0.07094748980724597, 0.03736557629148667, 0.02492131478963847, 0.017170592223449976)), (Array(216, 2423, 2426, 4993, 8020),Array(0.10011429904845326, 0.05631896060381116, 0.029576990277144867, 0.026338775967562476, 0.011919538108387324)), (Array(1665, 841, 1078, 1703, 259),Array(0.02090893858307959, 0.015297563921716708, 0.015181805530522997, 0.013835395518137523, 0.013160601333366323)))
>     vocabList: Array[String] = Array(runt, trakten, meter, havet, månaden, nederbörd, invånare, kvadratkilometer, terrängen, landet, närheten, januari, huvudstaden, medeltemperaturen, varmaste, kallaste, årsmedeltemperaturen, högsta, kommentarer, nasalandcover, nasapop, millimeter, genomsnitt, genomsnittlig, torraste, årsnederbörd, regnigaste, nasarain, punkten, juli, koppen, geonames, större, omgivningarna, kuperad, augusti, råder, huvudsak, närmaste, cerro, samhälle, provinsen, berg, toppen, lake, ganska, kilometer, februari, juni, öster, talet, vattendrag, nordväst, nordost, december, platt, söder, väster, norr, glesbefolkat, växer, sydväst, september, huvudsakligen, tätbefolkat, sydost, november, mycket, mars, svenska, född, kanada, ingår, består, nära, mindre, april, centrala, sverige, svensk, oktober, samma, from, kulle, stockholm, senare, kommun, antalet, största, omgivande, basen, olika, mexiko, bredden, norra, county, sydöstra, bergig, small, klimatzonen, jordbruksmark, kommunen, riktning, tidigare, stora, shift, södra, många, regionen, obefolkad, blandskog, området, följande, tillsammans, ottawa, växtlighet, lite, robotskapade, hemiboreala, östra, delstaten, creek, namn, västra, enligt, läst, namnet, ofta, math, började, value, sveriges, öppet, city, stor, countyt, exempel, stäppklimat, nord, början, busklandskap, kallt, plats, gallery, amerikansk, iran, socken, staden, mest, gunung, teheran, historia, vann, personer, gräsmarker, persiska, mexico, därefter, sträcker, grund, arean, spelade, endast, platser, medan, frankrike, cheshmeh, lövskog, carl, cirka, västlig, sydlig, linegrey, förekommer, norrut, övrigt, cobar, åren, john, först, gamla, söderut, klimatet, familjen, ökenklimat, filmen, idag, naturlig, kyrka, används, vissa, säsongen, mosaik, nordvästra, johan, barn, församling, kyrkan, antal, österut, delar, västerut, tropiskt, författare, iucn, bort, skådespelare, medelhavsklimat, slutet, göteborg, samhällen, omkring, nordöstra, arten, städsegrön, stockholms, saint, tiden, tyskland, indonesien, ofruktbar, loma, karl, tyska, savannklimat, world, distriktet, skrev, kallas, känd, band, främst, erik, stort, bästa, gången, finland, departementet, ontario, width, gray, sluttar, politiker, periodiskt, uppsala, sjön, istället, samtidigt, amerikanska, nuvarande, serien, york, ordningen, timeline, malmö, london, engelska, plotdata, gift, gånger, biografi, universitet, egen, franska, regi, tillbaka, file, trots, version, ryssland, island, bosnien, byggdes, peter, äldre, black, kring, santa, form, samband, kung, hett, deltog, sång, musik, maria, dessutom, därmed, slut, chile, europa, paris, jean, ytterligare, storbritannien, kallade, lång, land, sida, kända, album, albumet, hercegovina, sidan, miljoner, enda, medlemmar, verk, gruppen, jabal, film, period, översättning, england, övriga, jakarta, dotter, måste, gustaf, sydvästra, ledde, inlandsklimat, ordförande, mountain, anders, längre, norge, matcher, spanien, fortfarande, låten, savannskog, stad, flyttade, världen, spelare, bandet, källa, särskilt, lars, grupp, david, numera, professor, aldrig, brasilien, kriget, anna, varierad, bildades, visar, länder, birds, australien, världskriget, läggs, karriär, district, fredrik, provincia, tall, tysk, totalt, marocko, kort, pass, syfta, ledamot, park, tempererat, indien, urval, ryska, nytt, poäng, tillhör, notera, danmark, exempelvis, kartan, göteborgs, ändras, mount, internationella, arbetade, användes, puerto, stod, rabat, museum, släpptes, kallad, platserna, innehåller, university, robert, varav, människor, start, martin, color, nästan, förbli, arter, gitarr, såsom, dagens, höjd, världens, bergstopp, ungefär, liksom, rubrik, österrike, studerade, boreala, italien, korrigeras, mallen, rättvisande, ursprungligen, brittiska, paul, egna, slutade, säsong, kvinnor, mitten, klubben, guld, fortsatte, thomas, fuktigt, bild, boken, subtropiskt, lämnade, division, ville, andersson, djebel, språk, gifte, roll, kungliga, year, lika, förgreningar, huset, laget, medlem, population, spela, oftast, serie, liten, politiska, nils, arbete, högre, gustav, långa, förutom, afghanistan, procent, verksamhet, kina, william, james, regeringen, bergspass, spelades, kantonen, berlin, studier, själva, områden, punta, president, lyckades, british, brittisk, argentina, colombia, alto, bror, befolkningsutveckling, företaget, michael, vatten, love, lever, historik, lund, gora, europeiska, regnskogsklimat, republiken, chef, airport, sjöarna, spelar, varmt, landskommun, varandra, utbredning, innebär, släktet, respektive, rätt, grand, uppfördes, officiella, anses, sven, årets, använda, förlorade, skolan, område, league, henrik, quebrada, direkt, wien, columbia, santiago, japan, spelet, grundades, vita, slaget, dels, late, hill, titeln, george, little, referens, hand, kabul, böcker, international, familj, fransk, johansson, fontsize, startade, increment, olof, modern, radio, lilla, spng, https, gridcolor, stöd, clements, slott, avsnitt, trummor, användas, bottom, magnus, udde, minst, liknande, height, ghar, colors, vertical, använder, orientation, structure, ålder, föddes, svenskt, enwiki, längs, nilsson, imagesize, timeaxis, plotarea, scalemajor, dateformat, helsingfors, alignbars, yyyy, tillhörde, textcolor, bilder, talets, monsunklimat, långt, danska, uppdrag, volution, äldsta, svart, avled, arrondissementet, kungen, visade, tidigt, tidningen, sommaren, grande, insee, chiffres, togs, hösten, alltid, vanligen, madrid, problem, börjar, samtliga, länge, bestod, kronor, system, sankt, richard, charles, spel, låtar, gavs, uppe, flertal, pasir, troligen, south, moderna, åter, musiker, jawa, bibliografi, förlag, byggnaden, delas, axel, enbart, utgör, lagen, gammal, olympiska, ingick, företag, centrum, utveckling, omfattande, laguna, eget, samhället, växte, rock, person, sett, nådde, betydelse, partiet, kallades, north, grekland, nere, gunnar, gjordes, villa, strax, life, viss, live, resultat, stadens, kalifornien, utseende, medverkade, filmer, ordning, jebel, bukit, ledare, betyder, status, marie, handling, silver, sångare, ursprungliga, geografi, vägen, såväl, snabbt, röda, släkte, konst, gård, uppgick, with, ihop, återvände, vapen, administrativ, brdo, bättre, washington, skedde, adolf, underarter, valet, larsson, valde, sarajevo, utbildning, ursprung, national, födda, records, historiska, månader, norska, platsen, ordet, listan, algeriet, general, örebro, höga, verksam, finalen, koudiat, spår, program, kustklimat, ägde, arabiska, august, schweiz, polen, bygga, museet, final, lärare, beroende, handlar, tekniska, byggnader, dåvarande, allra, utmärkelser, roman, delhi, saguenay, match, starkt, delvis, daniel, livet, länet, lappland, press, finska, djur, sten, gränsen, fotboll, följd, sker, samarbete, ekonomiska, föreningen, räknas, programmet, arkitekt, sonora, stället, phou, bakgrund, yngre, stil, härad, använde, främsta, bengt, bildade, cruz, juan, just, konstnär, wilhelm, svarta, emellertid, våren, unga, west, alger, colorado, singel, alexander, river, tjänst, dansk, litteratur, innebar, louis, reservoar, smith, music, landets, pris, mont, skapa, brukar, skillnad, viktig, nordiska, målare, lokala, emot, däremot, lista, mängd, bodde, kultur, hustru, drottning, lades, bruk, spanska, entiteten, känt, priset, allmänna, förste, christian, ledning, chihuahua, befolkningen, krig, belägen, material, star, tidiga, följde, francisco, vanliga, grupper, piano, georg, medeltida, därpå, information, möjligt, tävlingen, arroyo, köpenhamn, aires, forskning, riket, buenos, johannes, italienska, vanligt, sydkorea, ungern, oscar, kroatien, peru, hölls, leder, henry, relativt, nyheter, albert, inklusive, egypten, beslut, ansåg, flyttades, spelat, grekiska, kompositör, jorden, höll, petit, försökte, slog, nova, militär, försök, nederländerna, guvernementet, fotbollsspelare, premiär, teater, valdes, samman, wood, barrskog, diskografi, verksamheten, ford, bong, förening, koudyet, socknen, städer, eriksson, denne, group, länsväg, omfattar, skola, utsågs, kalmar, plan, morro, senaste, produktion, olsson, norsk, gården, exemplar, sovjetunionen, arbetet, grundade, kyrkans, hall, texas, kvinna, tour, federationen, björn, vanlig, hålla, matchen, venezuela, orgel, utnämndes, american, röst, ersattes, staten, församlingen, skåne, distrikt, nationella, beskrevs, jesus, marken, skrivit, dennes, bytte, öppna, umeå, debuterade, skriven, antingen, folk, invigdes, turn, resa, svårt, kontrakt, massa, byggde, förslag, engelsk, kvinnliga, mode, savann, läns, bygger, ansågs, skriva, aktiv, lägre, eric, statens, såldes, tagit, monte, befolkning, tilldelades, skrevs, färg, levande, perioden, egenskaper, royal, kritik, arbeta, lämna, sullivan, stift, inre, sändes, journalist, barcelona, bulu)
>     topics: Array[Array[(String, Double)]] = Array(Array((länsväg,0.025059308870353184), (ford,0.023400523968226696), (grand,0.021195980743513396), (prix,0.01626841008683287), (loppet,0.011214220970687016)), Array((kommun,0.03489516880955898), (socken,0.025767746354246373), (stockholm,0.016433187014153725), (församling,0.014906573557897212), (svensk,0.010856865738603862)), Array((laguna,0.10405263400187738), (lago,0.03358165905918105), (aires,0.03229756279665303), (buenos,0.032173363022884996), (argentina,0.02460256231682459)), Array((talet,0.0057425403481240886), (sverige,0.00496435167053226), (svenska,0.004085380412935543), (landet,0.0037827665048466796), (senare,0.003671126083304182)), Array((albumet,0.010721479677025468), (album,0.010469511952728851), (låten,0.008977552368263735), (bandet,0.008915902027876533), (sång,0.007890189722208438)), Array((math,0.016560445257658117), (olika,0.011293495672173312), (exempel,0.010506676818781705), (används,0.008449219540389546), (ofta,0.007499141959420526)), Array((sjön,0.0542557984447277), (meter,0.040408714303919666), (kvadratkilometer,0.031171475853542692), (havet,0.030511596184153392), (avrinningsområdet,0.023540508986777616)), Array((geonames,0.15211150313293795), (vattendrag,0.04989770661531165), (robotskapade,0.049563866073212975), (mexiko,0.04549484447553375), (berg,0.03270669790929784)), Array((geonames,0.09255690838138343), (vattendrag,0.04920060501893783), (brasilien,0.045482792389492585), (ryssland,0.04374604159360783), (indonesien,0.03574394980068257)), Array((mountain,0.07927038318689555), (trakten,0.028093246774054958), (runt,0.02280181919266087), (meter,0.021026495614210137), (havet,0.018748571452188615)), Array((filmen,0.05751325153735175), (film,0.03443085235167512), (landskommun,0.019601654340390597), (filmer,0.016781617645777836), (regi,0.01658381820384268)), Array((kyrkan,0.026932065720601476), (kyrka,0.022371972152705526), (talet,0.0149631243059905), (gallery,0.008417565189216571), (kyrkans,0.006873690561937945)), Array((runt,0.04504264786028259), (trakten,0.037326055844557414), (meter,0.03257595419588929), (havet,0.029718365953605424), (månaden,0.029331310988027017)), Array((svenska,0.010213734058180836), (stockholm,0.0071866451286654335), (sverige,0.006683540279448935), (född,0.005883089788623208), (barn,0.0054395801833071545)), Array((arten,0.01740744406565029), (iucn,0.013162308863941057), (arter,0.012092680452178768), (förekommer,0.011448032616412316), (familjen,0.009096444292483394)), Array((verk,0.00854504531365484), (född,0.006693592306258885), (senare,0.005373850640154544), (paris,0.005045685049475926), (talet,0.004713423047922566)), Array((spelet,0.005652390811371258), (senare,0.004741662061397474), (många,0.004514445765318061), (mycket,0.004203273591058431), (olika,0.004084683065638065)), Array((santa,0.06084841209854684), (alto,0.051127666322883146), (runt,0.01853132239855695), (koudiet,0.01773031736821437), (mali,0.016088442450282316)), Array((naturreservat,0.04045624440182511), (lagoa,0.034005650512617336), (bond,0.02535678915947196), (beach,0.023028259444303992), (strand,0.01929037364979745)), Array((quebrada,0.09760147281027223), (vattendrag,0.09595713187524844), (colombia,0.049993025274011706), (venezuela,0.03046714380890495), (geonames,0.020460914651187472)), Array((gunung,0.04450632653283522), (runt,0.03192872030447543), (meter,0.025280264220534414), (havet,0.024144452977865827), (månaden,0.02158435649122772)), Array((talet,0.018287793328550822), (staden,0.011660261152121208), (området,0.0065906421664898955), (stora,0.00635955956012892), (gallery,0.006036999940163723)), Array((tubu,0.04725750825084046), (qimmat,0.023433361090418693), (drosophila,0.01323259071372079), (seco,0.013092630222814863), (male,0.011284949984719083)), Array((koudiat,0.0740933767994304), (koudyet,0.02902821451152532), (runt,0.01688992637013919), (marocko,0.015069746721666782), (meter,0.013077685991533665)), Array((fartyg,0.011875757162670324), (fartyget,0.010646493174694176), (sida,0.009891987765354214), (hästar,0.008193162135398413), (talet,0.005456629205741401)), Array((runt,0.04598539439861342), (meter,0.03443065132806134), (trakten,0.03258287132483066), (månaden,0.030395182408479566), (nederbörd,0.030354929698208518)), Array((birds,0.023879395302562268), (iucn,0.02002733234360079), (world,0.018538815765005035), (clements,0.016183152904712696), (läst,0.014904949179581357)), Array((lake,0.19825300837251528), (riktning,0.03267677751256385), (kilometer,0.031372696745660296), (kvadratkilometer,0.019441278449670286), (trakten,0.0192495821975038)), Array((svensk,0.029605227787081547), (född,0.02862249487074852), (amerikansk,0.027991174520283903), (skådespelare,0.022262250791660873), (politiker,0.015430182241441267)), Array((olympiska,0.023755348731854732), (meter,0.020176054236135908), (guld,0.018228241395018146), (plats,0.014998717568131764), (herrar,0.012198259818301941)), Array((universitet,0.01805314877885552), (född,0.013481544995189861), (professor,0.012079215089885553), (ledamot,0.011259797873311078), (uppsala,0.010672282286260678)), Array((bosnien,0.10447372402735176), (hercegovina,0.09695325932564189), (federationen,0.03459700308233657), (kulle,0.028370705755756644), (republika,0.02493111812949939)), Array((företaget,0.009569167419331554), (flygplan,0.0071050551840972), (motor,0.005528217027909177), (moll,0.00525733000491023), (modellen,0.004659204820748785)), Array((taung,0.045817120411665344), (barrage,0.03254208587767372), (buntu,0.029632272074815145), (omgången,0.02827867170768587), (herrarnas,0.01639467086457866)), Array((januari,0.012362892953283203), (februari,0.012334199975737302), (mars,0.011563682869594232), (december,0.01072995324382497), (programmet,0.008327916437339029)), Array((castle,0.015461014895925804), (cirka,0.01438616727808855), (invånare,0.013585745962777604), (öarna,0.012073701553225354), (havet,0.009914954538117455)), Array((översättning,0.04906384267437765), (författare,0.01844981022607237), (roman,0.013242853842446398), (bonnier,0.01164679731230208), (svenska,0.010784594256947616)), Array((geonames,0.11913277442194449), (county,0.1068740519361396), (vattendrag,0.10073116068293635), (creek,0.08612996755830332), (kanada,0.02096991701410841)), Array((morne,0.04862130674694627), (prince,0.012955210935212914), (nationalpark,0.012097185039298429), (haiti,0.011658647411780567), (runt,0.011398066152273945)), Array((lake,0.11858698163965047), (kanada,0.06075058933507229), (geonames,0.04188735863269003), (ontario,0.024933961017808128), (county,0.023368281723564005)), Array((strauss,0.018511806206536377), (league,0.018491874573148), (division,0.010229130946065097), (united,0.009923475795512782), (premier,0.009427019233066956)), Array((svensk,0.00822465048589354), (avsnitt,0.007279394538684116), (filmen,0.007140796703834101), (anders,0.0067471563229271215), (andersson,0.006729316593919472)), Array((trakten,0.044057251546186484), (runt,0.033629229480123694), (meter,0.03130301649642094), (månaden,0.03091824560881027), (havet,0.03067893992134706)), Array((cheshmeh,0.037308579989265865), (runt,0.03702054761922084), (trakten,0.0325624055866806), (iran,0.026139559696209277), (teheran,0.02438514776341585)), Array((född,0.009875959165704832), (mars,0.009217054727638114), (juni,0.009130097490158934), (oktober,0.008858377271573493), (september,0.008727261943084602)), Array((säsongen,0.020736851111366656), (spelade,0.01633330545811541), (matcher,0.015635380079718257), (spelare,0.01289782084286368), (vann,0.01279265489050279)), Array((from,0.0786507643327494), (shift,0.06423691443896841), (small,0.05705289322090071), (value,0.04409509137084651), (linegrey,0.031785742083129814)), Array((cerro,0.24746595175338434), (berg,0.07094748980724597), (kulle,0.03736557629148667), (loma,0.02492131478963847), (mexiko,0.017170592223449976)), Array((saint,0.10011429904845326), (gogae,0.05631896060381116), (dahr,0.029576990277144867), (wadi,0.026338775967562476), (sparbank,0.011919538108387324)), Array((bibcode,0.02090893858307959), (star,0.015297563921716708), (solen,0.015181805530522997), (stars,0.013835395518137523), (gånger,0.013160601333366323)))

In [ ]:
topics.zipWithIndex.filter({ case m => m._2 == 1944}).foreach { case (topic, i) =>
  println(s"TOPIC $i")
  topic.foreach { case (term, weight) => println(s"$term\t$weight") }
  println(s"==========")
}

  

  

Visualize Results
=================

In [ ]:
// Zip topic terms with topic IDs
val termArray = topics.zipWithIndex

  

>     termArray: Array[(Array[(String, Double)], Int)] = Array((Array((länsväg,0.025059308870353184), (ford,0.023400523968226696), (grand,0.021195980743513396), (prix,0.01626841008683287), (loppet,0.011214220970687016)),0), (Array((kommun,0.03489516880955898), (socken,0.025767746354246373), (stockholm,0.016433187014153725), (församling,0.014906573557897212), (svensk,0.010856865738603862)),1), (Array((laguna,0.10405263400187738), (lago,0.03358165905918105), (aires,0.03229756279665303), (buenos,0.032173363022884996), (argentina,0.02460256231682459)),2), (Array((talet,0.0057425403481240886), (sverige,0.00496435167053226), (svenska,0.004085380412935543), (landet,0.0037827665048466796), (senare,0.003671126083304182)),3), (Array((albumet,0.010721479677025468), (album,0.010469511952728851), (låten,0.008977552368263735), (bandet,0.008915902027876533), (sång,0.007890189722208438)),4), (Array((math,0.016560445257658117), (olika,0.011293495672173312), (exempel,0.010506676818781705), (används,0.008449219540389546), (ofta,0.007499141959420526)),5), (Array((sjön,0.0542557984447277), (meter,0.040408714303919666), (kvadratkilometer,0.031171475853542692), (havet,0.030511596184153392), (avrinningsområdet,0.023540508986777616)),6), (Array((geonames,0.15211150313293795), (vattendrag,0.04989770661531165), (robotskapade,0.049563866073212975), (mexiko,0.04549484447553375), (berg,0.03270669790929784)),7), (Array((geonames,0.09255690838138343), (vattendrag,0.04920060501893783), (brasilien,0.045482792389492585), (ryssland,0.04374604159360783), (indonesien,0.03574394980068257)),8), (Array((mountain,0.07927038318689555), (trakten,0.028093246774054958), (runt,0.02280181919266087), (meter,0.021026495614210137), (havet,0.018748571452188615)),9), (Array((filmen,0.05751325153735175), (film,0.03443085235167512), (landskommun,0.019601654340390597), (filmer,0.016781617645777836), (regi,0.01658381820384268)),10), (Array((kyrkan,0.026932065720601476), (kyrka,0.022371972152705526), (talet,0.0149631243059905), (gallery,0.008417565189216571), (kyrkans,0.006873690561937945)),11), (Array((runt,0.04504264786028259), (trakten,0.037326055844557414), (meter,0.03257595419588929), (havet,0.029718365953605424), (månaden,0.029331310988027017)),12), (Array((svenska,0.010213734058180836), (stockholm,0.0071866451286654335), (sverige,0.006683540279448935), (född,0.005883089788623208), (barn,0.0054395801833071545)),13), (Array((arten,0.01740744406565029), (iucn,0.013162308863941057), (arter,0.012092680452178768), (förekommer,0.011448032616412316), (familjen,0.009096444292483394)),14), (Array((verk,0.00854504531365484), (född,0.006693592306258885), (senare,0.005373850640154544), (paris,0.005045685049475926), (talet,0.004713423047922566)),15), (Array((spelet,0.005652390811371258), (senare,0.004741662061397474), (många,0.004514445765318061), (mycket,0.004203273591058431), (olika,0.004084683065638065)),16), (Array((santa,0.06084841209854684), (alto,0.051127666322883146), (runt,0.01853132239855695), (koudiet,0.01773031736821437), (mali,0.016088442450282316)),17), (Array((naturreservat,0.04045624440182511), (lagoa,0.034005650512617336), (bond,0.02535678915947196), (beach,0.023028259444303992), (strand,0.01929037364979745)),18), (Array((quebrada,0.09760147281027223), (vattendrag,0.09595713187524844), (colombia,0.049993025274011706), (venezuela,0.03046714380890495), (geonames,0.020460914651187472)),19), (Array((gunung,0.04450632653283522), (runt,0.03192872030447543), (meter,0.025280264220534414), (havet,0.024144452977865827), (månaden,0.02158435649122772)),20), (Array((talet,0.018287793328550822), (staden,0.011660261152121208), (området,0.0065906421664898955), (stora,0.00635955956012892), (gallery,0.006036999940163723)),21), (Array((tubu,0.04725750825084046), (qimmat,0.023433361090418693), (drosophila,0.01323259071372079), (seco,0.013092630222814863), (male,0.011284949984719083)),22), (Array((koudiat,0.0740933767994304), (koudyet,0.02902821451152532), (runt,0.01688992637013919), (marocko,0.015069746721666782), (meter,0.013077685991533665)),23), (Array((fartyg,0.011875757162670324), (fartyget,0.010646493174694176), (sida,0.009891987765354214), (hästar,0.008193162135398413), (talet,0.005456629205741401)),24), (Array((runt,0.04598539439861342), (meter,0.03443065132806134), (trakten,0.03258287132483066), (månaden,0.030395182408479566), (nederbörd,0.030354929698208518)),25), (Array((birds,0.023879395302562268), (iucn,0.02002733234360079), (world,0.018538815765005035), (clements,0.016183152904712696), (läst,0.014904949179581357)),26), (Array((lake,0.19825300837251528), (riktning,0.03267677751256385), (kilometer,0.031372696745660296), (kvadratkilometer,0.019441278449670286), (trakten,0.0192495821975038)),27), (Array((svensk,0.029605227787081547), (född,0.02862249487074852), (amerikansk,0.027991174520283903), (skådespelare,0.022262250791660873), (politiker,0.015430182241441267)),28), (Array((olympiska,0.023755348731854732), (meter,0.020176054236135908), (guld,0.018228241395018146), (plats,0.014998717568131764), (herrar,0.012198259818301941)),29), (Array((universitet,0.01805314877885552), (född,0.013481544995189861), (professor,0.012079215089885553), (ledamot,0.011259797873311078), (uppsala,0.010672282286260678)),30), (Array((bosnien,0.10447372402735176), (hercegovina,0.09695325932564189), (federationen,0.03459700308233657), (kulle,0.028370705755756644), (republika,0.02493111812949939)),31), (Array((företaget,0.009569167419331554), (flygplan,0.0071050551840972), (motor,0.005528217027909177), (moll,0.00525733000491023), (modellen,0.004659204820748785)),32), (Array((taung,0.045817120411665344), (barrage,0.03254208587767372), (buntu,0.029632272074815145), (omgången,0.02827867170768587), (herrarnas,0.01639467086457866)),33), (Array((januari,0.012362892953283203), (februari,0.012334199975737302), (mars,0.011563682869594232), (december,0.01072995324382497), (programmet,0.008327916437339029)),34), (Array((castle,0.015461014895925804), (cirka,0.01438616727808855), (invånare,0.013585745962777604), (öarna,0.012073701553225354), (havet,0.009914954538117455)),35), (Array((översättning,0.04906384267437765), (författare,0.01844981022607237), (roman,0.013242853842446398), (bonnier,0.01164679731230208), (svenska,0.010784594256947616)),36), (Array((geonames,0.11913277442194449), (county,0.1068740519361396), (vattendrag,0.10073116068293635), (creek,0.08612996755830332), (kanada,0.02096991701410841)),37), (Array((morne,0.04862130674694627), (prince,0.012955210935212914), (nationalpark,0.012097185039298429), (haiti,0.011658647411780567), (runt,0.011398066152273945)),38), (Array((lake,0.11858698163965047), (kanada,0.06075058933507229), (geonames,0.04188735863269003), (ontario,0.024933961017808128), (county,0.023368281723564005)),39), (Array((strauss,0.018511806206536377), (league,0.018491874573148), (division,0.010229130946065097), (united,0.009923475795512782), (premier,0.009427019233066956)),40), (Array((svensk,0.00822465048589354), (avsnitt,0.007279394538684116), (filmen,0.007140796703834101), (anders,0.0067471563229271215), (andersson,0.006729316593919472)),41), (Array((trakten,0.044057251546186484), (runt,0.033629229480123694), (meter,0.03130301649642094), (månaden,0.03091824560881027), (havet,0.03067893992134706)),42), (Array((cheshmeh,0.037308579989265865), (runt,0.03702054761922084), (trakten,0.0325624055866806), (iran,0.026139559696209277), (teheran,0.02438514776341585)),43), (Array((född,0.009875959165704832), (mars,0.009217054727638114), (juni,0.009130097490158934), (oktober,0.008858377271573493), (september,0.008727261943084602)),44), (Array((säsongen,0.020736851111366656), (spelade,0.01633330545811541), (matcher,0.015635380079718257), (spelare,0.01289782084286368), (vann,0.01279265489050279)),45), (Array((from,0.0786507643327494), (shift,0.06423691443896841), (small,0.05705289322090071), (value,0.04409509137084651), (linegrey,0.031785742083129814)),46), (Array((cerro,0.24746595175338434), (berg,0.07094748980724597), (kulle,0.03736557629148667), (loma,0.02492131478963847), (mexiko,0.017170592223449976)),47), (Array((saint,0.10011429904845326), (gogae,0.05631896060381116), (dahr,0.029576990277144867), (wadi,0.026338775967562476), (sparbank,0.011919538108387324)),48), (Array((bibcode,0.02090893858307959), (star,0.015297563921716708), (solen,0.015181805530522997), (stars,0.013835395518137523), (gånger,0.013160601333366323)),49))

In [ ]:
// Transform data into the form (term, probability, topicId)
val termRDD = sc.parallelize(termArray)
val termRDD2 =termRDD.flatMap( (x: (Array[(String, Double)], Int)) => {
  val arrayOfTuple = x._1
  val topicId = x._2
  arrayOfTuple.map(el => (el._1, el._2, topicId))
})

  

>     termRDD: org.apache.spark.rdd.RDD[(Array[(String, Double)], Int)] = ParallelCollectionRDD[2658] at parallelize at command-738986526041674:2
>     termRDD2: org.apache.spark.rdd.RDD[(String, Double, Int)] = MapPartitionsRDD[2659] at flatMap at command-738986526041674:3

In [ ]:
// Create DF with proper column names
val termDF = termRDD2.toDF.withColumnRenamed("_1", "term").withColumnRenamed("_2", "probability").withColumnRenamed("_3", "topicId")

  

>     termDF: org.apache.spark.sql.DataFrame = [term: string, probability: double ... 1 more field]

In [ ]:
display(termDF)

  

[TABLE]

Truncated to 30 rows

In [ ]:
// Create JSON data
val rawJson = termDF.toJSON.collect().mkString(",\n")

  

>     rawJson: String =
>     {"term":"länsväg","probability":0.025059308870353184,"topicId":0},
>     {"term":"ford","probability":0.023400523968226696,"topicId":0},
>     {"term":"grand","probability":0.021195980743513396,"topicId":0},
>     {"term":"prix","probability":0.01626841008683287,"topicId":0},
>     {"term":"loppet","probability":0.011214220970687016,"topicId":0},
>     {"term":"kommun","probability":0.03489516880955898,"topicId":1},
>     {"term":"socken","probability":0.025767746354246373,"topicId":1},
>     {"term":"stockholm","probability":0.016433187014153725,"topicId":1},
>     {"term":"församling","probability":0.014906573557897212,"topicId":1},
>     {"term":"svensk","probability":0.010856865738603862,"topicId":1},
>     {"term":"laguna","probability":0.10405263400187738,"topicId":2},
>     {"term":"lago","probability":0.03358165905918105,"topicId":2},
>     {"term":"aires","probability":0.03229756279665303,"topicId":2},
>     {"term":"buenos","probability":0.032173363022884996,"topicId":2},
>     {"term":"argentina","probability":0.02460256231682459,"topicId":2},
>     {"term":"talet","probability":0.0057425403481240886,"topicId":3},
>     {"term":"sverige","probability":0.00496435167053226,"topicId":3},
>     {"term":"svenska","probability":0.004085380412935543,"topicId":3},
>     {"term":"landet","probability":0.0037827665048466796,"topicId":3},
>     {"term":"senare","probability":0.003671126083304182,"topicId":3},
>     {"term":"albumet","probability":0.010721479677025468,"topicId":4},
>     {"term":"album","probability":0.010469511952728851,"topicId":4},
>     {"term":"låten","probability":0.008977552368263735,"topicId":4},
>     {"term":"bandet","probability":0.008915902027876533,"topicId":4},
>     {"term":"sång","probability":0.007890189722208438,"topicId":4},
>     {"term":"math","probability":0.016560445257658117,"topicId":5},
>     {"term":"olika","probability":0.011293495672173312,"topicId":5},
>     {"term":"exempel","probability":0.010506676818781705,"topicId":5},
>     {"term":"används","probability":0.008449219540389546,"topicId":5},
>     {"term":"ofta","probability":0.007499141959420526,"topicId":5},
>     {"term":"sjön","probability":0.0542557984447277,"topicId":6},
>     {"term":"meter","probability":0.040408714303919666,"topicId":6},
>     {"term":"kvadratkilometer","probability":0.031171475853542692,"topicId":6},
>     {"term":"havet","probability":0.030511596184153392,"topicId":6},
>     {"term":"avrinningsområdet","probability":0.023540508986777616,"topicId":6},
>     {"term":"geonames","probability":0.15211150313293795,"topicId":7},
>     {"term":"vattendrag","probability":0.04989770661531165,"topicId":7},
>     {"term":"robotskapade","probability":0.049563866073212975,"topicId":7},
>     {"term":"mexiko","probability":0.04549484447553375,"topicId":7},
>     {"term":"berg","probability":0.03270669790929784,"topicId":7},
>     {"term":"geonames","probability":0.09255690838138343,"topicId":8},
>     {"term":"vattendrag","probability":0.04920060501893783,"topicId":8},
>     {"term":"brasilien","probability":0.045482792389492585,"topicId":8},
>     {"term":"ryssland","probability":0.04374604159360783,"topicId":8},
>     {"term":"indonesien","probability":0.03574394980068257,"topicId":8},
>     {"term":"mountain","probability":0.07927038318689555,"topicId":9},
>     {"term":"trakten","probability":0.028093246774054958,"topicId":9},
>     {"term":"runt","probability":0.02280181919266087,"topicId":9},
>     {"term":"meter","probability":0.021026495614210137,"topicId":9},
>     {"term":"havet","probability":0.018748571452188615,"topicId":9},
>     {"term":"filmen","probability":0.05751325153735175,"topicId":10},
>     {"term":"film","probability":0.03443085235167512,"topicId":10},
>     {"term":"landskommun","probability":0.019601654340390597,"topicId":10},
>     {"term":"filmer","probability":0.016781617645777836,"topicId":10},
>     {"term":"regi","probability":0.01658381820384268,"topicId":10},
>     {"term":"kyrkan","probability":0.026932065720601476,"topicId":11},
>     {"term":"kyrka","probability":0.022371972152705526,"topicId":11},
>     {"term":"talet","probability":0.0149631243059905,"topicId":11},
>     {"term":"gallery","probability":0.008417565189216571,"topicId":11},
>     {"term":"kyrkans","probability":0.006873690561937945,"topicId":11},
>     {"term":"runt","probability":0.04504264786028259,"topicId":12},
>     {"term":"trakten","probability":0.037326055844557414,"topicId":12},
>     {"term":"meter","probability":0.03257595419588929,"topicId":12},
>     {"term":"havet","probability":0.029718365953605424,"topicId":12},
>     {"term":"månaden","probability":0.029331310988027017,"topicId":12},
>     {"term":"svenska","probability":0.010213734058180836,"topicId":13},
>     {"term":"stockholm","probability":0.0071866451286654335,"topicId":13},
>     {"term":"sverige","probability":0.006683540279448935,"topicId":13},
>     {"term":"född","probability":0.005883089788623208,"topicId":13},
>     {"term":"barn","probability":0.0054395801833071545,"topicId":13},
>     {"term":"arten","probability":0.01740744406565029,"topicId":14},
>     {"term":"iucn","probability":0.013162308863941057,"topicId":14},
>     {"term":"arter","probability":0.012092680452178768,"topicId":14},
>     {"term":"förekommer","probability":0.011448032616412316,"topicId":14},
>     {"term":"familjen","probability":0.009096444292483394,"topicId":14},
>     {"term":"verk","probability":0.00854504531365484,"topicId":15},
>     {"term":"född","probability":0.006693592306258885,"topicId":15},
>     {"term":"senare","probability":0.005373850640154544,"topicId":15},
>     {"term":"paris","probability":0.005045685049475926,"topicId":15},
>     {"term":"talet","probability":0.004713423047922566,"topicId":15},
>     {"term":"spelet","probability":0.005652390811371258,"topicId":16},
>     {"term":"senare","probability":0.004741662061397474,"topicId":16},
>     {"term":"många","probability":0.004514445765318061,"topicId":16},
>     {"term":"mycket","probability":0.004203273591058431,"topicId":16},
>     {"term":"olika","probability":0.004084683065638065,"topicId":16},
>     {"term":"santa","probability":0.06084841209854684,"topicId":17},
>     {"term":"alto","probability":0.051127666322883146,"topicId":17},
>     {"term":"runt","probability":0.01853132239855695,"topicId":17},
>     {"term":"koudiet","probability":0.01773031736821437,"topicId":17},
>     {"term":"mali","probability":0.016088442450282316,"topicId":17},
>     {"term":"naturreservat","probability":0.04045624440182511,"topicId":18},
>     {"term":"lagoa","probability":0.034005650512617336,"topicId":18},
>     {"term":"bond","probability":0.02535678915947196,"topicId":18},
>     {"term":"beach","probability":0.023028259444303992,"topicId":18},
>     {"term":"strand","probability":0.01929037364979745,"topicId":18},
>     {"term":"quebrada","probability":0.09760147281027223,"topicId":19},
>     {"term":"vattendrag","probability":0.09595713187524844,"topicId":19},
>     {"term":"colombia","probability":0.049993025274011706,"topicId":19},
>     {"term":"venezuela","probability":0.03046714380890495,"topicId":19},
>     {"term":"geonames","probability":0.020460914651187472,"topicId":19},
>     {"term":"gunung","probability":0.04450632653283522,"topicId":20},
>     {"term":"runt","probability":0.03192872030447543,"topicId":20},
>     {"term":"meter","probability":0.025280264220534414,"topicId":20},
>     {"term":"havet","probability":0.024144452977865827,"topicId":20},
>     {"term":"månaden","probability":0.02158435649122772,"topicId":20},
>     {"term":"talet","probability":0.018287793328550822,"topicId":21},
>     {"term":"staden","probability":0.011660261152121208,"topicId":21},
>     {"term":"området","probability":0.0065906421664898955,"topicId":21},
>     {"term":"stora","probability":0.00635955956012892,"topicId":21},
>     {"term":"gallery","probability":0.006036999940163723,"topicId":21},
>     {"term":"tubu","probability":0.04725750825084046,"topicId":22},
>     {"term":"qimmat","probability":0.023433361090418693,"topicId":22},
>     {"term":"drosophila","probability":0.01323259071372079,"topicId":22},
>     {"term":"seco","probability":0.013092630222814863,"topicId":22},
>     {"term":"male","probability":0.011284949984719083,"topicId":22},
>     {"term":"koudiat","probability":0.0740933767994304,"topicId":23},
>     {"term":"koudyet","probability":0.02902821451152532,"topicId":23},
>     {"term":"runt","probability":0.01688992637013919,"topicId":23},
>     {"term":"marocko","probability":0.015069746721666782,"topicId":23},
>     {"term":"meter","probability":0.013077685991533665,"topicId":23},
>     {"term":"fartyg","probability":0.011875757162670324,"topicId":24},
>     {"term":"fartyget","probability":0.010646493174694176,"topicId":24},
>     {"term":"sida","probability":0.009891987765354214,"topicId":24},
>     {"term":"hästar","probability":0.008193162135398413,"topicId":24},
>     {"term":"talet","probability":0.005456629205741401,"topicId":24},
>     {"term":"runt","probability":0.04598539439861342,"topicId":25},
>     {"term":"meter","probability":0.03443065132806134,"topicId":25},
>     {"term":"trakten","probability":0.03258287132483066,"topicId":25},
>     {"term":"månaden","probability":0.030395182408479566,"topicId":25},
>     {"term":"nederbörd","probability":0.030354929698208518,"topicId":25},
>     {"term":"birds","probability":0.023879395302562268,"topicId":26},
>     {"term":"iucn","probability":0.02002733234360079,"topicId":26},
>     {"term":"world","probability":0.018538815765005035,"topicId":26},
>     {"term":"clements","probability":0.016183152904712696,"topicId":26},
>     {"term":"läst","probability":0.014904949179581357,"topicId":26},
>     {"term":"lake","probability":0.19825300837251528,"topicId":27},
>     {"term":"riktning","probability":0.03267677751256385,"topicId":27},
>     {"term":"kilometer","probability":0.031372696745660296,"topicId":27},
>     {"term":"kvadratkilometer","probability":0.019441278449670286,"topicId":27},
>     {"term":"trakten","probability":0.0192495821975038,"topicId":27},
>     {"term":"svensk","probability":0.029605227787081547,"topicId":28},
>     {"term":"född","probability":0.02862249487074852,"topicId":28},
>     {"term":"amerikansk","probability":0.027991174520283903,"topicId":28},
>     {"term":"skådespelare","probability":0.022262250791660873,"topicId":28},
>     {"term":"politiker","probability":0.015430182241441267,"topicId":28},
>     {"term":"olympiska","probability":0.023755348731854732,"topicId":29},
>     {"term":"meter","probability":0.020176054236135908,"topicId":29},
>     {"term":"guld","probability":0.018228241395018146,"topicId":29},
>     {"term":"plats","probability":0.014998717568131764,"topicId":29},
>     {"term":"herrar","probability":0.012198259818301941,"topicId":29},
>     {"term":"universitet","probability":0.01805314877885552,"topicId":30},
>     {"term":"född","probability":0.013481544995189861,"topicId":30},
>     {"term":"professor","probability":0.012079215089885553,"topicId":30},
>     {"term":"ledamot","probability":0.011259797873311078,"topicId":30},
>     {"term":"uppsala","probability":0.010672282286260678,"topicId":30},
>     {"term":"bosnien","probability":0.10447372402735176,"topicId":31},
>     {"term":"hercegovina","probability":0.09695325932564189,"topicId":31},
>     {"term":"federationen","probability":0.03459700308233657,"topicId":31},
>     {"term":"kulle","probability":0.028370705755756644,"topicId":31},
>     {"term":"republika","probability":0.02493111812949939,"topicId":31},
>     {"term":"företaget","probability":0.009569167419331554,"topicId":32},
>     {"term":"flygplan","probability":0.0071050551840972,"topicId":32},
>     {"term":"motor","probability":0.005528217027909177,"topicId":32},
>     {"term":"moll","probability":0.00525733000491023,"topicId":32},
>     {"term":"modellen","probability":0.004659204820748785,"topicId":32},
>     {"term":"taung","probability":0.045817120411665344,"topicId":33},
>     {"term":"barrage","probability":0.03254208587767372,"topicId":33},
>     {"term":"buntu","probability":0.029632272074815145,"topicId":33},
>     {"term":"omgången","probability":0.02827867170768587,"topicId":33},
>     {"term":"herrarnas","probability":0.01639467086457866,"topicId":33},
>     {"term":"januari","probability":0.012362892953283203,"topicId":34},
>     {"term":"februari","probability":0.012334199975737302,"topicId":34},
>     {"term":"mars","probability":0.011563682869594232,"topicId":34},
>     {"term":"december","probability":0.01072995324382497,"topicId":34},
>     {"term":"programmet","probability":0.008327916437339029,"topicId":34},
>     {"term":"castle","probability":0.015461014895925804,"topicId":35},
>     {"term":"cirka","probability":0.01438616727808855,"topicId":35},
>     {"term":"invånare","probability":0.013585745962777604,"topicId":35},
>     {"term":"öarna","probability":0.012073701553225354,"topicId":35},
>     {"term":"havet","probability":0.009914954538117455,"topicId":35},
>     {"term":"översättning","probability":0.04906384267437765,"topicId":36},
>     {"term":"författare","probability":0.01844981022607237,"topicId":36},
>     {"term":"roman","probability":0.013242853842446398,"topicId":36},
>     {"term":"bonnier","probability":0.01164679731230208,"topicId":36},
>     {"term":"svenska","probability":0.010784594256947616,"topicId":36},
>     {"term":"geonames","probability":0.11913277442194449,"topicId":37},
>     {"term":"county","probability":0.1068740519361396,"topicId":37},
>     {"term":"vattendrag","probability":0.10073116068293635,"topicId":37},
>     {"term":"creek","probability":0.08612996755830332,"topicId":37},
>     {"term":"kanada","probability":0.02096991701410841,"topicId":37},
>     {"term":"morne","probability":0.04862130674694627,"topicId":38},
>     {"term":"prince","probability":0.012955210935212914,"topicId":38},
>     {"term":"nationalpark","probability":0.012097185039298429,"topicId":38},
>     {"term":"haiti","probability":0.011658647411780567,"topicId":38},
>     {"term":"runt","probability":0.011398066152273945,"topicId":38},
>     {"term":"lake","probability":0.11858698163965047,"topicId":39},
>     {"term":"kanada","probability":0.06075058933507229,"topicId":39},
>     {"term":"geonames","probability":0.04188735863269003,"topicId":39},
>     {"term":"ontario","probability":0.024933961017808128,"topicId":39},
>     {"term":"county","probability":0.023368281723564005,"topicId":39},
>     {"term":"strauss","probability":0.018511806206536377,"topicId":40},
>     {"term":"league","probability":0.018491874573148,"topicId":40},
>     {"term":"division","probability":0.010229130946065097,"topicId":40},
>     {"term":"united","probability":0.009923475795512782,"topicId":40},
>     {"term":"premier","probability":0.009427019233066956,"topicId":40},
>     {"term":"svensk","probability":0.00822465048589354,"topicId":41},
>     {"term":"avsnitt","probability":0.007279394538684116,"topicId":41},
>     {"term":"filmen","probability":0.007140796703834101,"topicId":41},
>     {"term":"anders","probability":0.0067471563229271215,"topicId":41},
>     {"term":"andersson","probability":0.006729316593919472,"topicId":41},
>     {"term":"trakten","probability":0.044057251546186484,"topicId":42},
>     {"term":"runt","probability":0.033629229480123694,"topicId":42},
>     {"term":"meter","probability":0.03130301649642094,"topicId":42},
>     {"term":"månaden","probability":0.03091824560881027,"topicId":42},
>     {"term":"havet","probability":0.03067893992134706,"topicId":42},
>     {"term":"cheshmeh","probability":0.037308579989265865,"topicId":43},
>     {"term":"runt","probability":0.03702054761922084,"topicId":43},
>     {"term":"trakten","probability":0.0325624055866806,"topicId":43},
>     {"term":"iran","probability":0.026139559696209277,"topicId":43},
>     {"term":"teheran","probability":0.02438514776341585,"topicId":43},
>     {"term":"född","probability":0.009875959165704832,"topicId":44},
>     {"term":"mars","probability":0.009217054727638114,"topicId":44},
>     {"term":"juni","probability":0.009130097490158934,"topicId":44},
>     {"term":"oktober","probability":0.008858377271573493,"topicId":44},
>     {"term":"september","probability":0.008727261943084602,"topicId":44},
>     {"term":"säsongen","probability":0.020736851111366656,"topicId":45},
>     {"term":"spelade","probability":0.01633330545811541,"topicId":45},
>     {"term":"matcher","probability":0.015635380079718257,"topicId":45},
>     {"term":"spelare","probability":0.01289782084286368,"topicId":45},
>     {"term":"vann","probability":0.01279265489050279,"topicId":45},
>     {"term":"from","probability":0.0786507643327494,"topicId":46},
>     {"term":"shift","probability":0.06423691443896841,"topicId":46},
>     {"term":"small","probability":0.05705289322090071,"topicId":46},
>     {"term":"value","probability":0.04409509137084651,"topicId":46},
>     {"term":"linegrey","probability":0.031785742083129814,"topicId":46},
>     {"term":"cerro","probability":0.24746595175338434,"topicId":47},
>     {"term":"berg","probability":0.07094748980724597,"topicId":47},
>     {"term":"kulle","probability":0.03736557629148667,"topicId":47},
>     {"term":"loma","probability":0.02492131478963847,"topicId":47},
>     {"term":"mexiko","probability":0.017170592223449976,"topicId":47},
>     {"term":"saint","probability":0.10011429904845326,"topicId":48},
>     {"term":"gogae","probability":0.05631896060381116,"topicId":48},
>     {"term":"dahr","probability":0.029576990277144867,"topicId":48},
>     {"term":"wadi","probability":0.026338775967562476,"topicId":48},
>     {"term":"sparbank","probability":0.011919538108387324,"topicId":48},
>     {"term":"bibcode","probability":0.02090893858307959,"topicId":49},
>     {"term":"star","probability":0.015297563921716708,"topicId":49},
>     {"term":"solen","probability":0.015181805530522997,"topicId":49},
>     {"term":"stars","probability":0.013835395518137523,"topicId":49},
>     {"term":"gånger","probability":0.013160601333366323,"topicId":49}

  

Plot visualization

In [ ]:
displayHTML(s"""
<!DOCTYPE html>
<meta charset="utf-8">
<style>

circle {
  fill: rgb(31, 119, 180);
  fill-opacity: 0.5;
  stroke: rgb(31, 119, 180);
  stroke-width: 1px;
}

.leaf circle {
  fill: #ff7f0e;
  fill-opacity: 1;
}

text {
  font: 14px sans-serif;
}

</style>
<body>
<script src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min.js"></script>
<script>

var json = {
 "name": "data",
 "children": [
  {
     "name": "topics",
     "children": [
      ${rawJson}
     ]
    }
   ]
};

var r = 1000,
    format = d3.format(",d"),
    fill = d3.scale.category20c();

var bubble = d3.layout.pack()
    .sort(null)
    .size([r, r])
    .padding(1.5);

var vis = d3.select("body").append("svg")
    .attr("width", r)
    .attr("height", r)
    .attr("class", "bubble");

  
var node = vis.selectAll("g.node")
    .data(bubble.nodes(classes(json))
    .filter(function(d) { return !d.children; }))
    .enter().append("g")
    .attr("class", "node")
    .attr("transform", function(d) { return "translate(" + d.x + "," + d.y + ")"; })
    color = d3.scale.category20();
  
  node.append("title")
      .text(function(d) { return d.className + ": " + format(d.value); });

  node.append("circle")
      .attr("r", function(d) { return d.r; })
      .style("fill", function(d) {return color(d.topicName);});

var text = node.append("text")
    .attr("text-anchor", "middle")
    .attr("dy", ".3em")
    .text(function(d) { return d.className.substring(0, d.r / 3)});
  
  text.append("tspan")
      .attr("dy", "1.2em")
      .attr("x", 0)
      .text(function(d) {return Math.ceil(d.value * 10000) /10000; });

// Returns a flattened hierarchy containing all leaf nodes under the root.
function classes(root) {
  var classes = [];

  function recurse(term, node) {
    if (node.children) node.children.forEach(function(child) { recurse(node.term, child); });
    else classes.push({topicName: node.topicId, className: node.term, value: node.probability});
  }

  recurse(null, root);
  return {children: classes};
}
</script>
""")